# Python script for data transformation 

## BRICARE:

BRICARE consists of 2 different types of files by year:

a. File after 2022 (2023-2024) = 79 kolom


b. File before 2022 (2019-2022) = 27 kolom


In [21]:
# Case Origin

origin_mappings = {
    "Sms": "SMS",
    "Telegram": "Telegram",
    "WebChat": "WebChat",
    "Sabrina": "Sabrina",
    "Twitter": "Twitter",
    "Facebook": "Facebook",
    "Instagram": "Instagram",
    "Pesan IB": "Pesan IB",
    "Email": "Email",
    "Video Banking": "Video Banking",
    "Media Cetak": "Media Cetak",
    "Media Online": "Media Online",
    "OJK": "OJK",
    "Lembaga Hukum Lainya": "Lembaga Hukum Lainya",
    "Media Umum Lainya": "Media Umum Lainya",
    "Phone": "Phone",
    "Surat": "Surat",
    "Walk In": "Walk In",
    "ËDC & Brilink": "EDC",
    "Brilink": "BRILink",
    "BRIMO": "BRIMO",
    "PPID": "PPID",
    "SP4N LAPOR": "SP4N LAPOR",
    "Media Konsumen": "Media Konsumen",
    "Bank Indonesia": "Bank Indonesia",
    "Ombudsman": "Ombudsman",
    "BPKN": "BPKN",
    "MMS": "MMS",
    "PKSS Oten": "PKSS Oten",
    "BRIPENS": "BRIPENS",
    "Ceria": "Ceria"
}
len(origin_mappings)

31

### File Type A


Data Extraction for File Type A must be 2 Files:


A.1 Columns (without "Details")


A.2 Details only 

Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['Create_Date','TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID"

- Columns shoud be mapped based on their Call_Type_ID:

['Produk','Jenis_Produk','Jenis_Laporan']

- PLEASE ADD CIF

#### File A.1 Columns (without "Details"). Please use this if the file is txt

In [7]:
import pandas as pd
import re
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_file(file_path):

    data = []
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}')

    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')

            date_index = next(i for i, part in enumerate(parts) if date_pattern.match(part))

            ticket_id = parts[0] 
            call_type_id = parts[1]  
            description = ';'.join(parts[2:date_index])  
            create_date = parts[date_index]  

      
            data.append([ticket_id, call_type_id, description, create_date] + parts[date_index + 1:])


    df = pd.DataFrame(data, columns=column_names)

    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

    return df


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.txt"

df = parse_file(file_path)
df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
   

df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())



df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_uat{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

Data saved to bricare_uat20230101_20230101.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('NULL', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('None', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with

#### File A.1 Columns (without "Details"). Please use this if the file is csv

In [46]:
import pandas as pd
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) > 78:
                description = ';'.join(parts[2:-75])
                new_parts = parts[:2] + [description] + parts[-75:]
                data.append(new_parts)
            else:
                data.append(parts)
    
    df = pd.DataFrame(data, columns=column_names)
    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
    
    df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
    df.fillna('', inplace=True)
    df = df.replace(['0', 0], '')

    columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
    
    df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())

    return df
file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.csv"
df = parse_csv(file_path)

startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"
df.to_csv(filename, index=False)

filename


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


'bricare_20230101_20230101.csv'

## Call Type di tes MMS

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Data Mapping - Case_Type.csv"
df=pd.read_csv(path)
# case_types = ['8634', '8635', '8636', '8623', '8624', '8628', '8629', '8630', '8412', '8625', '8615', '8616', '8617', '8618', '8619', '8638', '8620']

case_types = ['8412']
# Filter the dataframe
df = df[df['Case Types'].isin(case_types)]
df['Segment'] = df['Segment'].replace('All', '')
df
df.to_csv(r'C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Case_Type_MMS2.csv', index=False)

## Call Type Server

In [28]:
## use the fix document

import pandas as pd
import re

# Load the Excel file
file_path = r"D:\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (5).xlsx"
df = pd.read_excel(file_path, sheet_name='FIX Upload - Update Here')

#df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old', 'T4.3 - Technical Team', 'T5.1 - Third Party', 'T5.2 - Third Party', 'T5.3 - Third Party', 'Jenis Produk dan atau Layanan LKPBU', 'Kategori Permasalahan LKPBU', 'Jenis Transaksi LKPBU', 'Penyebab Pengaduan LKPBU', 'Jenis Produk LBUT', 'Sandi Jenis  Produk LBUT', 'Sebab Pengaduan LBUT', 'Sandi Penyebab LBUT'])


# df=df.drop(columns=['Actual SLA\n(days) Old','Target SLA \n(days) Old', 'BRI Notes','Team Name', 'New SLA', 'Status', 'crosscek', 'GIC', 'Maker', 'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data'])

columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
df[columns_to_replace] = df[columns_to_replace].replace('-', '')

if 'Active' in df.columns:
    df['Active'] = df['Active'].astype(str).str.upper()


df['Detail BRICare'] = df['Detail BRICare'].fillna('-')
df

def clean_case_types(value):
    if pd.isna(value) or value == '':
        return '0'
    match = re.match(r'(\d{4})-(\d{2})', str(value))
    if match:
        year, month = match.groups()
        return f'{year}-{int(month)}'
    value = re.sub(r'\.0$', '', str(value))
    return value

df['Case Types'] = df['Case Types'].apply(clean_case_types) 
df['external_id'] = df['Case Types']
df['send to drone'] = df['send to drone'].astype(str)

# Then convert all values in 'Column2' to uppercase
df['send to drone'] = df['send to drone'].str.upper()


output_path = 'Cleaned4_Call_Type.xlsx'
output_path2 = 'Cleaned4_Call_Type.csv'
df.to_excel(output_path, index=False)
df.to_csv(output_path2, index=False)


In [10]:
# Check if all call types are covered

import pandas as pd

# Define the paths to your CSV files
file1_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto.csv"
# file2_path = r"D:\dataquality2\Combined3_Cleaned_Call_Type.csv"

file2_path = r"D:\dataquality2\Cleaned4_Call_Type.csv"

# Load the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Extract the 'Case Types' column from each DataFrame
case_types_1 = df1['calltypeidtxt'].astype(str)
case_types_2 = df2['Case Types'].astype(str)

# Convert to sets for comparison
set_case_types_1 = set(case_types_1)
set_case_types_2 = set(case_types_2)

# Find values in file1 that are missing in file2
missing_values = set_case_types_1 - set_case_types_2

# Convert the missing values set to a DataFrame
missing_values_df = pd.DataFrame(list(missing_values), columns=['Missing Case Types'])

# Save the missing values to a new CSV file
# missing_values_df.to_csv('Missing_Case_Types.csv', index=False)
missing_values_df
# print(f"Missing values from file1 that are not in file2 have been saved to 'Missing_Case_Types.csv'")



,Missing Case Types


In [11]:
# Remove duplicates if any

# remove any duplicates

import pandas as pd

# Path to the CSV file
# path = r"D:\dataquality2\Combined3_Cleaned_Call_Type.csv"

path = r"D:\dataquality2\Cleaned4_Call_Type.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(path)

# Step 1: Check for duplicates in 'Case Types'
unique_case_types = df['Case Types'].unique()
num_unique_case_types = len(unique_case_types)

print(f"Number of unique case types: {num_unique_case_types}")

# Find and print duplicate values in the 'Case Types' column
duplicate_case_types = df['Case Types'][df['Case Types'].duplicated()].unique()
print("Duplicate case types:")
for case_type in duplicate_case_types:
    print(case_type)

# Step 2: Identify and remove rows where 'Case Types' is a duplicate and 'Active' is False

# Identify duplicate 'Case Types' values
duplicates = df[df.duplicated(subset=['Case Types'], keep=False)]

# Find rows where 'Case Types' is a duplicate and 'Active' is False
to_drop = duplicates[(duplicates['Active'] == False)]

# Drop these rows from the original DataFrame
df_cleaned = df.drop(to_drop.index)

# Optionally, save the cleaned DataFrame to a new CSV file
output_path = r"D:\dataquality2\Combined_Cleaned_Call_Type.csv"
df_cleaned.to_csv(output_path, index=False)

# Print the number of rows dropped
print(f"Number of rows dropped: {len(to_drop)}")

# Display the first few rows of the cleaned DataFrame
print(df_cleaned.head())


Number of unique case types: 520
Duplicate case types:
Number of rows dropped: 0
   No Case Types  Active               User Grouping_New        Segment  \
0   1       1000    True               Resolution Center  Individu Umum   
1   2       1002    True               Resolution Center  Individu Umum   
2   3       1003    True                 Digital Banking  Individu Umum   
3   4       1005    True  Card, Digital Lending & Assset  Individu Umum   
4   5       1006    True                Operational Risk  Individu Umum   

  Product    Sub Product                 Case Category  \
0   Loans  KTA - Digital                       Inquiry   
1   Loans  KTA - Digital  Products / Promotion Inquiry   
2   Loans  KTA - Digital                       Request   
3   Loans  KTA - Digital                       Request   
4   Loans  KTA - Digital                       Request   

                                Old Case Description  \
0  CERIA - Status Pengajuan, Aplikasi, Cara & Syarat   
1      

: 

In [ ]:
# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')

# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None

# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'Level 1 OLA',
#     'Level 2 OLA',
#     'Level 3 OLA',
#     'Level 4 OLA',
#     'T1 - Agent CCT/Frontliner',
#     'Total SLA',
#     'New SLA'
# ]
        
# # Function to remove .0 from float values and handle empty strings
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     if value == '':
#         return np.nan
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero).astype('Int64')

# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # df = df.drop(columns=columns_to_check)
# df.to_csv('Cleaned3_Call_Type_missing.csv', index=False)


In [ ]:
# # Cleaning missing call type

# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')



# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Ensure columns_to_format exist in df
# existing_columns_to_format = [col for col in columns_to_format if col in df.columns]

# # Function to format columns
# def format_columns(col):
#     try:
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col

# # Apply the formatting function to selected columns
# df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)

# # Ensure the 'active' column exists before modifying
# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # Convert relevant columns to numeric for addition if they exist
# numeric_columns = [
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team'
# ]

# existing_numeric_columns = [col for col in numeric_columns if col in df.columns]

# for col in existing_numeric_columns:
#     df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# # Combine columns with addition
# if 'T2.2 - Supporting Unit' in df.columns and 'T2.3 - Supporting Unit' in df.columns:
#     df['T2.2 - Supporting Unit'] += df['T2.3 - Supporting Unit']
#     df = df.drop(columns=['T2.3 - Supporting Unit'])

# if 'T3.1 - Product Owner' in df.columns and 'T3.2 - Product Owner' in df.columns and 'T3.3 - Product Owner' in df.columns:
#     df['T3.1 - Product Owner'] += df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
#     df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# if 'T4.1 - Technical Team' in df.columns and 'T4.2 - Technical Team' in df.columns:
#     df['T4.1 - Technical Team'] += df['T4.2 - Technical Team']
#     df = df.drop(columns=['T4.2 - Technical Team'])

# # Format the combined columns to remove .0
# combined_columns = ['T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team']
# for col in combined_columns:
#     if col in df.columns:
#         df[col] = df[col].apply(format_columns)

# # Columns to replace '-' with empty string
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Ensure columns_to_replace exist in df
# existing_columns_to_replace = [col for col in columns_to_replace if col in df.columns]

# # Replace '-' with empty string in selected columns
# df[existing_columns_to_replace] = df[existing_columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# # Add the 'external_id' column
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)
    
# # Save the cleaned data to a CSV file
# df.to_csv('Cleaned2_Call_Type_missing.csv', index=False)




In [9]:
# import pandas as pd
# import re

# # Load the Excel file
# file_path = r"C:\Users\maste\Downloads\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (2).xlsx"
# df = pd.read_excel(file_path, sheet_name='FIX Upload - Update Here')
# # df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old', 'T4.3 - Technical Team', 'T5.1 - Third Party', 'T5.2 - Third Party', 'T5.3 - Third Party', 'Jenis Produk dan atau Layanan LKPBU', 'Kategori Permasalahan LKPBU', 'Jenis Transaksi LKPBU', 'Penyebab Pengaduan LKPBU', 'Jenis Produk LBUT', 'Sandi Jenis  Produk LBUT', 'Sebab Pengaduan LBUT', 'Sandi Penyebab LBUT'])

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None

# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # df = df.drop(columns=columns_to_check)

# # Replace '-' with empty string in selected columns
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# df[columns_to_replace] = df[columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'Level 1 OLA',
#     'Level 2 OLA',
#     'Level 3 OLA',
#     'Level 4 OLA',
#     'T1 - Agent CCT/Frontliner',
#     'Total SLA',
#     'New SLA'
    
# ]
		
# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero).astype('Int64')


# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# df['Detail BRICare'] = df['Detail BRICare'].fillna('-')
# # Save the modified dataframe to a new Excel file
# output_path = 'Cleaned3_Call_Type.xlsx'
# output_path2 = 'Cleaned3_Call_Type.csv'
# df.to_excel(output_path, index=False)
# df.to_csv(output_path2, index=False)

# # Output path of the updated file
# print(output_path)
# print(output_path2)


In [7]:
# import pandas as pd
# import numpy as np
# import re

# path = r"C:\Users\maste\Downloads\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan.xlsx"
# # path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# pd.set_option('display.max_columns', None)  
# # pd.set_option('display.max_rows', None) 
# df = pd.read_excel(path)
# df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old','T4.3 - Technical Team','T5.1 - Third Party','T5.2 - Third Party','T5.3 - Third Party','Jenis Produk dan atau Layanan LKPBU','Kategori Permasalahan LKPBU','Jenis Transaksi LKPBU','Penyebab Pengaduan LKPBU','Jenis Produk LBUT','Sandi Jenis  Produk LBUT','Sebab Pengaduan LBUT','Sandi Penyebab LBUT'])

# df = df.replace(0, np.nan)
# df = df.fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to format columns
# def format_columns(col):
#     try:
#         # Convert to float to handle empty strings and non-numeric values
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col  # Return original value if conversion fails

# # Apply the formatting function to selected columns
# df[columns_to_format] = df[columns_to_format].applymap(format_columns)
# df['active'] = df['active'].astype(str).str.upper()

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None


# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # Apply the function to each row

# df = df.drop(columns=columns_to_check)
# # ====================================================================================================================Convert relevant columns to numeric for addition
# # df['T2.2 - Supporting Unit'] = pd.to_numeric(df['T2.2 - Supporting Unit'], errors='coerce').fillna(0)
# # df['T2.3 - Supporting Unit'] = pd.to_numeric(df['T2.3 - Supporting Unit'], errors='coerce').fillna(0)
# # df['T3.1 - Product Owner'] = pd.to_numeric(df['T3.1 - Product Owner'], errors='coerce').fillna(0)
# # df['T3.2 - Product Owner'] = pd.to_numeric(df['T3.2 - Product Owner'], errors='coerce').fillna(0)
# # df['T3.3 - Product Owner'] = pd.to_numeric(df['T3.3 - Product Owner'], errors='coerce').fillna(0)
# # df['T4.1 - Technical Team'] = pd.to_numeric(df['T4.1 - Technical Team'], errors='coerce').fillna(0)
# # df['T4.2 - Technical Team'] = pd.to_numeric(df['T4.2 - Technical Team'], errors='coerce').fillna(0)

# # # Combine columns with addition
# # df['T2.2 - Supporting Unit'] = df['T2.2 - Supporting Unit'] + df['T2.3 - Supporting Unit']
# # df = df.drop(columns=['T2.3 - Supporting Unit'])

# # df['T3.1 - Product Owner'] = df['T3.1 - Product Owner'] + df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
# # df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# # df['T4.1 - Technical Team'] = df['T4.1 - Technical Team'] + df['T4.2 - Technical Team']
# # df = df.drop(columns=['T4.2 - Technical Team'])

# # # Format the combined columns to remove .0
# # df['T2.2 - Supporting Unit'] = df['T2.2 - Supporting Unit'].apply(format_columns)
# # df['T3.1 - Product Owner'] = df['T3.1 - Product Owner'].apply(format_columns)
# # df['T4.1 - Technical Team'] = df['T4.1 - Technical Team'].apply(format_columns)
# # =============================================================================================================================================================
# # Columns to replace '-' with empty string



# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Replace '-' with empty string in selected columns
# df[columns_to_replace] = df[columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)

# df.to_csv('Cleaned3_Call_Type.csv', index=False)
# # df['external_id'].unique()


C:\Users\maste\AppData\Local\Temp\ipykernel_52768\1505204889.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_format] = df[columns_to_format].applymap(format_columns)


In [29]:
# # Cleaning missing call type

# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')



# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Ensure columns_to_format exist in df
# existing_columns_to_format = [col for col in columns_to_format if col in df.columns]

# # Function to format columns
# def format_columns(col):
#     try:
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col

# # Apply the formatting function to selected columns
# df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)

# # Ensure the 'active' column exists before modifying
# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # Convert relevant columns to numeric for addition if they exist
# numeric_columns = [
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team'
# ]

# existing_numeric_columns = [col for col in numeric_columns if col in df.columns]

# for col in existing_numeric_columns:
#     df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# # Combine columns with addition
# if 'T2.2 - Supporting Unit' in df.columns and 'T2.3 - Supporting Unit' in df.columns:
#     df['T2.2 - Supporting Unit'] += df['T2.3 - Supporting Unit']
#     df = df.drop(columns=['T2.3 - Supporting Unit'])

# if 'T3.1 - Product Owner' in df.columns and 'T3.2 - Product Owner' in df.columns and 'T3.3 - Product Owner' in df.columns:
#     df['T3.1 - Product Owner'] += df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
#     df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# if 'T4.1 - Technical Team' in df.columns and 'T4.2 - Technical Team' in df.columns:
#     df['T4.1 - Technical Team'] += df['T4.2 - Technical Team']
#     df = df.drop(columns=['T4.2 - Technical Team'])

# # Format the combined columns to remove .0
# combined_columns = ['T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team']
# for col in combined_columns:
#     if col in df.columns:
#         df[col] = df[col].apply(format_columns)

# # Columns to replace '-' with empty string
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Ensure columns_to_replace exist in df
# existing_columns_to_replace = [col for col in columns_to_replace if col in df.columns]

# # Replace '-' with empty string in selected columns
# df[existing_columns_to_replace] = df[existing_columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# # Add the 'external_id' column
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)
    
# # Save the cleaned data to a CSV file
# df.to_csv('Cleaned2_Call_Type_missing.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_62668\3534049870.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)


In [6]:
# check if the missing call type and Cleaned file not overlapping case types


import pandas as pd

# Define the paths to your two CSV files
file1_path = r"D:\dataquality2\Cleaned3_Call_Type_missing.csv"
file2_path = r"D:\dataquality2\Cleaned3_Call_Type.csv"

# Load the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Extract the 'Case Types' column from each DataFrame
case_types_1 = df1['Case Types'].astype(str)
case_types_2 = df2['Case Types'].astype(str)

# Check for duplicates within each file
duplicates_1 = case_types_1[case_types_1.duplicated()]
duplicates_2 = case_types_2[case_types_2.duplicated()]

# Print duplicates within each file
if not duplicates_1.empty:
    print("Duplicates in File 1:")
    print(duplicates_1)
else:
    print("No duplicates in File 1.")

if not duplicates_2.empty:
    print("Duplicates in File 2:")
    print(duplicates_2)
else:
    print("No duplicates in File 2.")

# Check if each file has unique values in the 'Case Types' column
unique_case_types_1 = case_types_1.nunique() == len(case_types_1)
unique_case_types_2 = case_types_2.nunique() == len(case_types_2)

print(f"File 1 has all unique values: {unique_case_types_1}")
print(f"File 2 has all unique values: {unique_case_types_2}")

# Convert to sets for comparison
set_case_types_1 = set(case_types_1)
set_case_types_2 = set(case_types_2)

# Find overlapping values
overlapping_values = set_case_types_1.intersection(set_case_types_2)

if overlapping_values:
    print("There are overlapping values in 'Case Types' between the two files:")
    print(overlapping_values)
else:
    print("No overlapping values in 'Case Types' between the two files.")

# Optionally, save the results to CSV files if needed
# pd.Series(list(overlapping_values)).to_csv('Overlapping_Values.csv', index=False)

if not duplicates_1.empty:
    duplicates_1.to_csv('Duplicates_in_File1.csv', index=False)
if not duplicates_2.empty:
    duplicates_2.to_csv('Duplicates_in_File2.csv', index=False)


No duplicates in File 1.
Duplicates in File 2:
443    1201
Name: Case Types, dtype: object
File 1 has all unique values: True
File 2 has all unique values: False
No overlapping values in 'Case Types' between the two files.


In [7]:
# Combine Cleaned Call Type with Missing Call Type:

import pandas as pd

# Load the uploaded CSV files
file_1_path = r"D:\dataquality2\Cleaned3_Call_Type_missing.csv"
file_2_path = r"D:\dataquality2\Cleaned3_Call_Type.csv"

df1 = pd.read_csv(file_1_path)
df2 = pd.read_csv(file_2_path)

# Identify columns in File 2 that are not in File 1
missing_in_file_1 = set(df2.columns) - set(df1.columns)

# Ensure all columns from the larger dataframe are present in the smaller dataframe before combining
# Add missing columns to df1 with NaN values
for col in missing_in_file_1:
    df1[col] = pd.NA

# Reorder columns in df1 to match the order of columns in df2
df1 = df1[df2.columns]

# Combine the dataframes
combined_df = pd.concat([df1, df2], ignore_index=True)

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     combined_df[column] = combined_df[column].apply(remove_decimal_zero)


# Columns to format
columns_to_format = [
    'Level 1 OLA',
    'Level 2 OLA',
    'Level 3 OLA',
    'Level 4 OLA',
    'T1 - Agent CCT/Frontliner',
    'Total SLA',
    'New SLA'
]
        
# Function to remove .0 from float values and handle empty strings
def remove_decimal_zero(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    if value == '':
        return np.nan
    return value

# Apply function to specified columns
for column in columns_to_format:
    combined_df[column] = combined_df[column].apply(remove_decimal_zero).astype('Int64')



# Save the combined dataframe to a new CSV file
combined_file_path = 'Combined3_Cleaned_Call_Type.csv'
combined_file_path2 = 'Combined3_Cleaned_Call_Type.xlsx'
combined_df.to_csv(combined_file_path, index=False)
combined_df.to_excel(combined_file_path2, index=False)




C:\Users\maste\AppData\Local\Temp\ipykernel_1100\104105547.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [4]:
# # Compare the call type Pak SUyanto/ bricare with master call type data


# import pandas as pd


# calltype_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto.csv")
# # final_df = pd.read_excel(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan.xlsx", sheet_name=None)
# final_call_type_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto2.csv")

# # final_call_type_df = final_df['Full Call Type Fix']


# calltype_ids = calltype_df['calltypeidtxt'].astype(str)
# case_types = final_call_type_df['Case Types'].astype(str)


# missing_calltype_ids = calltype_ids[~calltype_ids.isin(case_types)]

# missing_calltype_ids_df = missing_calltype_ids.to_frame(name='Missing Call Type IDs')


# # missing_calltype_ids_df.to_csv(r"C:\Users\maste\Downloads\missing_calltype_ids.csv", index=False)

# # print("Missing call type IDs have been saved to missing_calltype_ids.csv")
# missing_calltype_ids_df

,Missing Call Type IDs


In [8]:
# # Add Escalation Team


# import pandas as pd

# # Load the provided CSV files
# combined_cleaned_call_type = pd.read_csv(r"D:\dataquality2\Combined_Cleaned_Call_Type.csv")
# sla_ola_new_user_grouping = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan - Appendix Team Name.csv")

# # Create a mapping from the SLA & OLA dataframe
# team_mapping = sla_ola_new_user_grouping.set_index('/xc//.vs')['Full Name / Queue Name'].to_dict()

# # Adding more mappings based on the provided information
# additional_mappings = {
#     'RO BRILink': 'RO - Business Channel BRILink',
#     'RO ITE': 'RO - Information, Technology & E-Channel',
#     'BRANCH(Uker)': 'Branch - Operational Unit',
#     'CDD(CLF-1DATAM)': 'CDD - Clf-1 Credit Card Init - DATAM',
#     'CDD(CLF-1 DATAM)': 'CDD - Clf-1 Credit Card Init - DATAM',
#     'CCTFrontliner': 'CCT - Contact Center Frontliner'
# }

# # Update the original mapping with additional mappings
# team_mapping.update(additional_mappings)

# # Function to clean team names
# def clean_team_name_further(team_name):
#     if pd.isna(team_name):
#         return team_name
#     # Remove specific patterns and clean up the string
#     team_name = team_name.replace("Agent", "").replace("Include", "").strip()
#     team_name = team_name.lstrip("-").strip()
#     # Remove spaces within names for better mapping
#     team_name = team_name.replace(" ", "")
#     team_name = ' '.join(word.strip('-').strip() for word in team_name.split())
#     return team_name

# # Applying the cleaning function to the correct column 'Team Name'
# combined_cleaned_call_type['Cleaned Team Name'] = combined_cleaned_call_type['Team Name'].apply(clean_team_name_further)

# # Function to clean and translate team names with specific rules
# def clean_and_translate_team_name_with_specific_rules(team_name, mapping):
#     if pd.isna(team_name) or team_name == "" or isinstance(team_name, float):
#         return [None, None, None, None]
#     team_name_cleaned = ' '.join(word.strip('-').strip() for word in str(team_name).split())
#     # Specific rule: if the team name contains "T1-CCT/Frontliner", return None for all levels
#     if "T1-CCT/Frontliner" in team_name:
#         return [None, None, None, None]
#     # Remove "Insurance Company" and "Vendor"
#     team_name_cleaned = team_name_cleaned.replace("InsuranceCompany", "").replace("Vendor", "").strip()
#     # Translate using the mapping
#     teams = [mapping.get(t.strip(), t.strip()) for t in team_name_cleaned.split(',')]
#     return teams + [None] * (4 - len(teams))

# # Reapplying the cleaning and translation process with the specific rules
# levels = combined_cleaned_call_type['Cleaned Team Name'].apply(lambda x: clean_and_translate_team_name_with_specific_rules(x, team_mapping))

# # Ensuring the result is a DataFrame with correct None values
# level_columns = pd.DataFrame(levels.tolist(), columns=['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team'], index=combined_cleaned_call_type.index)

# # Creating placeholders for OLA levels
# ola_columns = pd.DataFrame(levels.tolist(), columns=['Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA'], index=combined_cleaned_call_type.index)

# # Assigning the translated names to the escalation levels and placeholders to OLA levels
# combined_cleaned_call_type = pd.concat([combined_cleaned_call_type, level_columns, ola_columns], axis=1)

# # Manually correcting specific known issues
# index_to_correct = combined_cleaned_call_type['Cleaned Team Name'] == 'T1-CCT/Frontliner'
# combined_cleaned_call_type.loc[index_to_correct, ['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA']] = None

# index_to_correct = combined_cleaned_call_type['Cleaned Team Name'].str.contains('CDD\(ACUOTO\),CDD\(ACUCHARGEBACK\),CDD\(CLF-1DATAM\)', na=False)
# combined_cleaned_call_type.loc[index_to_correct, ['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA']] = [
#     'CDD - Auth,Chrgbck & UserCo - OTO',
#     'CDD - Auth,Chrgbck & UserCo - CHARGEBACK',
#     'CDD - Clf-1 Credit Card Init - DATAM',
#     None,
#     'CDD - Auth,Chrgbck & UserCo - OTO',
#     'CDD - Auth,Chrgbck & UserCo - CHARGEBACK',
#     'CDD - Clf-1 Credit Card Init - DATAM',
#     None
# ]

# # Keeping all specified columns
# columns_to_keep = [
#     'No', 'Case Types', 'active', 'User Grouping_New', 'Segment', 'Product', 'Sub Product', 'Case Category', 'Case Description',
#     'New Case Description', 'Required Sub-Description', 'Sub', 'Doc Intake', 'List Kebutuhan Dokumen', 'Suggestion', 'Suggestion - Breakdown',
#     'Level of Effort', 'Range of Days \n(Actual SLA)', 'Actual SLA\n(days)', 'Target SLA \n(days)', 'BRI Notes', 'Team Name', 'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team', 'Total SLA', 'New SLA', 'Status', 'GIC', 'Maker',
#     'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data', 'Detail BRICare', 'Unnamed: 58', 'external_id', 'Cleaned Team Name',
#     'Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA'
# ]

# # Selecting the specified columns
# final_dataset_with_all_columns = combined_cleaned_call_type[columns_to_keep]

# # Saving the final dataset to a new CSV file
# final_dataset_with_all_columns.to_csv('Final_Cleaned_Escalation_Teams_All_Columns.csv', index=False)



In [54]:
import pandas as pd

path=r"C:\Users\maste\Downloads\Final_Revised_Combined_Cleaned_Call_Type_with_Specific_Rules_Corrected.csv"

df = pd.read_csv(path)
columns_to_keep = [
    'No', 'Case Types', 'active', 'User Grouping_New', 'Segment', 'Product', 'Sub Product', 'Case Category', 'Case Description',
    'New Case Description', 'Required Sub-Description', 'Sub', 'Doc Intake', 'List Kebutuhan Dokumen', 'Suggestion', 'Suggestion - Breakdown',
    'Level of Effort', 'Range of Days \n(Actual SLA)', 'Actual SLA\n(days)', 'Target SLA \n(days)', 'BRI Notes', 'Team Name', 'T1 - Agent CCT/Frontliner',
    'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team', 'Total SLA', 'New SLA', 'Status', 'GIC', 'Maker',
    'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data', 'Detail BRICare', 'Unnamed: 58', 'external_id', 'Cleaned Team Name',
    'Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team'
]

# Selecting the specified columns
df = df[columns_to_keep]

columns_to_process = [
    'Level 1 Escalation Team',
    'Level 2 Escalation Team',
    'Level 3 Escalation Team',
    'Level 4 Escalation Team'
]

# Replace "Vendor" and "Insurance Company" with an empty string in the specified columns
for column in columns_to_process:
    df[column] = df[column].replace(['Vendor', 'Insurance Company'], '')
    
df['Detail BRICare'] = df['Detail BRICare'].fillna('-')

df.to_csv('final_cleaned_callType.csv', index=False)
# Save to Excel
df.to_excel('final_cleaned_callType.xlsx', index=False)

#### Call type mapping for columns 'Produk', 'Jenis Produk', 'Jenis Laporan'

In [8]:
import pandas as pd


user_dataset_path = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
user_df = pd.read_csv(user_dataset_path)
master_df_path = r"D:\dataquality\master_calltype.csv"
master_df = pd.read_csv(master_df_path)


master_df = master_df.rename(columns={
    'Case Types': 'Call_Type_ID', 
    'Product': 'Produk', 
    'Sub Product': 'Jenis_Produk', 
    'Case Category': 'Jenis_Laporan'
})



user_df['Call_Type_ID'] = user_df['Call_Type_ID'].astype(str)
master_df['Call_Type_ID'] = master_df['Call_Type_ID'].astype(str)


merged_df = pd.merge(user_df, master_df[['Call_Type_ID', 'Produk', 'Jenis_Produk', 'Jenis_Laporan']], on='Call_Type_ID', how='left')

user_df['Produk'] = merged_df['Produk_y']
user_df['Jenis_Produk'] = merged_df['Jenis_Produk_y']
user_df['Jenis_Laporan'] = merged_df['Jenis_Laporan_y']



user_df.to_csv(user_dataset_path, index=False)

#### File A.2 Details only. Please use this if the file is txt

In [9]:
import pandas as pd

def process_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:  
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]  
                current_entry.append(parts[3].strip())  
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries


def remove_bom_and_strip(df):
    return df.applymap(lambda x: x.replace('\ufeff', '').strip() if isinstance(x, str) else x)


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.txt"
processed_data = process_text_data(file_path)


df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])

if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]

# df_final=df_final.iloc[:10]
df_final.iloc[:10].to_csv('details_uat_20230101_20230101.csv', index=False)



#### File A.2 Details only. Please use this if the file is csv

In [49]:
import pandas as pd

def process_csv_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]
                current_entry.append(parts[3].strip())
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries

file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.csv"
processed_data = process_csv_data(file_path)

df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])


if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]


df_final = df_final.iloc[:10]
output_path = "details_20230101_20230101.csv"
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to details_20230101_20230101.csv


#### Merge the file A.1 and file A.2

In [50]:
import pandas as pd

#just take 10 lines for an example
path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df.iloc[:10].to_csv(path,index=False)

In [12]:
import pandas as pd


file_path_1 = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
file_path_2 = r"D:\dataquality2\details_uat_20230101_20230101.csv"


df_tenline_bricare = pd.read_csv(file_path_1)
df_detail_bricare_10line = pd.read_csv(file_path_2)

df_detail_bricare_10line.columns = ['Ticket_ID', 'Details']

merged_df = pd.merge(df_tenline_bricare, df_detail_bricare_10line, on='Ticket_ID', how='left')


output_file_path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"


column_to_move="Details"
merged_df = merged_df[[col for col in merged_df if col != column_to_move][:3] + [column_to_move] + [col for col in merged_df if col != column_to_move][3:]] 

merged_df.to_csv(output_file_path, index=False)

### File Type B


Data Extraction for File Type B (27 columns)


Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['TanggalClosed', 'tanggalTransaksi','Create_Date']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID" if any

- PLEASE ADD CIF



In [52]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]


path = r"C:\Users\maste\Downloads\BRICARE_25042024 masking.csv"


data = pd.read_csv(path, delimiter=';')


data['Column1'] = data['Column1'].astype(str)
data_cleaned = data[data['Column1'].str.match(r'TTB\d+')]

data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)
data_cleaned = data_cleaned[data_cleaned['Column2'].str.match(r'^\d{4}$')]

data_cleaned['Column4'] = pd.to_datetime(data_cleaned['Column4'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['Column4'])


data_to_drop = ['Column28', 'Column29', 'Column30', 'Column31', 'Column32']
data_cleaned = data_cleaned.drop(columns=data_to_drop)


if len(data_cleaned.columns) <= len(column_list):
    data_cleaned.columns = column_list[:len(data_cleaned.columns)]


data_cleaned.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
data_cleaned.fillna('', inplace=True)
data_cleaned = data_cleaned.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    data_cleaned[column] = pd.to_datetime(data_cleaned[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    data_cleaned[column] = data_cleaned[column].apply(lambda x: '' if pd.isna(x) else x)

# Just take 10 lines for an example
data_cleaned = data_cleaned.iloc[:10]


data_cleaned.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(data_cleaned['Create_Date']))
enddate = pd.Timestamp(max(data_cleaned['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


data_cleaned.to_csv(filename, index=False)

print(f"Data saved to {filename}")


C:\Users\maste\AppData\Local\Temp\ipykernel_33500\3452762791.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)


Data saved to bricare_20200101_20200101.csv


## Check the number of columns

In [55]:

path1=r"D:\dataquality\bricare_20230101_20230101.csv"
path2=r"D:\dataquality\bricare_20200101_20200101.csv"

df1=pd.read_csv(path1)
df2=pd.read_csv(path2)
print(len(df1.columns))
print(len(df2.columns))

print(df1.columns)
print(df2.columns)

79
27
Index(['Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Details', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name', 'Jenis_Produk', 'Last_Modified_By', 'Merchant_ID',
       'Modified_Date', 'NOTAS', 'Produk', 'SLA_Status', 'TID',
       'tanggalAttachmentDone', 'Tgl_Assigned', 'Tgl_Eskalasi', 'AnalisaSkils',
       'Attachment_', 'Bank_BRI', 'Biaya_Admin', 'Suku_Bunga', 'Bunga',
       'Butuh_Attachment', 'Cicilan', 'Hasil_Kunjungan', 'Log_Name',
       'MMS_Ticket_Id', 'Mass_Ticket_Upload_Flag', 'Nama_Supervisor',
       'Nama_TL', 'Nama_Wakabag', 'Nasabah_Prioritas', 'Notify_By',
       'Organization', 'Output_Settlement', 'phone_

## To Compare two files

In [18]:
# Load the newly uploaded files for detailed comparison
processed_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"
expected_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"

# Read the files
processed_df_newest = pd.read_csv(processed_file_path_newest)
expected_df_newest = pd.read_csv(expected_file_path_newest)

# Check for exact match first
exact_match = processed_df_newest.equals(expected_df_newest)

# Initialize a dictionary to store detailed comparison results
comparison_details = {
    'shape_match': processed_df_newest.shape == expected_df_newest.shape,
    'columns_match': processed_df_newest.columns.equals(expected_df_newest.columns),
    'column_differences': {},
    'value_differences': {}
}

# Compare each aspect if exact match is not true
if not exact_match:
    # Check shape
    if not comparison_details['shape_match']:
        comparison_details['shape_details'] = {
            'processed_shape': processed_df_newest.shape,
            'expected_shape': expected_df_newest.shape
        }
    
    # Check columns
    if not comparison_details['columns_match']:
        comparison_details['column_details'] = {
            'processed_columns': processed_df_newest.columns.tolist(),
            'expected_columns': expected_df_newest.columns.tolist()
        }

    # Check column-by-column values
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            comparison_details['column_differences'][column] = processed_df_newest[column].compare(expected_df_newest[column])

# Summarize value differences
if not comparison_details['columns_match']:
    value_differences = {}
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            value_differences[column] = processed_df_newest[column].compare(expected_df_newest[column])
    comparison_details['value_differences'] = value_differences

comparison_details


{'shape_match': True,
 'columns_match': True,
 'column_differences': {},
 'value_differences': {}}

# To collect all Error logs in a path

directory_path is the error logs path as well as the location where the combined error log file will be saved


In [41]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Normalize the 'TICKET_ID' to ensure duplicates are identified
    combined_df['TICKET_ID'] = combined_df['TICKET_ID'].str.upper()

    combined_df = combined_df.drop_duplicates(subset='TICKET_ID')

    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"Combined error logs saved to: {output_path}")
    return output_path, combined_df

directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
output_path, combined_df = combine_error_logs(directory_path)


Combined error logs saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv


In [3]:
import os
import pandas as pd


directory = r"C:\Users\maste\OneDrive\file\log\log"


combined_df = pd.DataFrame()


for filename in os.listdir(directory):
    if 'error' in filename:
        file_path = os.path.join(directory, filename)
        try:
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except Exception as e:
            print(f"Could not read {filename}: {e}")


combined_file_path = os.path.join(directory, 'combined_errors.csv')
combined_df.to_csv(combined_file_path, index=False)

print(f"Combined file saved to {combined_file_path}")


Combined file saved to C:\Users\maste\OneDrive\file\log\log\combined_errors.csv


# To cleanse user data

In [49]:
import pandas as pd
from io import StringIO
import html


# Function to decode HTML entities in a DataFrame
def decode_html_entities(df):
    df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)
    return df_decoded


file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\USER.txt" 

# Read the raw file content
with open(file_path, 'r') as file:
    raw_data = file.readlines()
    
# Split headers and data    
header = raw_data[0].replace("&#124;", "|").strip()
data = [line.replace("&#124;", "|").strip() for line in raw_data[1:]]

# Read the cleaned data into a pandas DataFrame
df_cleaned = pd.DataFrame([line.split('|') for line in data], columns=header.split('|'))

# Decode HTML entities
df_cleaned = decode_html_entities(df_cleaned)
df_cleaned

# Remove the quotes in Dataframe
def remove_quotes(df):
    df.columns = df.columns.str.replace('"', '')
    df = df.apply(lambda col: col.str.replace('"', '', regex=True))
    return df

df_cleaned = remove_quotes(df_cleaned)
df_cleaned
df_cleaned.to_csv("user_data.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_43760\4048578058.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)


# Working on the data

In [3]:
import pandas as pd

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Option 1: Check for quoted fields or use different delimiter
try:
    df = pd.read_csv(path, sep=',', on_bad_lines='skip') 
except pd.errors.ParserError as e:
    print(f"Error parsing file: {e}")


C:\Users\maste\AppData\Local\Temp\ipykernel_15156\3036651791.py:7: DtypeWarning: Columns (1,8,17,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', on_bad_lines='skip')


In [4]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"
]

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Initialize lists to store data and problematic lines
data = []
problematic_lines = []

def parse_line(line):
    parts = line.split(',')
    try:
        # Identifying Ticket_ID
        ticket_id_index = next(i for i, part in enumerate(parts) if part.startswith('TTB'))
        ticket_id = parts[ticket_id_index]
        
        # Identifying Call_Type_ID
        call_type_id_index = ticket_id_index + 1
        call_type_id = parts[call_type_id_index]
        
        # Identifying Create_Date (the part that looks like a datetime string)
        create_date_index = next(i for i, part in enumerate(parts) if '-' in part and ':' in part)
        create_date = parts[create_date_index]
        
        # Call_Type is everything between Call_Type_ID and Create_Date
        call_type = ' '.join(parts[call_type_id_index + 1:create_date_index])
        
        # The rest of the fields in order
        rest = parts[create_date_index + 1:]
        
        # Combine into a single list in the correct order
        structured_line = [ticket_id, call_type_id, call_type, create_date] + rest
        
        # If the length is correct, return it
        if len(structured_line) == len(column_list):
            return structured_line
        # If there are extra fields, handle them (for example, by merging or ignoring)
        elif len(structured_line) > len(column_list):
            return structured_line[:len(column_list)]
        else:
            return None
    except StopIteration:
        # If any part of the parsing fails, consider the line problematic
        return None

# Read the file line by line and parse it
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        parsed_line = parse_line(lines[i])
        if parsed_line:
            data.append(parsed_line)
            i += 1
        else:
            # Check if merging the next line solves the issue
            if i + 1 < len(lines):
                merged_line = lines[i].strip() + ' ' + lines[i + 1].strip()
                parsed_line = parse_line(merged_line)
                if parsed_line:
                    data.append(parsed_line)
                    i += 2
                else:
                    problematic_lines.append((i, lines[i]))
                    i += 1
            else:
                problematic_lines.append((i, lines[i]))
                i += 1

# Convert the collected data into a DataFrame
df = pd.DataFrame(data, columns=column_list)


df['Ticket_ID'] = df['Ticket_ID'].astype(str)
df = df[df['Ticket_ID'].str.match(r'TTB\d+')]

df['Call_Type_ID'] = df['Call_Type_ID'].astype(str)
df = df[df['Call_Type_ID'].str.match(r'^\d{4}$')]

df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce')
df = df.dropna(subset=['Create_Date'])

df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')

columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)


df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricaredatareal_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")


problematic_df = pd.DataFrame(problematic_lines, columns=['Line_Number', 'Data'])


problematic_filename = f"bricaredatareal_problematic_{formatted_startdate}_{formatted_enddate}.csv"
problematic_df.to_csv(problematic_filename, index=False)

print(f"Problematic data saved to {problematic_filename}")


print(f"Number of rows in dataframe: {len(df)}")
print(f"Number of problematic lines: {len(problematic_df)}")


Data saved to bricaredatareal_20200101_20200201.csv
Problematic data saved to bricaredatareal_problematic_20200101_20200201.csv
Number of rows in dataframe: 463581
Number of problematic lines: 881


# Create masked Data for UAT


To Mask:

Nama_Nasabah
No_Rekening
full_name
no_telepon
approver_name = beda dengan Nama_Nasabah
user_login_name = beda 
Log_Name
Nama_Supervisor
Nama_TL
Nama_Wakabag
Remark

Kolom Detail:

Kode Cabang          : 0307 4 digits
No Kartu             : 5221843130736932 16 digits
No Rekening          : 030701098507501 15 digits
Nama                 : SITI SHOLEHA
No ID                : 3316022012770004

In [13]:
import pandas as pd
from faker import Faker
import random 

fake = Faker('id_ID')
pd.set_option('display.max_columns', None)


path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"
df = pd.read_csv(path)

def mask_data(df):
    df['Nama_Nasabah'] = df['Nama_Nasabah'].apply(lambda x: fake.name())
    df['No_Rekening'] = df['No_Rekening'].apply(lambda x: fake.bban())
    df['full_name'] = df['full_name'].apply(lambda x: fake.name())
    df['no_telepon'] = df['no_telepon'].apply(lambda x: fake.phone_number())
    df['approver_name'] = df['approver_name'].apply(lambda x: fake.name())
    df['user_login_name'] = df['user_login_name'].apply(lambda x: fake.user_name())
    df['Log_Name'] = df['Log_Name'].apply(lambda x: fake.user_name())
    df['Nama_Supervisor'] = df['Nama_Supervisor'].apply(lambda x: fake.name())
    df['Nama_TL'] = df['Nama_TL'].apply(lambda x: fake.name())
    df['Nama_Wakabag'] = df['Nama_Wakabag'].apply(lambda x: fake.name())
    return df


df_masked = mask_data(df)

def generate_nik():
    return f'{fake.random_number(digits=16)}'

def mask_detail(detail):
    if isinstance(detail, float) and pd.isna(detail):
        return detail
    lines = str(detail).split('\n')
    masked_lines = []
    for line in lines:
        if 'Kode Cabang' in line:
            line = f'Kode Cabang          : {fake.random_number(digits=4)}'
        elif 'No Kartu' in line:
            line = f'No Kartu             : {fake.credit_card_number()}'
        elif 'No Rekening' in line:
            line = f'No Rekening          : {fake.bban()}'
        elif 'Nama' in line:
            line = f'Nama                 : {fake.name()}'
        elif 'No ID' in line:
            line = f'No ID                : {generate_nik()}'
        masked_lines.append(line)
    return '\n'.join(masked_lines)

# df_masked = df_masked.iloc[:10]


# Ensure 'Details' column is treated as string and apply mask_detail function
df_masked['Details'] = df_masked['Details'].astype(str).apply(mask_detail)



# gateway_options = [
#     'Email', 'Phone', 'Instagram', 'Walk-In', 'MMS', 
#     'Twitter', 'Facebook', 'BRImo', 'BRILink', 'Sabrina', 'Ceria'
# ]

#UAT
gateway_options = [
    'Email', 'Phone', 'Web', 'Facebook', 'Twitter'
]
# Assign random values to the 'gateway' column
df_masked['gateway'] = df_masked['gateway'].apply(lambda x: random.choice(gateway_options))




# df_masked = df_masked.iloc[:10]
df_masked
output_path = r"D:\dataquality2\bricare_uat_20230101_20230101_masking.csv"
df_masked.to_csv(output_path, index=False)


In [19]:
import pandas as pd

path=r"bricare_uat_20230101_20230101_masking.csv"
df=pd.read_csv(path)
df['status']='New'
df.to_csv(r"bricare_uat_20230101_20230101_masking_new.csv", index=False)

# To create data to test in SIT


In [94]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\SIT\dummy_data_case1100000.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
# df['User*']='00GMR0000000dCf2AI'
df['User*']='00GMR0000000dEH2AY'
df['Status']='Escalated'
df['Status'] = df['Status'].replace('New', 'Escalated')
df['Case Origin']='Email'
df['Case Type']='a1BMR00000010h72AA'
df['Account']='001MR000004C7l2YAC'

df.to_csv('dummy_data_uat60k.csv', index=False)

# df['Merchant ID']
# df['TID']
# df['User*']

# To create data to test in UAT

In [92]:
import pandas as pd


path=r"D:\dataquality\bricare_20230101_20230101.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
df.to_csv('dummy_data_case_uat60k.csv', index=False)

In [10]:
import pandas as pd


file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\CHM\CHM\Part2\bricare_get_cleansing_77_kolom(1).csv"
df = pd.read_csv(file_path, delimiter=';')


output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\bricare_get_cleansing_77_kolom_fixed_return.csv"
df=df.iloc[:1]
df['status']='Return'
df.to_csv(output_file_path, index=False)

# output_file_path

# Feature ID        

In [54]:
import pandas as pd

# Load the CSV files
ref_fiturid_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\ref_fiturid.csv"
id_calltype_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_calltype_uat.csv"

ref_fiturid = pd.read_csv(ref_fiturid_path)
id_calltype = pd.read_csv(id_calltype_path)

# Convert calltype and NAME columns to string type
ref_fiturid['calltype'] = ref_fiturid['calltype'].astype(str)
id_calltype['NAME'] = id_calltype['NAME'].astype(str)

# Merge the dataframes to add the Call_Type_ID column based on the calltype column
ref_fiturid = ref_fiturid.merge(id_calltype[['NAME', 'ID']], left_on='calltype', right_on='NAME', how='left')

# Rename the ID column to Call_Type_ID
ref_fiturid.rename(columns={'ID': 'Call_Type_ID'}, inplace=True)

# Drop the unnecessary NAME column from the merge
ref_fiturid.drop(columns=['NAME'], inplace=True)

# Save the updated dataframe to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv"
# ref_fiturid=ref_fiturid.iloc[62:63]
ref_fiturid.to_csv(output_path, index=False)


print(f"Updated CSV file saved to {output_path}")



Updated CSV file saved to C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv


# Knowledge

## To see the characteristics of all files and save it as excel files. FILE: EXCEL

In [1]:
import pandas as pd
import os

# Function to find the longest length of values in each column
def find_longest_lengths_per_column(df):
    lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
    max_lengths = lengths.max()
    return max_lengths

# Set the folder path
# folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24"
folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\knowledge\new_WI_12aug"

# Get all Excel files in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Load the files dynamically
dfs = []
rows_count = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_excel(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    dfs.append((file_name, df))
    rows_count.append((file_name, len(df)))

# Calculate the longest length for each column in each file
longest_lengths = {}
for file_name, df in dfs:
    longest_lengths[file_name] = find_longest_lengths_per_column(df)

# Combine results into a single DataFrame
results = pd.DataFrame(longest_lengths).transpose()

# Create DataFrame for rows count
rows_count_df = pd.DataFrame(rows_count, columns=["File Name", "Total Rows"])

# Print the results
results = results.transpose()
results.to_excel('allcolumns_artikel2aug.xlsx')

# Save rows count to a new Excel file
rows_count_df.to_excel('allcolumns_artikel_rows_count.xlsx', index=False)

# Find the longest column and its length for each file
longest_columns_info = []
for file_name, df in dfs:
    lengths = find_longest_lengths_per_column(df)
    longest_col = lengths.idxmax()
    longest_length = lengths.max()
    longest_columns_info.append({"File": file_name, "Column": longest_col, "Length": longest_length})

# Convert the list to a DataFrame
longest_columns_df = pd.DataFrame(longest_columns_info)
longest_columns_df.to_excel('allcolumns_artikel_col12aug.xlsx', index=False)


## To see the characteristics of all files and save it as excel files. FILE: CSV

In [1]:
import pandas as pd
import os

# Function to find the longest length of values in each column
def find_longest_lengths_per_column(df):
    lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
    max_lengths = lengths.max()
    return max_lengths

# Set the folder path
folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug"

# Get all CSV files in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Load the files dynamically
dfs = []
rows_count = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    dfs.append((file_name, df))
    rows_count.append((file_name, len(df)))

# Calculate the longest length for each column in each file
longest_lengths = {}
for file_name, df in dfs:
    longest_lengths[file_name] = find_longest_lengths_per_column(df)

# Combine results into a single DataFrame
results = pd.DataFrame(longest_lengths).transpose()

# Create DataFrame for rows count
rows_count_df = pd.DataFrame(rows_count, columns=["File Name", "Total Rows"])

# Print the results
results = results.transpose()
results.to_excel('allcolumns_artike29aug.xlsx')

# Save rows count to a new Excel file
rows_count_df.to_excel('allcolumns_artikel_rows_count29aug.xlsx', index=False)

# Find the longest column and its length for each file
longest_columns_info = []
for file_name, df in dfs:
    lengths = find_longest_lengths_per_column(df)
    longest_col = lengths.idxmax()
    longest_length = lengths.max()
    longest_columns_info.append({"File": file_name, "Column": longest_col, "Length": longest_length})

# Convert the list to a DataFrame
longest_columns_df = pd.DataFrame(longest_columns_info)
longest_columns_df.to_excel('allcolumns_artikel_col29aug.xlsx', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2989052216.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2989052216.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2989052216.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2989052216.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2989052216.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use

In [3]:
import pandas as pd
import os

# Function to find the longest length of values in each column
def find_longest_lengths_per_column(df):
    lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
    max_lengths = lengths.max()
    return max_lengths

# Set the folder path
# folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_wise\sample_wise"

folder_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24"

# List of file names
file_names = [
    "sample_wi_100.xlsx",
    "sample_wise_produk_58.xlsx",
    "sample_wise_program_41.xlsx",
    "sample_wise_promo_90.xlsx"
]

# Load the files dynamically
dfs = []
rows_count = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_excel(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    dfs.append((file_name, df))
    rows_count.append((file_name, len(df)))

# Calculate the longest length for each column in each file
longest_lengths = {}
for file_name, df in dfs:
    longest_lengths[file_name] = find_longest_lengths_per_column(df)

# Combine results into a single DataFrame
results = pd.DataFrame(longest_lengths).transpose()

# Create DataFrame for rows count
rows_count_df = pd.DataFrame(rows_count, columns=["File Name", "Total Rows"])

# Print the results
results = results.transpose()
results.to_excel('allcolumns_artikel2.xlsx')

# Save rows count to a new Excel file
rows_count_df.to_excel('allcolumns_artikel_rows_count.xlsx', index=False)

# Optional: Find the longest column and its length for each file
# longest_columns_info = []
# for file_name, lengths in results.iterrows():
#     longest_col = lengths.idxmax()
#     longest_length = lengths.max()
#     longest_columns_info.append({"File": file_name, "Column": longest_col, "Length": longest_length})

# Convert the list to a DataFrame
# longest_columns_df = pd.DataFrame(longest_columns_info)
# longest_columns_df.to_excel('longest_columns_info.xlsx', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2514165187.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)


## Working Instruction

In [12]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the Excel file
# path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\knowledge\new_WI_6aug\knowledge_wi.xlsx"
# df = pd.read_excel(path)


path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\knowledge_wi_29_agustus_2024.csv"
df = pd.read_csv(path)

# Drop the 'Unnamed: 0' column if it exists
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns = df.select_dtypes(include=[object]).columns
df[text_columns] = df[text_columns].applymap(clean_text)

# Function to generate sequential URL names
def generate_url_name(index):
    return f"WI-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df))]

# Check the 'calltype' column and replace "-" or empty values with "No Call Type"
df['calltype'] = df['calltype'].apply(lambda x: 'No Call Type' if pd.isnull(x) or x.strip() == '-' else x)

df['Article_Type'] = 'Working Instruction'
df['record_type'] = 'Working Instructions'

# Function to limit words in the title to 20 words
def limit_words(text, word_limit=20):
    if pd.isnull(text) or text.strip() == '':
        return 'No Title'
    words = text.split()
    limited_text = ' '.join(words[:word_limit])
    return limited_text

# Add the new 'title' column, handle missing values in concatenation
df['title'] = df['calltype'].fillna('No Call Type') + ' - ' + df['kategori'].fillna('') + ' - ' + df['sub_kategori'].fillna('')
df['title'] = df['title'].apply(limit_words)

# Save the cleaned DataFrame to a CSV file
df.to_csv('cleaned_wise_wi_29aug.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_6260\681736892.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[text_columns] = df[text_columns].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
import pandas as pd


path= r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\knowledge\new_WI_6aug\cleaned_wise_promo_90_6aug.csv"
df =pd.read_csv(path)

df=df.iloc[:1]
df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\knowledge\new_WI_6aug\cleaned_wise_promo_90_6aug_1line.csv", index=False)

## Product

In [10]:
import json
import pandas as pd
from bs4 import BeautifulSoup
from prettytable import PrettyTable
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt
import os

# Load the new Excel file
# file_path_new = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\knowledge\new_WI_6aug\knowledge_produk.xlsx"
# df_program = pd.read_excel(file_path_new)

file_path_new = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\knowledge_produk_29_agustus_2024.csv"
df_program = pd.read_csv(file_path_new)

# Drop the 'Unnamed: 0' column if it exists
df_program = df_program.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        clean_text = clean_text.strip()
        return clean_text
    return text

# Add the new 'title' column
df_program.insert(0, 'title', df_program['jenis_produk'] + ' - ' + df_program['produk'] + ' - ' + df_program['varian_produk'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PD-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_program['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_program))]
df_program['calltype'] = 'No Call Type'
df_program['Article_Type'] = 'Product'
df_program['record_type'] = 'Product'

# Apply the clean_text function to all text columns
text_columns_program = df_program.select_dtypes(include=[object]).columns
df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)

# Function to save DataFrame to Excel with auto-fit columns and borders
def save_df_to_excel(df, file_path):
    writer = pd.ExcelWriter(file_path, engine='openpyxl')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Auto-fit columns
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        worksheet.column_dimensions[column].width = adjusted_width

    # Add borders to cells
    thin_border = openpyxl.styles.Border(
        left=openpyxl.styles.Side(style='thin'),
        right=openpyxl.styles.Side(style='thin'),
        top=openpyxl.styles.Side(style='thin'),
        bottom=openpyxl.styles.Side(style='thin')
    )

    for row in worksheet.iter_rows():
        for cell in row:
            cell.border = thin_border

    workbook.save(file_path)

# Function to generate a picture of the DataFrame
def generate_df_picture(df, file_path):
    fig, ax = plt.subplots(figsize=(12, len(df) // 2))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    plt.savefig(file_path, bbox_inches='tight', pad_inches=0.1)
    plt.close()

# Function to check if a string is valid JSON and convert it to readable format
def process_json(text, row_idx, col_name, save_path, generate_pictures):
    try:
        data = json.loads(text)
        if isinstance(data, list) and 'table_data' in data[0]:
            # Save the JSON data to a separate Excel file
            table_data = data[0]['table_data']
            rows = []

            for row in table_data:
                row_values = [item['value'] for item in row]
                rows.append(row_values)

            json_df = pd.DataFrame(rows[1:], columns=rows[0])
            json_file_path = os.path.join(save_path, f'json_table_{row_idx}_{col_name}.xlsx')
            save_df_to_excel(json_df, json_file_path)

            if generate_pictures:
                # Generate picture of the table
                img_file_path = json_file_path.replace('.xlsx', '.png')
                generate_df_picture(json_df, img_file_path)

            table = PrettyTable()
            table.align = "l"  # Align all text to the left

            headers = [item['value'] for item in table_data[0]]
            table.field_names = headers
            
            # Calculate the maximum width for each column
            max_widths = {header: len(header) for header in headers}
            for row in table_data[1:]:
                for i, item in enumerate(row):
                    value_length = len(str(item['value']).strip())
                    if value_length > max_widths[headers[i]]:
                        max_widths[headers[i]] = value_length
            
            for row in table_data[1:]:
                row_values = [item['value'].strip() if isinstance(item['value'], str) else item['value'] for item in row]
                table.add_row(row_values)
            
            # Adjust column widths
            for field in table.field_names:
                table.max_width[field] = max_widths[field]
            
            return str(table)
        return text
    except (ValueError, TypeError, KeyError):
        return text

# Process and clean JSON-like text in all columns, saving JSON data to separate files
def process_all_json_texts(df, save_path, generate_pictures):
    json_columns_rows = []
    text_columns_program = df.select_dtypes(include=[object]).columns
    for column in text_columns_program:
        for idx, value in enumerate(df[column]):
            if isinstance(value, str) and value.strip().startswith('[') and value.strip().endswith(']'):
                df.at[idx, column] = process_json(value, idx, column, save_path, generate_pictures)
                json_columns_rows.append((column, idx))
    return json_columns_rows

# Define the path where to save JSON files and whether to generate pictures
save_path = r"D:\dataquality2\product_artikel29aug"
generate_pictures = False

# Create the directory if it does not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Process all JSON texts in the DataFrame
json_columns_rows = process_all_json_texts(df_program, save_path, generate_pictures)

# Save the cleaned DataFrame to a new CSV file
output_file_path = 'cleaned_product_29aug.csv'
df_program.to_csv(output_file_path, index=False)

# Display lines 39 to 42 for user reference
# lines_39_to_42 = df_program.iloc[38:42]
# print(lines_39_to_42['Biaya'].values)

# Add comments indicating columns and rows with JSON texts
for column, row in json_columns_rows:
    print(f"Column '{column}' at row {row} contains JSON text. Saved to '{os.path.join(save_path, f'json_table_{row}_{column}.xlsx')}'.")


C:\Users\maste\AppData\Local\Temp\ipykernel_6260\2709541421.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Column 'produk' at row 12 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_12_produk.xlsx'.
Column 'bunga' at row 4 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_4_bunga.xlsx'.
Column 'limits' at row 4 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_4_limits.xlsx'.
Column 'limits' at row 6 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_6_limits.xlsx'.
Column 'biaya' at row 4 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_4_biaya.xlsx'.
Column 'biaya' at row 6 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_6_biaya.xlsx'.
Column 'biaya' at row 44 contains JSON text. Saved to 'D:\dataquality2\product_artikel29aug\json_table_44_biaya.xlsx'.


## to handle empty values in column id_product

In [11]:
# Load the CSV file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\cleaned_product_29aug.csv"
df = pd.read_csv(file_path)

# Fill empty values in 'id_produk' with values from 'URL_Name'
df['id_produk'].fillna(df['URL_Name'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\cleaned_product_29aug.csv"
df.to_csv(output_path, index=False)

output_path


C:\Users\maste\AppData\Local\Temp\ipykernel_6260\693236902.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['id_produk'].fillna(df['URL_Name'], inplace=True)


'C:\\Users\\maste\\Downloads\\dataloader_v60.0.2\\server\\artikel\\knowledge\\new_29aug\\cleaned_product_29aug.csv'

In [25]:
# import pandas as pd
# from bs4 import BeautifulSoup

# # Load the new Excel file
# file_path_new = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_produk_75.xlsx"
# df_new = pd.read_excel(file_path_new)

# # Drop the 'Unnamed: 0' column if it exists
# df_new = df_new.drop(columns=['Unnamed: 0'], errors='ignore')

# # Function to clean text by removing HTML tags and unwanted characters
# def clean_text(text):
#     if isinstance(text, str):
#         # Remove HTML tags
#         clean_text = BeautifulSoup(text, "html.parser").get_text()
#         # Replace \n and \t with actual new lines and tabs
#         clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
#         # Remove trailing and leading spaces
#         clean_text = clean_text.strip()
#         return clean_text
#     return text

# # Apply the clean_text function to all text columns
# text_columns_new = df_new.select_dtypes(include=[object]).columns
# df_new[text_columns_new] = df_new[text_columns_new].applymap(clean_text)

# # Convert the date format
# # df_new['modified_time'] = pd.to_datetime(df_new['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# # Add the new 'title' column
# df_new.insert(0, 'title', df_new['jenis_produk'] + ' - ' + df_new['produk'] + ' - ' + df_new['varian_produk'] + ' - ' + df_new['nama_knowledge'])

# # Function to generate sequential URL names
# def generate_url_name(index):
#     return f"PD-{index:07d}"

# # Apply the function to generate URL names based on the row index + 1
# df_new['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_new))]

# # Save the cleaned DataFrame to a CSV file
# df_new['calltype'] = 'No Call Type'
# df_new['Article_Type']='Product'
# df_new['record_type']='Product'
# # df_new=df_new.iloc[:1]
# df_new.to_csv('cleaned_wise_produk_75.csv', index=False)


## Program

In [9]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the new Excel file
file_path_program = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\knowledge_program_29_agustus_2024.csv"
df_program = pd.read_csv(file_path_program)

# Drop the 'Unnamed: 0' column if it exists
df_program = df_program.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns_program = df_program.select_dtypes(include=[object]).columns
df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)

# Convert the date format for 'modified_time', 'start', and 'end' columns
# df_program['modified_time'] = pd.to_datetime(df_program['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_program['start'] = pd.to_datetime(df_program['start'], errors='coerce').dt.strftime('%Y-%m-%d')
df_program['end'] = pd.to_datetime(df_program['end'], errors='coerce').dt.strftime('%Y-%m-%d')

# Add the new 'title' column
# df_program.insert(0, 'title', df_program['nama_program'])

df_program.insert(0, 'title', df_program['nama'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PG-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_program['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_program))]

# Save the cleaned DataFrame to a CSV file
df_program['calltype'] = 'No Call Type'
df_program['Article_Type']='Program'
df_program['record_type']='Program'
# df_program=df_program.iloc[:1]
df_program.to_csv('cleaned_wise_program_29aug.csv', index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_6260\1595748788.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


## Promo

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the new Excel file
# file_path_promo = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_promo_107.xlsx"
# df_promo = pd.read_excel(file_path_promo)


file_path_promo = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\knowledge_promo_29_agustus_2024.csv"
df_promo = pd.read_csv(file_path_promo)

# Drop the 'Unnamed: 0' column if it exists
df_promo = df_promo.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns_promo = df_promo.select_dtypes(include=[object]).columns
df_promo[text_columns_promo] = df_promo[text_columns_promo].applymap(clean_text)

# Convert the date format for 'modified_time', 'start', and 'end' columns
# df_promo['modified_time'] = pd.to_datetime(df_promo['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_promo['start'] = pd.to_datetime(df_promo['start'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df_promo['end'] = pd.to_datetime(df_promo['end'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# Add the new 'title' column
df_promo.insert(0, 'title', df_promo['kategori'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PM-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_promo['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_promo))]

# Save the cleaned DataFrame to a CSV file
df_promo['calltype'] = 'No Call Type'
df_promo['Article_Type']='Promotion'
df_promo['record_type']='Promotion'

# Save the cleaned DataFrame to a CSV file
# df_promo=df_promo.iloc[:1]
df_promo.to_csv('cleaned_wise_promo_29aug.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_6260\967382301.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_promo[text_columns_promo] = df_promo[text_columns_promo].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


## to handle duplicates 

In [8]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\cleaned_wise_promo_29aug.csv"
df = pd.read_csv(file_path)

# Create a dictionary to track occurrences of 'id_promo'
id_counts = {}

def modify_duplicates(id_promo):
    # Increase count each time an id_promo is seen
    if id_promo in id_counts:
        id_counts[id_promo] += 1
        # Append a number to duplicates without an underscore
        return f"{id_promo}{id_counts[id_promo]}"
    else:
        id_counts[id_promo] = 0  # First occurrence is not modified
        return id_promo

# Apply the function to 'id_promo' column
df['id_promo'] = df['id_promo'].apply(modify_duplicates)

# Save the updated DataFrame to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\artikel\knowledge\new_29aug\cleaned_wise_promo_29aug_done.csv"
df.to_csv(output_path, index=False)

output_path


'C:\\Users\\maste\\Downloads\\dataloader_v60.0.2\\server\\artikel\\knowledge\\new_29aug\\cleaned_wise_promo_29aug_done.csv'

In [1]:
import pandas as pd

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare(WI).csv"
df=pd.read_csv(path, encoding='ISO-8859-1')
df = df.fillna('')
df['Article Type']='Working Instruction'

output=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\artikel_all.csv"



# Change 'Brilink' to 'BRILink' in 'Type' column
df['Type'] = df['Type'].replace('Brilink', 'BRILink')

# Remove rows with 'Wholesale' in 'Type' column
df = df[df['Type'] != 'Wholesale']

# df['Type'].unique()

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)



df.to_csv('artikel2.csv', index= False)
# df.iloc[:1].to_csv(output, index= False)

df



,Unnamed: 0,Call Type,Call Type Info,Type,User,Title,Detail Bricare,Deskripsi,Empati,Konfirmasi,...,Verifikasi,Gali Informasi,Pembuatan Laporan,Konfirmasi Ulang,Percepatan Komplain,"Solusi, Informasi",Edukasi & Cross Selling,Closing,Article Type,URL_name
0,1,1000,Nasabah Menanyakan Informasi Pengajuan Terkait...,CERIA,Agent Contact Center,1000 - Nasabah Menanyakan Informasi Pengajuan ...,nomor ponsel nasabah yang bisa dihubungi\nID C...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\t\t\t...,Jika Nasabah menanyakan Cara & Syarat Pengajua...,...,1\tCara & Syarat pengajuan Aplikasi\nVerifikas...,1\tCara & Syarat pengajuan Aplikasi\nGali Info...,1\tCara & Syarat pengajuan Aplikasi\nBuat lapo...,,,1\tCara & Syarat pengajuan Aplikasi\nInformasi...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1000-Nasabah-Menanyakan-Informasi-Pengajuan-Te...
1,2,1002,Nasabah Menyakan Terkait Promo dan Program CERIA,CERIA,Agent Contact Center,1002 - Nasabah Menyakan Terkait Promo dan Prog...,,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\nJika...,Konfirmasi :\nJika Nasabah menanyakan promo ce...,...,Verifikasi\nTidak dilakukan Verifikasi\n,Gali Informasi\t\n\t i. Jenis promo/...,Pembuatan Laporan\t\n\ti. Nomor laporan ti...,,,Informasi & Solusi \t\t\n\t i. \...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1002-Nasabah-Menyakan-Terkait-Promo-dan-Progra...
2,3,1003,Nasabah Mengajukan Pelunasan Awal Cicilan CERIA,CERIA,Agent Contact Center,1003 - Nasabah Mengajukan Pelunasan Awal Cicil...,,Nasabah menghubungi Contact BRI / datang ke Un...,"Empati sesuai kondisi (pilih salah satu)\n""Bap...","Konfirmasi :\n""Untuk Permintaan Pelunasan awal...",...,Verifikasi ( Buka CLOS/ Finnachel )\t\nUntuk v...,Gali Informasi\t\n1.\tTanyakan alasan pelunasa...,Pembuatan Laporan \t\n1.\tBuat Laporan dengan ...,,,Informasi & Solusi\n1.\tCicilan telah berjalan...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1003-Nasabah-Mengajukan-Pelunasan-Awal-Cicilan...
3,4,1005,Nasabah Mengajukan Pemblokiran CERIA,CERIA,Agent Contact Center,1005 - Nasabah Mengajukan Pemblokiran CERIA,Nasabah mengajukan pemblokiran Sementara Akun ...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati\nJika nasabah infokan ingin melakukan p...,"Konfirmasi: \n""Untuk pemblokiran bersifat seme...",...,Verifikasi ( Buka WBS/ CLOS )\t\nUntuk verifik...,Gali Informasi\t\n1.\tTanyakan alasan perminta...,Pembuatan Laporan\nNasabah mengajukan pembloki...,,,Informasi & Solusi \t\n1.\tAgent Konfirmasi ke...,"Kalimat Edukasi\n""Kami informasikan Bank BRI t...",Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1005-Nasabah-Mengajukan-Pemblokiran-CERIA
4,5,1008,Nasabah Mengajukan Pengaktifan Akun Ceria Terb...,CERIA,Agent Contact Center,1008 - Nasabah Mengajukan Pengaktifan Akun Cer...,,,,,...,,,,,,,,,Working Instruction,1008-Nasabah-Mengajukan-Pengaktifan-Akun-Ceria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,,,,,,,,,,,...,,,,,,,,,Working Instruction,
438,,,,,,,,,,,...,,,,,,,,,Working Instruction,
439,,,,,,,,,,,...,,,,,,,,,Working Instruction,
440,,,,,,,,,,,...,,,,,,,,,Working Instruction,


In [91]:
import pandas as pd

pd.set_option('display.max_columns', None)
df= pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424041601915.csv")
df.to_csv('error.csv', index=False)

In [23]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare 2.xlsx"

df = pd.read_excel(path)
df=df.iloc[:1]
# df['Gali Informasi']

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)

df.to_csv('artikel_bullet.csv', index=False)

# Generate Roles


## generate 1k roles

In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/mnt/data/extract_role.csv'
roles_df = pd.read_csv(file_path)

# Display the content of the CSV file to understand its structure
roles_df.head()


# Number of additional roles needed to reach a total of 1000
num_additional_roles_needed = 1000 - len(expanded_roles_df)

# Duplicate the existing roles to meet the required number of roles
additional_roles_more = expanded_roles_df.sample(num_additional_roles_needed, replace=True).reset_index(drop=True)

# Generate unique names for the additional roles
additional_roles_more['NAME'] = additional_roles_more['NAME'] + "_more_" + (additional_roles_more.index + 1).astype(str)

# Combine the original roles with the additional roles
expanded_roles_df_more = pd.concat([expanded_roles_df, additional_roles_more], ignore_index=True)

# Ensure we have a total of 1000 roles
expanded_roles_df_more = expanded_roles_df_more.head(1000)

# Save the expanded roles to a new CSV file
output_file_path_more = 'expanded_roles_1000.csv'
expanded_roles_df_more.to_csv(output_file_path_more, index=False)

output_file_path_more



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/extract_role.csv'

## Check Extraction Data



### check data before 2023 = 27 columns

In [13]:
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]

column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

import pandas as pd

path=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_27_kolom.csv"
path2=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_79_kolom.csv"
# df= pd.read_csv(path, delimiter=';')
df= pd.read_csv(path2, delimiter=';')


# column_list_set = set(column_list)
column_list_set = set(column_names)
df_columns_set = set(df.columns)

missing_columns = column_list_set - df_columns_set
extra_columns = df_columns_set - column_list_set

if not missing_columns and not extra_columns:
    print("The column names match.")
else:
    if missing_columns:
        print(f"Missing columns in DataFrame: {missing_columns}")
    if extra_columns:
        print(f"Extra columns in DataFrame: {extra_columns}")

################################################
# column_list_set2 = set(column_names)
# df_columns_set2 = set(df2.columns)

# missing_columns2 = column_list_set2 - df_columns_set2
# extra_columns2 = df_columns_set2 - column_list_set2

# if not missing_columns and not extra_columns:
#     print("The column names match.")
# else:
#     if missing_columns2:
#         print(f"Missing columns in DataFrame: {missing_columns2}")
#     if extra_columns2:
#         print(f"Extra columns in DataFrame: {extra_columns2}")


Missing columns in DataFrame: {'Tgl_Assigned'}
Extra columns in DataFrame: {'TglAssigned', 'cifno', 'Details'}


Data dengan 27 kolom:
- jumlah kolom sudah ok
- picklist Walk In harus diganti ke Walk-In
- attachment
- penambahan nama file diakhir "27"



Data dengan 79 kolom:
- picklist Walk In harus diganti ke Walk-In
- penambahan nama file diakhir "79"



Data Zendesk:
- harus hilangkan double quotes ""
- delimiternya ;
- nama file zendesk


Data Omni:
- SMS: harus format csv

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060524083553095.csv"
df=pd.read_csv(path)
df['ERROR']

0    Unable to create/update fields: LastModifiedDa...
1    Unable to create/update fields: LastModifiedDa...
2    Unable to create/update fields: LastModifiedDa...
3    Unable to create/update fields: LastModifiedDa...
4    Unable to create/update fields: LastModifiedDa...
5    Unable to create/update fields: LastModifiedDa...
6    Unable to create/update fields: LastModifiedDa...
7    Unable to create/update fields: LastModifiedDa...
8    Unable to create/update fields: LastModifiedDa...
9    Unable to create/update fields: LastModifiedDa...
Name: ERROR, dtype: object

# Zendesk

In [33]:
import pandas as pd

file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\Data Zendesk 1-15 Januari 2024 - sampel.csv"

import csv


# Read the CSV file with proper handling of quotes and custom separator
with open(file_path, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',', quotechar='"')
    rows = [row for row in reader]

# Split the combined columns that have commas within them
split_rows = []
for row in rows:
    split_row = row[0].split(',') + row[1:]  # Split the first column and add the rest as they are
    split_rows.append(split_row)

# Create DataFrame from the processed rows
df = pd.DataFrame(split_rows[1:], columns=split_rows[0])

# Remove double quotes from column names
df.columns = df.columns.str.replace('"', '')

# Remove double quotes from all data in the DataFrame
df = df.replace('"', '', regex=True)

# Display the first few rows of the cleaned DataFrame
print(df.head())
df

# If you need to save the cleaned DataFrame to a new CSV file
# cleaned_file_path = '/mnt/data/Cleaned_Zendesk_Data.csv'
# df.to_csv(cleaned_file_path, index=False)

# print(f"Cleaned data saved to {cleaned_file_path}")



  Ticket ID Ticket channel   Assignee ID   Assignee name    Requester ID  \
0   3435272    Any channel  405257525413  Agent Sosmed 4  27064719563033   
1   3435273       Facebook  405257335633  Agent Sosmed 3  27064768687769   

     Requester name                                     Ticket subject  \
0        vianovia94  [IGC] Minimal opo boloo... ......#relsvideo #r...   
1  Masruroh Sodikin                                                  P   

  Requester created - Timestamp Ticket created - Timestamp  \
0           2024-01-01T00:03:30        2024-01-01T00:03:30   
1           2024-01-01T00:04:15        2024-01-01T00:04:15   

  Ticket solved - Timestamp                 Tickets  
0       2024-01-01T00:30:56  1.00000000000000000000  
1       2024-01-01T00:46:27  1.00000000000000000000  


,Ticket ID,Ticket channel,Assignee ID,Assignee name,Requester ID,Requester name,Ticket subject,Requester created - Timestamp,Ticket created - Timestamp,Ticket solved - Timestamp,Tickets
0,3435272,Any channel,405257525413,Agent Sosmed 4,27064719563033,vianovia94,[IGC] Minimal opo boloo... ......#relsvideo #r...,2024-01-01T00:03:30,2024-01-01T00:03:30,2024-01-01T00:30:56,1.00000000000000000000
1,3435273,Facebook,405257335633,Agent Sosmed 3,27064768687769,Masruroh Sodikin,P,2024-01-01T00:04:15,2024-01-01T00:04:15,2024-01-01T00:46:27,1.00000000000000000000


In [2]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\bricare_20221213_20240604_case_account.csv"
# df=pd.read_csv(path)
# df



path2=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\case.csv"
df=pd.read_csv(path2)
df

ParserError: Error tokenizing data. C error: Expected 2 fields in line 86, saw 3


### Dummy data for Alex7

# Data Account Alex7

In [7]:
# Create dummy data for account with quoting (DONE)

import csv
import random
import os

def generate_dummy_data(file_path, file_name, num_rows):
    # Define possible values for each column
    account_names = ['John Doe', 'Jane Smith', 'Mike Brown', 'Lisa Green', 'Mark Taylor']
    account_owners = ['Owner A', 'Owner B', 'Owner C', 'Owner D']
    nasabah_types = ['Nasabah', 'Non Nasabah']
    account_record_types = ['Personal', 'Non Personal']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Writing to the CSV file
    with open(os.path.join(file_path, file_name), mode='w', newline='') as file:
        writer = csv.writer(file, quotechar='"', quoting=csv.QUOTE_ALL)  # Enforce quoting for all fields
        writer.writerow(['Account Name', 'CIF No', 'Account Owner', 'No Telp', 'Email', 'Nasabah Type', 'Account Record Type'])
        
        for _ in range(num_rows):
            account_name = random.choice(account_names)
            email = f"{account_name.split(' ')[0].lower()}.{account_name.split(' ')[1].lower()}@example.com"
            writer.writerow([
                account_name,
                ''.join(["{}".format(random.randint(0, 9)) for _ in range(10)]),  # 10-digit CIF No
                random.choice(account_owners),
                f'+62{random.randint(1000000000, 9999999999)}',  # Phone number
                email,
                random.choice(nasabah_types),
                random.choice(account_record_types)
            ])


file_path = 'D:\dataquality2'  # Adjust the path as needed
file_name = 'dummy_data_account.csv'
num_rows = 1  # Adjust the number of rows as needed

generate_dummy_data(file_path, file_name, num_rows)


## Data Case ALex 7


num of rows 

In [33]:
def create_salesforce_case_dummy_data(num_rows=100, start_ttb=1):
    import pandas as pd
    import random

    # Define the columns and possible values
    statuses = ["New", "Working", "Escalated"]
    types = ["Electronic", "Electrical", "Mechanical"]
    case_reasons = ["Performance", "Breakdown"]

    # Generate sample data
    data = {
        "Status": [random.choice(statuses) for _ in range(num_rows)],
        "Type": [random.choice(types) for _ in range(num_rows)],
        "Case Reason": [random.choice(case_reasons) for _ in range(num_rows)],
        "Subject": [f"Subject {i+1}" for i in range(num_rows)],
        "Description": [f"Description of issue {i+1}" for i in range(num_rows)],
        "Legacy_Ticket_ID": [f"TTB{start_ttb + i:06d}" for i in range(num_rows)],  # Generating 6-digit random numbers in order
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to CSV
    file_path = 'dummy_casefor_alex11.csv'
    df['record_type'] = 'Case Migration'
    df.to_csv(file_path, index=False)

    return file_path

# Example usage
file_path = create_salesforce_case_dummy_data(num_rows=5, start_ttb=1006)
file_path


'dummy_casefor_alex11.csv'

In [4]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\bricaredatareal_20200101_20200201.csv"

df=pd.read_csv(path)
df=df.iloc[:100]
df.to_csv('bricare_100_pakpondah.csv',index=False)

# data dummy



C:\Users\maste\AppData\Local\Temp\ipykernel_5108\2256201397.py:5: DtypeWarning: Columns (14,16,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


In [1]:
import pandas as pd
import math

def generate_dummy_files(base_filename, total_rows, start_ttb='TTB0000000001'):
    
    rows_per_file = 200000
    num_files = math.ceil(total_rows / rows_per_file)
    
    start_number = int(start_ttb[3:])
    
    for file_index in range(num_files):
       
        if file_index == num_files - 1:
            current_rows = total_rows % rows_per_file or rows_per_file
        else:
            current_rows = rows_per_file
        
        data = {
            'Status': ['Cancelled'] * current_rows,
            'Priority': ['Low'] * current_rows,
            'Call Type': ['1000'] * current_rows,
            'record_type': ['Case Migration'] * current_rows,
            'Create_Date': ['1999-01-01'] * current_rows,
            'TanggalClosed': ['1999-01-01'] * current_rows,
            'Legacy_ticket_id': [f'TTB{str(i).zfill(12)}' for i in range(start_number + file_index * rows_per_file, start_number + file_index * rows_per_file + current_rows)]
        }
        
        df = pd.DataFrame(data)
        
        current_filename = f"{base_filename}_part{file_index + 1}.csv"
        
        df.to_csv(current_filename, index=False)
        print(f"{current_filename} with {current_rows} rows created successfully.")

base_filename = 'dummy_data_casenumber_fix'
total_rows = 200000
start_ttb = 'TTB000000000001'
generate_dummy_files(base_filename, total_rows, start_ttb)


dummy_data_casenumber_fix_part1.csv with 200000 rows created successfully.


In [3]:
import pandas as pd
import math
import os

def generate_dummy_files(folder_path, base_filename, total_rows, start_ttb='TTB0000000001'):
    # Define rows per file
    rows_per_file = 300000
    # Calculate number of files required
    num_files = math.ceil(total_rows / rows_per_file)
    
    # Convert start TTB to an integer number
    start_number = int(start_ttb[3:])
    
    for file_index in range(num_files):
        # Determine number of rows for the current file
        if file_index == num_files - 1:
            current_rows = total_rows % rows_per_file or rows_per_file
        else:
            current_rows = rows_per_file
        
        # Generate data for the current file
        data = {
            'status': ['Cancelled'] * current_rows,
            'Priority': ['Low'] * current_rows,
            'Call_Type_ID': ['1000'] * current_rows,
            'record_type': ['Case Migration'] * current_rows,
            'Create_Date' : ['1999-01-01'] * current_rows,
            'TanggalClosed' : ['1999-01-01'] * current_rows,
            'Legacy_Ticket_ID': [f'TTB{str(i).zfill(10)}' for i in range(start_number + file_index * rows_per_file, start_number + file_index * rows_per_file + current_rows)]
        }
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Create filename for the current file
        current_filename = os.path.join(folder_path, f"{base_filename}_part{file_index + 1}.csv")
        
        # Save DataFrame to CSV
        df.to_csv(current_filename, index=False)
        print(f"{current_filename} with {current_rows} rows created successfully.")
    

# Define folder path, base filename, total rows, and start TTB
folder_path = r"D:"
base_filename = 'dummy_data_casenumber_fix'
total_rows = 200000
start_ttb = 'TTB0000000001'

# Generate dummy files
generate_dummy_files(folder_path, base_filename, total_rows, start_ttb)


D:dummy_data_casenumber_fix_part1.csv with 200000 rows created successfully.


In [3]:
import pandas as pd

path=r"D:\dataquality2\dummy_data_casenumber_fix_part1.csv"

df =pd.read_csv(path)
df = df.drop(columns=['Legacy_ticket_id'])
df.to_csv(path,index=False)

## Data Case ALex 7
Join the two files based on Ticket ID

In [18]:
#extract the ID and CaseNumber:
# SELECT Id, CaseNumber
# FROM Case 
# WHERE CreatedById = '005MR0000004GJ7YAM'

# from the real data to join the ID or Case Number and update the records in Case 

import pandas as pd

# Load the data from the uploaded files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_150_casenumberandID.csv")
# test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\Test Data Dummy.csv")
test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124052713145.csv")


# Split the 'ID;"CASENUMBER"' column into two separate columns
extract_df[['ID', 'CASENUMBER']] = extract_df['ID;"CASENUMBER"'].str.split(';', expand=True)

# Drop the original combined column
extract_df.drop(columns=['ID;"CASENUMBER"'], inplace=True)

# Remove leading and trailing quotes from the columns
extract_df['ID'] = extract_df['ID'].str.strip('"')
extract_df['CASENUMBER'] = extract_df['CASENUMBER'].str.strip('"')

# Rename columns for easier merging
extract_df.rename(columns={'CASENUMBER': 'Ticket_ID'}, inplace=True)

# Merge the dataframes based on 'Ticket_ID'
merged_df = test_data_dummy_df.merge(extract_df, on='Ticket_ID', how='left')

# Reorder the columns to place 'ID' as the first column
columns = ['ID'] + [col for col in merged_df.columns if col != 'ID']
merged_df = merged_df[columns]

# Save the updated dataframe to a new CSV file
# merged_df=merged_df.iloc[:1]
merged_df['Call_Type_ID']='1000'
merged_df=merged_df.drop('ID', axis=1)
merged_df.to_csv('Updated_Test_Data_Dummy_2row.csv', index=False)


KeyError: 'Ticket_ID'

In [38]:
# the extracted data from Salesforce after being updated 


import pandas as pd

pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\case_150.csv"
path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_case_uat_120k.csv"

df=pd.read_csv(path, delimiter=';')
df2 = df[['ID', 'STATUS','CASENUMBER', 'SCC_LEGACY_TICKET_ID__C']]

# df3 = df2[df2['SCC_LEGACY_TICKET_ID__C'].notna()]
# df2.iloc[70990:]





C:\Users\maste\AppData\Local\Temp\ipykernel_17788\1435779221.py:11: DtypeWarning: Columns (47,48,84,108,124,130,132,189,200,203,204,229,247) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path, delimiter=';')


## Update the Supported Objects in Queue Name



In [35]:
import pandas as pd

# First Extract all the Queue ID from Group Object, you may use this query:
# SELECT Id
# where Type ='Queue'


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_group_queueid.csv"

df = pd.read_csv(path)

# add one column 'SObjectType' = Case, see the example below:
# QueueId,SObjectType
# 00GMR0000000trT2AQ,Case

df['SObjectType']='Case'
df.to_csv('extract_group_queueid.csv', index=False)

# Go to Data Loader 'insert' and choose  'QueueSobject' and insert the file with two columns: Id and SObjectType = Case

In [1]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124041536955.csv"


df = pd.read_csv(path)
# df= df.drop('ERROR', axis=1)
# df= df.drop('ERROR.1', axis=1)
df.to_csv(path, index=False)

In [41]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_0_27.csv"
df = pd.read_csv(path, delimiter=';')
df.columns

Index(['cifno', 'Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name'],
      dtype='object')

In [45]:
import pandas as pd
import numpy as np

# Define the file path
# path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_0_27.csv"
path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_1000001_27.csv"

# Load the data
df = pd.read_csv(path, delimiter=';')

# df = df.iloc[:1]
# Change the year to 2019
df['Create_Date'] = pd.to_datetime(df['Create_Date'])
df['Create_Date'] = df['Create_Date'].apply(lambda x: x.replace(year=2019))

def generate_random_16_digits():
    return ''.join([str(np.random.randint(0, 10)) for _ in range(16)])

# Add a new column 'Nomor_Kartu' with random 16-digit numbers
df['Nomor_Kartu'] = df.apply(lambda x: generate_random_16_digits(), axis=1)

# Remove the .0 from Call_Type_ID by converting it to an integer
df['Call_Type_ID'] = '8412'
df['Call_Type_ID'] = df['Call_Type_ID'].astype(int)

def generate_random_15_to_18_digits():
    return ''.join([str(np.random.randint(0, 10)) for _ in range(np.random.randint(15, 19))])

# Update the 'No_Rekening' column with random 15 to 18 digit numbers
df['No_Rekening'] = df.apply(lambda x: generate_random_15_to_18_digits(), axis=1)

# add recordtype
df['record_type']='Case Migration'

df.to_csv('bricare_27col_100k_2.csv', index=False)


# df=df[df['Ticket_ID'] == 'TTB000025269322']
# df

# Extract just the 'Ticket_ID' column and sort it
# df2 = df['Ticket_ID'].sort_values(ascending=True)
# df2

# Display the sorted Series
# df2.to_csv('tiket_bricare.csv', index=False)


#dummy 120k 

In [21]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv"

df=pd.read_csv(path)



df['Ticket_ID'] = df['Ticket_ID'].str.replace('TTB', 'TICKET')

df.to_csv('bricare_120k_ticket.csv', index=False)

In [22]:
import pandas as pd

# Load the data files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\extract_dummyfromserver_120k.csv.xls")
bricare_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")

# Check the lengths of the CaseNumber and Ticket_ID columns
extract_length = len(extract_df['CaseNumber'])
bricare_length = len(bricare_df['Ticket_ID'])

# Ensure the number of CaseNumber matches the number of Ticket_ID
if extract_length >= bricare_length:
    # Trim the extract dataframe to match the length of the bricare dataframe
    trimmed_extract_df = extract_df.iloc[:bricare_length]
    
    # Replace Ticket_ID values with CaseNumber values
    bricare_df['Ticket_ID'] = trimmed_extract_df['CaseNumber'].values
    
    # Save the updated bricare dataframe to a new CSV file
    output_file_path = 'updated_bricare_120k_ticket.csv'
    # bricare_df.to_csv(output_file_path, index=False)
    
    print("Ticket_ID values have been successfully replaced and saved to 'updated_bricare_120k_ticket.csv'.")
else:
    print("Error: The number of CaseNumber entries is less than the number of Ticket_ID entries. Ensure they are one-to-one.")

# Display the first few rows of the updated bricare dataframe
bricare_df.head()


Ticket_ID values have been successfully replaced and saved to 'updated_bricare_120k_ticket.csv'.


,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,Nominal,...,email,full_name,no_telepon,approver_login,approver_name,SLAResolution,submitter_login_id,submitter_user_group,user_login_name,record_type
0,LD86852,TICKET000998035,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b2714defcfef8c65d3154367beb604aa,878667067646102,23300800.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
1,RCAW297,TICKET000998036,8412,Nasabah BRI gagal tarik tunai & terdebet di AT...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b69e3868bc8b6c3231df7d691a5595fe,5073214431798791,1000000.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
2,AOY4038,TICKET000998037,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,948421d448fb51f1cf27584b1256a120,6430217575043687,246300.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
3,AUA0263,TICKET000998038,8412,Nasabah BRI gagal transfer & terdebet jaringan...,2019-08-06 16:52:14,Phone,Complaint - Transaction,a4aa532c65d1503dcc41ef396d3c42b8,4067829670996616,100000.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
4,HOL2556,TICKET000998039,8412,Kartu ATM BRI Tertelan di MESIN ATM,2019-08-06 16:52:14,Phone,Request,95d4f5421b5b387726b5d619dbe24155,272778316775473552,0.0,...,NaN,fcc2dc136c7c3d69fb080cdd2b1648f1,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90119221.0,LCC-CCTCALL,fcc2dc136c7c3d69fb080cdd2b1648f1,Case Migration


In [30]:
import pandas as pd

path=r"D:\dataquality2\updated_bricare_120k_ticket_with_id.csv"

df = pd.read_csv(path)

df=df.iloc[:1]
df.to_csv('updated_bricare_1_ticket.csv', index=False)


In [31]:
import pandas as pd

# Load the data files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\extract_dummyfromserver_120k.csv.xls")
bricare_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")

# Check the lengths of the CaseNumber and Ticket_ID columns
extract_length = len(extract_df['CaseNumber'])
bricare_length = len(bricare_df['Ticket_ID'])

# Ensure the number of CaseNumber matches the number of Ticket_ID
if extract_length >= bricare_length:
    # Trim the extract dataframe to match the length of the bricare dataframe
    trimmed_extract_df = extract_df.iloc[:bricare_length]
    
    # Replace Ticket_ID values with CaseNumber values
    bricare_df['Ticket_ID'] = trimmed_extract_df['CaseNumber'].values
    
    # Add the Id column from the extract dataframe to the bricare dataframe
    bricare_df['Id'] = trimmed_extract_df['Id'].values
    
    # Reorder the columns to place Id as the first column
    cols = bricare_df.columns.tolist()
    cols = ['Id'] + [col for col in cols if col != 'Id']
    bricare_df = bricare_df[cols]
    
    # Save the updated bricare dataframe to a new CSV file
    output_file_path = 'updated_bricare_120k_ticket_with_id.csv'
    bricare_df['Nomor_Kartu']='1706269727199000'
    bricare_df['No_Rekening']='878667067646102'
    
    bricare_df.to_csv(output_file_path, index=False)
    
    print("Ticket_ID values have been successfully replaced and the Id column has been added and saved to 'updated_bricare_120k_ticket_with_id.csv'.")
else:
    print("Error: The number of CaseNumber entries is less than the number of Ticket_ID entries. Ensure they are one-to-one.")

# Display the first few rows of the updated bricare dataframe
bricare_df.head()


Ticket_ID values have been successfully replaced and the Id column has been added and saved to 'updated_bricare_120k_ticket_with_id.csv'.


,Id,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,...,email,full_name,no_telepon,approver_login,approver_name,SLAResolution,submitter_login_id,submitter_user_group,user_login_name,record_type
0,500MR000004Q2wkYAC,LD86852,TICKET000998035,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b2714defcfef8c65d3154367beb604aa,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
1,500MR000004Q2wlYAC,RCAW297,TICKET000998036,8412,Nasabah BRI gagal tarik tunai & terdebet di AT...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b69e3868bc8b6c3231df7d691a5595fe,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
2,500MR000004Q2wmYAC,AOY4038,TICKET000998037,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,948421d448fb51f1cf27584b1256a120,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
3,500MR000004Q2wnYAC,AUA0263,TICKET000998038,8412,Nasabah BRI gagal transfer & terdebet jaringan...,2019-08-06 16:52:14,Phone,Complaint - Transaction,a4aa532c65d1503dcc41ef396d3c42b8,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
4,500MR000004Q2woYAC,HOL2556,TICKET000998039,8412,Kartu ATM BRI Tertelan di MESIN ATM,2019-08-06 16:52:14,Phone,Request,95d4f5421b5b387726b5d619dbe24155,878667067646102,...,NaN,fcc2dc136c7c3d69fb080cdd2b1648f1,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90119221.0,LCC-CCTCALL,fcc2dc136c7c3d69fb080cdd2b1648f1,Case Migration


In [44]:
from datetime import datetime
import pytz

# Original date and time in UTC
utc_time_str_2 = "2024-06-26T21:30:20.000+0000"
utc_time_2 = datetime.strptime(utc_time_str_2, "%Y-%m-%dT%H:%M:%S.%f%z")

# Convert to Jakarta time
jakarta_tz = pytz.timezone("Asia/Jakarta")
jakarta_time_2 = utc_time_2.astimezone(jakarta_tz)

# Format the Jakarta time as a string
jakarta_time_str_2 = jakarta_time_2.strftime("%Y-%m-%d %H:%M:%S")

jakarta_time_str_2


'2024-06-27 04:30:20'

In [69]:
from datetime import datetime
import pytz

# Jakarta date and time string
jakarta_time_str_2 = "2024-06-27 01:10:20"

# Define Jakarta timezone
jakarta_tz = pytz.timezone("Asia/Jakarta")

# Parse the Jakarta time string into a datetime object
jakarta_time_2 = datetime.strptime(jakarta_time_str_2, "%Y-%m-%d %H:%M:%S")

# Localize the Jakarta time
jakarta_time_2 = jakarta_tz.localize(jakarta_time_2)

# Convert to UTC
utc_time_2 = jakarta_time_2.astimezone(pytz.utc)

# Format the UTC time as a string
utc_time_str_2 = utc_time_2.strftime("%Y-%m-%dT%H:%M:%S.%f%z")

utc_time_str_2


'2024-06-26T18:10:20.000000+0000'

In [64]:
# Reload the bricare files and the 220k file
file_220k = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\220K.csv.xls")
bricare_1 = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")
bricare_2 = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k_2.csv")

# Merge Id and CaseNumber into bricare files
bricare_1 = bricare_1.merge(file_220k[['Id', 'CaseNumber']], left_index=True, right_index=True)
bricare_2 = bricare_2.merge(file_220k[['Id', 'CaseNumber']], left_index=True, right_index=True)

# Replace Ticket_ID with CaseNumber from 220K file
bricare_1['Ticket_ID'] = bricare_1['CaseNumber']
bricare_2['Ticket_ID'] = bricare_2['CaseNumber']

# Drop the temporary 'CaseNumber' column
bricare_1.drop(columns=['CaseNumber'], inplace=True)
bricare_2.drop(columns=['CaseNumber'], inplace=True)

# Combine the Ticket_IDs from both files to identify overlaps
combined_ticket_ids = pd.concat([bricare_1['Ticket_ID'], bricare_2['Ticket_ID']])

# Identify duplicate Ticket_IDs
duplicates = combined_ticket_ids[combined_ticket_ids.duplicated()]

# Reassign duplicates in bricare_2 to ensure uniqueness
for i, dup in enumerate(duplicates):
    bricare_2.loc[bricare_2['Ticket_ID'] == dup, 'Ticket_ID'] += f'_DUP{i}'

# Save the transformed files
bricare_1.to_csv('bricare_27col_100k_transformed_no_suffix.csv', index=False)
bricare_2.to_csv('bricare_27col_100k_2_transformed_no_suffix.csv', index=False)

# Check if Ticket_IDs are completely different between the two files
ticket_id_1 = set(bricare_1['Ticket_ID'])
ticket_id_2 = set(bricare_2['Ticket_ID'])

# Ensure no Ticket_ID is shared between the two files
unique_across_files = ticket_id_1.isdisjoint(ticket_id_2)


unique_across_files

True

In [65]:
import pandas as pd

# Load the files into pandas DataFrames
file1 = pd.read_csv(r"D:\dataquality2\bricare_27col_100k_transformed_no_suffix.csv")
file2 = pd.read_csv(r"D:\dataquality2\bricare_27col_100k_2_transformed_no_suffix.csv")

# Extract the Ticket_ID column from both DataFrames
ticket_ids_file1 = file1['Ticket_ID']
ticket_ids_file2 = file2['Ticket_ID']

# Check if there are any common values
common_ids = set(ticket_ids_file1).intersection(set(ticket_ids_file2))

if len(common_ids) == 0:
    print("All values in the Ticket_ID column are completely different between the two files.")
else:
    print("There are common values in the Ticket_ID column between the two files.")
    print("Common values:", common_ids)





All values in the Ticket_ID column are completely different between the two files.


## Compare the Group Name between SF and BRicare

from pak Suyanto
"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Group name ds_ej.csv"

from SF
"C:\Users\maste\Downloads\dataloader_v60.0.2\server\CSV 2 - Group.csv"

In [76]:
import pandas as pd
import csv

import pandas as pd


file1_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\ds_ej.csv"
file2_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\CSV 2 - Group.csv"

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)


user_groups_ds_ej = df1['usergrouptxt'].str.strip()
queue_group_names = df2['Queue & Group Name'].str.strip()


missing_groups = user_groups_ds_ej[~user_groups_ds_ej.isin(queue_group_names)]

missing_groups_list = missing_groups.tolist()
len(missing_groups_list)

996

In [6]:
# Load the CSV and Excel files
csv_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Pak Hadi Green.xlsx"
excel_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned.xlsx"

# Read the files
df_csv = pd.read_excel(csv_file_path)
df_excel = pd.read_excel(excel_file_path)

# Extract the relevant columns for comparison
case_types = set(df_csv['Case Types'].astype(str))
call_type_numbers = set(df_excel['Call Type Number'].astype(str))

# Find the missing and common case types
missing_case_types = case_types - call_type_numbers
common_case_types = case_types & call_type_numbers

# Save the results to CSV files
missing_case_types_df = pd.DataFrame(missing_case_types, columns=["Missing Case Types"])
common_case_types_df = pd.DataFrame(common_case_types, columns=["Common Case Types"])

missing_case_types_df.to_csv('case_types_missing.csv', index=False)
common_case_types_df.to_csv('case_types_common.csv', index=False)

# Provide the links to download the files
len(missing_case_types_df), len(common_case_types_df)


(10, 345)

In [23]:
import pandas as pd

# path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned - TBM.xlsx"

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned - TBM.xlsx"


df= pd.read_excel(path)
pd.set_option('display.max_colwidth', None)
# df = df[df['Call Type Number']== 'UMI-3'] # Modify this
df['Send to Drone?'] = df['Send to Drone?'].replace({'No': 'FALSE', 'Yes': 'TRUE'})
df['Active'] = df['Active'].astype(str).str.upper()
df['external_id']=df['Call Type Number']


# df.to_csv('CallType_umi3.csv', index=False)
df.to_csv('CallType_cleaned.csv', index=False)
df

,Call Type Number,Active,Send to Drone?,Additional Details (Formulir Detail),Customer Segment,Product,Sub Product,Case Category,Case Type Description,Sub-Description,...,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,external_id
0,1000,TRUE,FALSE,nomor ponsel nasabah yang bisa dihubungi: \nID CERIA:\nNO HP yg terdaftar:\nAlasan:,Individu Umum,Loans,KTA - Digital,Inquiry,Nasabah Menanyakan Informasi Pengajuan Terkait CERIA,"1. Cara, Syarat & Ketentuan\n2. Status\n3. Pembatalan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
1,1002,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Products / Promotion Inquiry,Nasabah Menyakan Terkait Promo dan Program CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002
2,1003,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pelunasan Awal Cicilan CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003
3,1005,TRUE,FALSE,Nasabah mengajukan pemblokiran Sementara Akun Ceria\nNomor ID :\nAlasan pemblokiran :\n\nMohon bantuan tindak lanjut\nTerima kasih,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pemblokiran CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1005
4,1008,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pengaktifan Akun Ceria Terblokir karena Fraud Detection System,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,TRB-8,TRUE,FALSE,NaN,Korporasi,Servicing,Cash Management,Inquiry,Nasabah Menanyakan Pengajuan Pendaftaran Product BRIVA tidak disetujui pada QLOLA Apps,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRB-8
380,TRB-9,TRUE,FALSE,NaN,Korporasi,Servicing,Cash Management,Inquiry,Nasabah Menanyakan Pengajuan Pendaftaran Product Corporate Billing Management (BRI CBM) tidak disetujui pada QLOLA Apps,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRB-9
381,UMI-1,TRUE,FALSE,NaN,UMKM,Transaction Banking,Merchant Solutions,Inquiry,"Calon Mitra UMI Menanyakan Cara, Syarat, dan Ketentuan mengenai Pengajuan Menjadi Mitra UMI",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UMI-1
382,UMI-2,TRUE,FALSE,NaN,UMKM,Transaction Banking,Merchant Solutions,Inquiry,Mitra UMI Menanyakan Informasi Fee Transaksi,1. Disbursment\n2. Loan quality fee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UMI-2


# Casenumber

In [6]:
import pandas as pd

def check_ticket_ids(file_path, case_column, output_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Extract the numeric part of the ticket IDs
    df['Numeric_ID'] = df[case_column].str.extract(r'(\d+)')
    
    # Drop rows with NaN values in 'Numeric_ID'
    df = df.dropna(subset=['Numeric_ID'])
    
    # Convert the numeric part to integers
    df['Numeric_ID'] = df['Numeric_ID'].astype(int)
    
    # Sort the DataFrame by the numeric ID
    df_sorted = df.sort_values(by='Numeric_ID').reset_index(drop=True)
    
    # Find duplicates
    duplicates = df_sorted[df_sorted.duplicated(subset='Numeric_ID', keep=False)]
    
    # Find gaps
    df_sorted['Next_ID'] = df_sorted['Numeric_ID'].shift(-1)
    df_sorted['Gap'] = df_sorted['Next_ID'] - df_sorted['Numeric_ID']
    gaps = df_sorted[df_sorted['Gap'] != 1]
    
    # Detailed gap analysis
    gap_details = []
    for idx, row in gaps.iterrows():
        if not pd.isna(row['Next_ID']):
            gap_start = row[case_column]
            gap_end = df_sorted.iloc[idx + 1][case_column]
            gap_size = row['Gap'] - 1
            gap_details.append({'Gap_Start': gap_start, 'Gap_End': gap_end, 'Gap_Size': gap_size})
    
    # Save results to CSV files
    if not duplicates.empty:
        duplicates.to_csv(output_path + '_duplicates.csv', index=False)
        print("Duplicates found and saved to CSV.")
    else:
        print("No duplicates found.")
    
    if gap_details:
        gaps_df = pd.DataFrame(gap_details)
        gaps_df.to_csv(output_path + '_gaps.csv', index=False)
        print("Gaps found and saved to CSV.")
    else:
        print("No gaps found.")

# Example usage
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Case_2024-07-03_okta.csv"
case_column = 'CASENUMBER'
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\output"

check_ticket_ids(file_path, case_column, output_path)
#select SCC_Legacy_Ticket_ID__c, CaseNumber from case where SCC_Legacy_Ticket_ID__c ='TTB0000088100' 


Duplicates found and saved to CSV.
Gaps found and saved to CSV.


In [1]:
# To split the extracted files, the file containing all Legacy ID having non-empty values


import os
import pandas as pd

def split_csv(input_file, output_directory, output_prefix, rows_per_file=200000):
    # Read the CSV file with semicolon as the delimiter
    df = pd.read_csv(input_file, delimiter=';')

    # Ensure the SCC_LEGACY_TICKET_ID__C column is empty
    df['SCC_LEGACY_TICKET_ID__C'] = ''

    # Get the total number of rows in the DataFrame
    total_rows = len(df)

    # Calculate the number of files needed
    num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file != 0 else 0)

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    for i in range(num_files):
        start_row = i * rows_per_file
        end_row = start_row + rows_per_file

        # Slice the DataFrame to get the chunk
        chunk = df.iloc[start_row:end_row]

        # Create a filename for the chunk
        output_file = os.path.join(output_directory, f"{output_prefix}_part_{i+1}.csv")

        # Save the chunk to a CSV file, including the header
        chunk.to_csv(output_file, index=False, sep=';')

        print(f"Saved {output_file}")

# Example usage
if __name__ == "__main__":
    input_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\extract.csv"       
    output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2"     
    output_prefix = 'output'
    rows_per_file = 200000                

    split_csv(input_file, output_directory, output_prefix, rows_per_file)


Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_1.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_2.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_3.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_4.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_5.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_6.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_7.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_8.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_9.csv


In [3]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\Dummy\dummy_data_casenumber60_part600.csv"

df = pd.read_csv(path)
#
df=df.iloc[:63000]
# df = df.drop(columns=['SUBJECT','PRIORITY','STATUS'])
df.to_csv(path, index=False)

df

,Status,Priority,Call Type,record_type,Legacy_ticket_id
0,Cancelled,Low,1000,Case Migration,TTB0061614201
1,Cancelled,Low,1000,Case Migration,TTB0061614202
2,Cancelled,Low,1000,Case Migration,TTB0061614203
3,Cancelled,Low,1000,Case Migration,TTB0061614204
4,Cancelled,Low,1000,Case Migration,TTB0061614205
...,...,...,...,...,...
62995,Cancelled,Low,1000,Case Migration,TTB0061677196
62996,Cancelled,Low,1000,Case Migration,TTB0061677197
62997,Cancelled,Low,1000,Case Migration,TTB0061677198
62998,Cancelled,Low,1000,Case Migration,TTB0061677199


# List of Account files in server

In [1]:
import pytesseract
from PIL import Image
import pandas as pd

# Path to your image file
image_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\WhatsApp Image 2024-07-05 at 15.51.41_53d1f445.jpg"

# Load the image using PIL
image = Image.open(image_path)

# Perform OCR on the image to extract text
extracted_text = pytesseract.image_to_string(image)

# Split the text into lines
lines = extracted_text.split('\n')

# Filter out non-CSV lines and sort them in ascending order
csv_files = sorted([line for line in lines if 'csv' in line])

# Create a DataFrame to display the sorted list of CSV files
csv_files_df = pd.DataFrame(csv_files, columns=['CSV Files'])

# Save the DataFrame to an Excel file
sorted_excel_file_path = "sorted_csv_file_names.xlsx"
csv_files_df.to_excel(sorted_excel_file_path, index=False)

print(f"Excel file with sorted CSV names saved at: {sorted_excel_file_path}")


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

# Update the Legacy Ticket Id into blank or NuLL

In [18]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonemppty77_cleaned2.csv"


df = pd.read_csv(path, delimiter=';')
df['SCC_LEGACY_TICKET_ID__C']=''
df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonemppty77_cleaned2.csv",index=False)
df.head()

,ID,SCC_LEGACY_TICKET_ID__C,CASENUMBER
0,500Mg000005shaOIAQ,,TTB0000122839
1,500Mg000005shaPIAQ,,TTB0000122840
2,500Mg000005shaQIAQ,,TTB0000122841
3,500Mg000005shaRIAQ,,TTB0000122842
4,500Mg000005shaSIAQ,,TTB0000122843


In [14]:
import pandas as pd

# Load the uploaded CSV file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonempty_1line.csv"
df = pd.read_csv(file_path, delimiter=';')

df.replace('', pd.NA, inplace=True)

df['case_origin']='X'
df['SCC_LEGACY_TICKET_ID__C']=''
df_cleaned_specific = df.dropna(subset=['ID'])

df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\cleaned_prod_1line.csv",index=False)
df_cleaned_specific.head()


,ID,SCC_LEGACY_TICKET_ID__C,CASENUMBER,case_origin
0,500Mg000005sZYzIAM,,TTB0000088100,X


# List Account

In [66]:
import re

# Path to the file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\list_acount.txt"

# Regular expression pattern to match the filenames
pattern = re.compile(r'bricare_\d{8}_\d{8}_\d+_bricare_case_account_\w+\.csv')

# List to store the extracted filenames
filenames = []

# Read the file and extract filenames
with open(file_path, 'r') as file:
    for line in file:
        matches = pattern.findall(line)
        filenames.extend(matches)

# Display the extracted filenames
for filename in filenames:
    print(filename)

# Optionally, save the filenames to a new file
output_path = 'extracted_filenames.txt'
with open(output_path, 'w') as output_file:
    for filename in filenames:
        output_file.write(f"{filename}\n")

print(f"Extracted filenames saved to {output_path}")


bricare_20200806_20240430_0_bricare_case_account_business.csv
bricare_20200806_20240430_0_bricare_case_account_person_account.csv
bricare_20200806_20240430_10000001_bricare_case_account_business.csv
bricare_20200806_20240430_10000001_bricare_case_account_person_account.csv
bricare_20200806_20240430_1000001_bricare_case_account_business.csv
bricare_20200806_20240430_1000001_bricare_case_account_person_account.csv
bricare_20200806_20240430_100001_bricare_case_account_business.csv
bricare_20200806_20240430_100001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10100001_bricare_case_account_business.csv
bricare_20200806_20240430_10100001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10200001_bricare_case_account_business.csv
bricare_20200806_20240430_10200001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10300001_bricare_case_account_business.csv
bricare_20200806_20240430_10300001_bricare_case_account_person_account.csv
bricare_2020

In [85]:
import re
import dns.resolver

def is_valid_email(email):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if re.match(regex, email) is None:
        return False
    
    domain = email.split('@')[1]
    try:
        dns.resolver.resolve(domain, 'MX')
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN, dns.resolver.Timeout):
        return False
    
    return True

def correct_email(email):
    corrections = {
        'gmial.com': 'gmail.com',
        'gamil.com': 'gmail.com',
        'yahooo.com': 'yahoo.com',
        'hotmial.com': 'hotmail.com',
        'hotmal.com': 'hotmail.com',
    }
    
    if '@' not in email:
        return email
    
    local_part, domain = email.split('@')
    
    # Common typo corrections for domains
    if domain in corrections:
        domain = corrections[domain]
    
    # Repeated characters correction (e.g., "john..doe@gmail.com")
    local_part = re.sub(r'\.{2,}', '.', local_part)
    
    corrected_email = f"{local_part}@{domain}"
    
    if is_valid_email(corrected_email):
        return corrected_email
    else:
        return email

# Example usage
emails = [
    "john..doe@gmial.com",
    "invalid-email.com",
    "jane.doe@yahooo.com",
    "test@invalid-domain.com",
    "hilmi@gmail.com master"
]

corrected_emails = [correct_email(email) for email in emails]
print("Corrected emails:", corrected_emails)


Corrected emails: ['john.doe@gmail.com', 'invalid-email.com', 'jane.doe@yahoo.com', 'test@invalid-domain.com', 'hilmi@gmail.com master']


# To Compare the Queue Name and Excalation Team

In [ ]:
# To Compare the Queue Name and Excalation Team

import pandas as pd
from difflib import get_close_matches

# Load the Excel file
excel_file_path = '/mnt/data/(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (3).xlsx'
excel_data = pd.read_excel(excel_file_path, sheet_name='Appendix Team Name')

# Load the CSV file
csv_file_path = '/mnt/data/extract_Que.csv'
extract_que_df = pd.read_csv(csv_file_path)

# Extract relevant columns
appendix_names = excel_data['Full Name / Queue Name'].dropna().unique()
extract_names = extract_que_df['NAME'].dropna().unique()

# Find matches
matches = {name: get_close_matches(name, extract_names, n=1, cutoff=0.6) for name in appendix_names}

# Filter out names without matches
matches = {k: v for k, v in matches.items() if v}

# Convert matches to a DataFrame for better readability
matches_df = pd.DataFrame([(k, v[0]) for k, v in matches.items()], columns=['Appendix Team Name', 'Extract Name'])

# Define a function to describe the differences between two strings
def describe_difference(str1, str2):
    if str1 == str2:
        return "Exact match"
    differences = []
    if len(str1) != len(str2):
        differences.append("Different lengths")
    if str1.strip() != str2.strip():
        differences.append("Whitespace differences")
    for i, (char1, char2) in enumerate(zip(str1, str2)):
        if char1 != char2:
            differences.append(f"Different character at position {i}: '{char1}' vs '{char2}'")
    return "; ".join(differences) if differences else "Other differences"

# Add a column describing the differences
matches_df['Difference Details'] = matches_df.apply(lambda row: describe_difference(row['Appendix Team Name'], row['Extract Name']), axis=1)

import ace_tools as tools; tools.display_dataframe_to_user(name="Matched Names with Differences", dataframe=matches_df)

matches_df.head()


In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\Matched_Names_with_Differences.csv"

df=pd.read_csv(path)
df.to_excel('match_queue_name_vs_escalation_name.xlsx', index=False)



# Combine all error logs into 2 different files: Business and Person Account

In [10]:
import os
import pandas as pd

folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account"


all_files = os.listdir(folder_path)


error_files = [f for f in all_files if "error" in f]


business_data = pd.DataFrame()
person_account_data = pd.DataFrame()


for file_name in error_files:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path, delimiter=';')
    

    data['Nama'] = data['Nama'].apply(lambda x: x[-250:] if isinstance(x, str) else x)
    
    # combine the columns "Email" and "no telp"
    data['Description'] = data[['No_telp', 'Email']].apply(lambda x: ' | '.join(x.dropna().astype(str)), axis=1)
    

    data = data.drop(columns=['No_telp', 'Email'])
    

    business_data = pd.concat([business_data, data[data['record_type'] == 'Business']])
    person_account_data = pd.concat([person_account_data, data[data['record_type'] == 'Person Account']])




business_file_path = os.path.join(folder_path, 'account_business_records.csv')
person_account_file_path = os.path.join(folder_path, 'account_person_account_records.csv')


business_data.to_csv(business_file_path, index=False)
person_account_data.to_csv(person_account_file_path, index=False)

print(f"Business records saved to: {business_file_path}")
print(f"Person Account records saved to: {person_account_file_path}")

Business records saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\account_business_records.csv
Person Account records saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\account_person_account_records.csv


# To check and analyze the Account data

In [ ]:
import csv
import os

def detect_delimiter(file_path):
    with open(file_path, 'r') as file:
        header = file.readline()
        for delimiter in [',', ';', '\t', '|']:
            if delimiter in header:
                return delimiter
    return None

def analyze_file(file_path, expected_delimiter):
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter=expected_delimiter)
        line_number = 0
        inconsistent_lines = []
        for row in reader:
            line_number += 1
            if len(row) != len(reader.fieldnames):
                inconsistent_lines.append(line_number)
        return inconsistent_lines

def main():
    file_paths = ["path_to_your_first_csv_file.csv", "path_to_your_second_csv_file.csv"]  

    for file_path in file_paths:
        if not os.path.isfile(file_path):
            print(f"File not found: {file_path}")
            continue
        
        detected_delimiter = detect_delimiter(file_path)
        if detected_delimiter:
            print(f"Detected delimiter for {file_path}: {detected_delimiter}")
            inconsistent_lines = analyze_file(file_path, detected_delimiter)
            if inconsistent_lines:
                print(f"Inconsistent lines in {file_path}: {inconsistent_lines}")
            else:
                print(f"No inconsistencies found in {file_path}")
        else:
            print(f"Could not detect delimiter for {file_path}")

if __name__ == "__main__":
    main()


In [2]:
import csv
import os

def detect_delimiter(file_path):
    with open(file_path, 'r') as file:
        header = file.readline()
        for delimiter in [',', ';', '\t', '|']:
            if delimiter in header:
                return delimiter
    return None

def analyze_file(file_path, expected_delimiter):
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter=expected_delimiter)
        line_number = 0
        inconsistent_lines = []
        field_count = len(next(reader))
        for row in reader:
            line_number += 1
            if len(row) != field_count:
                inconsistent_lines.append(line_number)
        return inconsistent_lines

def process_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                detected_delimiter = detect_delimiter(file_path)
                if detected_delimiter:
                    print(f"Detected delimiter for {file_path}: {detected_delimiter}")
                    inconsistent_lines = analyze_file(file_path, detected_delimiter)
                    if inconsistent_lines:
                        print(f"Inconsistent lines in {file_path}: {inconsistent_lines}")
                    else:
                        print(f"No inconsistencies found in {file_path}")
                else:
                    print(f"Could not detect delimiter for {file_path}")

def main():
    folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\test"
    if os.path.isdir(folder_path):
        process_folder(folder_path)
    else:
        print(f"Folder not found: {folder_path}")

if __name__ == "__main__":
    main()


Processing file: C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv
Detected delimiter for C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv: ;
Inconsistent lines in C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv: [1]
Processing file: C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv
Detected delimiter for C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv: ;
Inconsistent lines in C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv: [2, 3]


In [4]:
import pandas as pd




import csv

# Input and output file paths
input_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\error071124083558031.csv"
output_file = r"c:\Users\maste\Downloads\dataloader_v60.0.2\server\account\error071124083558030_output.csv"

# Read the input CSV file
with open(input_file, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile, delimiter=';')
    rows = list(reader)

# Process the rows to add double quotes where necessary
processed_rows = []
for row in rows:
    if row == rows[0]:
        # Keep the header row as is
        processed_rows.append(row)
    else:
        # Add double quotes around non-empty values
        processed_row = ['' if value == '""' else f'"{value}"' if value else '' for value in row]
        processed_rows.append(processed_row)

# Write the processed rows to the output CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile, delimiter=';')
    writer.writerows(processed_rows)

print("CSV file has been processed and saved.")


CSV file has been processed and saved.


In [6]:
import os
import pandas as pd

def save_shapes_to_excel(folder_path, output_file):
    data = []


    for filename in os.listdir(folder_path):

        if filename.endswith('.csv'):
     
            file_path = os.path.join(folder_path, filename)
      
            df = pd.read_csv(file_path)
    
            rows, columns = df.shape
         
            data.append([filename, rows, columns])

 
    df_shapes = pd.DataFrame(data, columns=['Name', 'Rows', 'Columns'])


    df_shapes.to_excel(output_file, index=False)

    print(f"Shapes of CSV files saved to {output_file}")


folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account"
output_file = 'csv_shapes.xlsx'


save_shapes_to_excel(folder_path, output_file)


Shapes of CSV files saved to csv_shapes.xlsx


# Role 

In [5]:
import pandas as pd
import re

# Load the Excel file
file_path = r"C:\Users\maste\Downloads\BRI - User Matrix v1.2.xlsx"
df = pd.read_excel(file_path, sheet_name='Role2')

df['']

df.to_csv('Role2.csv',index=False)

In [8]:
import pandas as pd

path=r"D:\dataquality2\bricare_uat20230101_20230101.csv"

df= pd.read_csv(path)

df['Chanel'].unique()

array([nan, 'ATM BANK LAIN', 'Aplikasi', 'EDC', 'CRM', 'ATM', 'ATM BRI'],
      dtype=object)

In [31]:
import os
import pandas as pd

def format_ticket_id(ticket_id):
    if pd.notna(ticket_id) and ticket_id.startswith("TTB"):
        number_part = ticket_id[3:]
        formatted_number = number_part[-10:].zfill(10)
        return f"TTB{formatted_number}"
    return ticket_id

def process_file(file_path):
    try:
        df = pd.read_csv(file_path)
        if "Ticket_ID" in df.columns:
            df["Ticket_ID"] = df["Ticket_ID"].apply(format_ticket_id)
        if "Legacy_Ticket_ID" in df.columns:
            df["Legacy_Ticket_ID"] = df["Legacy_Ticket_ID"].apply(format_ticket_id)
        output_path = file_path.replace(".csv", "_processed.csv")
        df.to_csv(output_path, index=False)
        print(f"Processed and saved: {output_path}")
        return df
    except PermissionError as e:
        print(f"PermissionError: {e}")
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)


folder_path = r"D:\test_case - Copy"
process_all_files_in_folder(folder_path)


Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed.csv


In [32]:
import os
import csv

def format_ticket_id(ticket_id):
    if ticket_id and ticket_id.startswith("TTB"):
        number_part = ticket_id[3:]
        formatted_number = number_part[-10:].zfill(10)
        return f"TTB{formatted_number}"
    return ticket_id

def process_file(file_path):
    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            rows = list(reader)
            fieldnames = reader.fieldnames
        

        for row in rows:
            if "Ticket_ID" in row:
                row["Ticket_ID"] = format_ticket_id(row["Ticket_ID"])
            if "Legacy_Ticket_ID" in row:
                row["Legacy_Ticket_ID"] = format_ticket_id(row["Legacy_Ticket_ID"])
        

        output_path = file_path.replace(".csv", "_processed.csv")
        with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
        
        print(f"Processed and saved: {output_path}")
    except PermissionError as e:
        print(f"PermissionError: {e}")
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)


folder_path = r"D:\test_case - Copy"
process_all_files_in_folder(folder_path)



Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed_processed.csv


In [26]:
import pandas as pd

path=r"D:\test_case - Copy\dummy_data_case_uat - 79 - Copy.csv"

df=pd.read_csv(path)
df

,case_number,record_type,cif,Ticket_ID,Call_Type_ID,Call_Type,Details,Create_Date,gateway,Jenis_Laporan,...,Tanggal_Settlement,Tgl_Foward,Tgl_In_Progress,Tgl_Returned,Ticket_Referensi,Tiket_Urgency,Tipe_Remark,UniqueID,users,Usergroup_ID
0,TICKET001339022,Case Migration,STGD 980,TTB001339022,1000,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,2023-01-01 07:07:15,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4


# Spliter

In [8]:
import os
import pandas as pd

def split_file(input_file, output_dir, rows_per_file):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read the large file
    data = pd.read_csv(input_file)
    
    # Calculate the number of smaller files needed
    num_files = len(data) // rows_per_file + (1 if len(data) % rows_per_file != 0 else 0)
    
    for i in range(num_files):
        # Determine the start and end row for the current chunk
        start_row = i * rows_per_file
        end_row = min((i + 1) * rows_per_file, len(data))
        
        # Slice the data for the current chunk
        chunk = data.iloc[start_row:end_row]
        
        # Determine the output file path
        output_file = os.path.join(output_dir, f'file_part_{i+1}.csv')
        
        # Save the chunk to a CSV file with column headers
        chunk.to_csv(output_file, index=False)
        
        print(f'Saved {output_file}')
        
if __name__ == "__main__":
    # Example usage
    # input_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_3mil_ttb.csv"
    # output_dir = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\Delete TTB prod_300k"
    input_file = r"D:\Case\duplicate_records_to_delete.csv"
    output_dir = r"D:\Case"
    rows_per_file = 2000000
    
    split_file(input_file, output_dir, rows_per_file)


Saved D:\Case\file_part_1.csv
Saved D:\Case\file_part_2.csv
Saved D:\Case\file_part_3.csv


In [15]:
import os
import pandas as pd

# Define the directory containing the CSV files
input_directory  = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
dummy_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_dummy\dummy_data_casenumber_fix_part1.csv"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output"



os.makedirs(output_directory, exist_ok=True)

# Placeholder data from the dummy file
dummy_df = pd.read_csv(dummy_file)
dummy_row = dummy_df.iloc[0].to_dict()

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
all_ticket_ids_27 = set()
all_ticket_ids_79 = set()
csv_files_27 = []
csv_files_79 = []

for file in os.listdir(input_directory):
    if file.endswith('_27.csv'):
        csv_files_27.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'])
        all_ticket_ids_27.update(df['Ticket_ID'].unique())
    elif file.endswith('_79.csv'):
        csv_files_79.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'])
        all_ticket_ids_79.update(df['Ticket_ID'].unique())

def process_files(csv_files, all_ticket_ids, file_suffix):
    # Generate the full range of Ticket_IDs
    all_ticket_ids = sorted(all_ticket_ids)
    min_id = int(all_ticket_ids[0][3:]) 
    max_id = int(all_ticket_ids[-1][3:])  
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = dummy_row.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = generate_legacy_ticket_id(ticket_id)
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file))
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{file.split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27')
process_files(csv_files_79, all_ticket_ids_79, '79')


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.


In [16]:
import os
import pandas as pd

# Define the directory containing the CSV files
input_directory  = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
dummy_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_dummy"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Collect all dummy data from the dummy files
dummy_files = [os.path.join(dummy_directory, file) for file in os.listdir(dummy_directory) if file.endswith('.csv')]
dummy_df = pd.concat([pd.read_csv(file) for file in dummy_files], ignore_index=True)
dummy_row = dummy_df.iloc[0].to_dict()

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
all_ticket_ids_27 = set()
all_ticket_ids_79 = set()
csv_files_27 = []
csv_files_79 = []

for file in os.listdir(input_directory):
    if file.endswith('_27.csv'):
        csv_files_27.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'])
        all_ticket_ids_27.update(df['Ticket_ID'].unique())
    elif file.endswith('_79.csv'):
        csv_files_79.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'])
        all_ticket_ids_79.update(df['Ticket_ID'].unique())

def process_files(csv_files, all_ticket_ids, file_suffix):
    # Generate the full range of Ticket_IDs
    all_ticket_ids = sorted(all_ticket_ids)
    min_id = int(all_ticket_ids[0][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    max_id = int(all_ticket_ids[-1][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = dummy_row.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = generate_legacy_ticket_id(ticket_id)
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file))
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{file.split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27')
process_files(csv_files_79, all_ticket_ids_79, '79')


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.


In [18]:
# handle other delimiter

import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\bricare_20230101_20230101_79.csv"
path2=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\bricare_20230101_20230101_79.csv"

df =pd.read_csv(path)
df['record_type']='Case Migration'
df.to_csv(path2,index=False)

In [26]:
import os
import pandas as pd

# Define the directories
input_directory  = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
dummy_directory = r"D:\output"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"


# Define the delimiter
delimiter = ','  # Change this to the appropriate delimiter if necessary (e.g., ';' for semicolon)

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to read and process dummy files in chunks
def read_dummy_files(dummy_directory, delimiter):
    dummy_row = None
    for file in os.listdir(dummy_directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(dummy_directory, file), delimiter=delimiter)
            if dummy_row is None:
                dummy_row = df.iloc[0].to_dict()
    return dummy_row

# Collect dummy data from the dummy files
dummy_row = read_dummy_files(dummy_directory, delimiter)

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
all_ticket_ids_27 = set()
all_ticket_ids_79 = set()
csv_files_27 = []
csv_files_79 = []

for file in os.listdir(input_directory):
    if file.endswith('_27.csv'):
        csv_files_27.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_27.update(df['Ticket_ID'].unique())
    elif file.endswith('_79.csv'):
        csv_files_79.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_79.update(df['Ticket_ID'].unique())

def process_files(csv_files, all_ticket_ids, file_suffix):
    # Generate the full range of Ticket_IDs
    all_ticket_ids = sorted(all_ticket_ids)
    min_id = int(all_ticket_ids[0][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    max_id = int(all_ticket_ids[-1][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = dummy_row.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = ticket_id  # Ensure Legacy_ticket_id matches Ticket_ID
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file), delimiter=delimiter)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure all Legacy_ticket_id values match Ticket_ID
    combined_df['Legacy_ticket_id'] = combined_df['Ticket_ID']

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    # Replace NaN with empty strings
    combined_df.fillna('', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{os.path.basename(file).split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False, sep=delimiter)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27')
process_files(csv_files_79, all_ticket_ids_79, '79')



Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.


In [8]:
import os
import pandas as pd

def clean_csv_files(input_folder, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            # Construct the full file path
            file_path = os.path.join(input_folder, filename)
            
            # Read the CSV file
            data = pd.read_csv(file_path)
            
            # Remove the 'Legacy_Ticket_ID' column if it exists
            if 'Legacy_Ticket_ID' in data.columns:
                data = data.drop(columns=['Legacy_Ticket_ID'])
            
            # Rename the 'Legacy_ticket_id' column to 'Legacy_Ticket_ID' if it exists
            if 'Legacy_ticket_id' in data.columns:
                data = data.rename(columns={'Legacy_ticket_id': 'Legacy_Ticket_ID'})
            
            # Replace all NaN values with empty strings
            data = data.fillna('')
            
            # Remove '.0' at the end of all values
            data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))
            
          
            
            # Make 'closed_date' one day after 'create_date'
            if 'Create_Date' in data.columns:
                data['Create_Date'] = pd.to_datetime(data['Create_Date'], errors='coerce')
                data['TanggalClosed'] = data['Create_Date'] + pd.DateOffset(days=1)
                data['TanggalClosed'] = data['TanggalClosed'].dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # Save the cleaned data to the output folder
            cleaned_file_path = os.path.join(output_folder, filename)
            data.to_csv(cleaned_file_path, index=False)
            print(f"Processed and saved: {cleaned_file_path}")

# Define the input and output folders
input_folder = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"
output_folder = r"D:\cleaned"

# Call the function to clean all CSV files
clean_csv_files(input_folder, output_folder)


Processed and saved: D:\cleaned\bricare_20200101_20200101_27_part1.csv
Processed and saved: D:\cleaned\bricare_20230101_20230101_79_part1.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_21452\2828063160.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))
C:\Users\maste\AppData\Local\Temp\ipykernel_21452\2828063160.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))


## USE THIS to cleanse Case Data

In [1]:
import os
import pandas as pd

# Define the directories
input_directory  = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
dummy_directory = r"D:\output"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"

# Define the delimiter
delimiter = ','  # Change this to the appropriate delimiter if necessary (e.g., ';' for semicolon)

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to read and process dummy files in chunks
def read_dummy_files(dummy_directory, delimiter):
    dummy_row = None
    for file in os.listdir(dummy_directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(dummy_directory, file), delimiter=delimiter)
            if dummy_row is None:
                dummy_row = df.iloc[0].to_dict()
    return dummy_row

# Collect dummy data from the dummy files
dummy_row = read_dummy_files(dummy_directory, delimiter)

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
all_ticket_ids_27 = set()
all_ticket_ids_79 = set()
csv_files_27 = []
csv_files_79 = []

for file in os.listdir(input_directory):
    if file.endswith('_27.csv'):
        csv_files_27.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_27.update(df['Ticket_ID'].unique())
    elif file.endswith('_79.csv'):
        csv_files_79.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_79.update(df['Ticket_ID'].unique())

def process_files(csv_files, all_ticket_ids, file_suffix):
    # Generate the full range of Ticket_IDs
    all_ticket_ids = sorted(all_ticket_ids)
    min_id = int(all_ticket_ids[0][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    max_id = int(all_ticket_ids[-1][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = dummy_row.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = ticket_id  # Ensure Legacy_ticket_id matches Ticket_ID
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file), delimiter=delimiter)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure all Legacy_ticket_id values match Ticket_ID
    combined_df['Legacy_ticket_id'] = combined_df['Ticket_ID']

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    # Replace NaN with empty strings
    combined_df.fillna('', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{os.path.basename(file).split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False, sep=delimiter)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27')
process_files(csv_files_79, all_ticket_ids_79, '79')

# To read the files in order, you can sort the filenames programmatically
output_files = sorted(os.listdir(output_directory), key=lambda x: (x.split('_')[1], int(x.split('part')[1].split('.')[0])))

for file in output_files:
    df = pd.read_csv(os.path.join(output_directory, file), delimiter=delimiter)
    print(f"Reading file: {file}")
    # Process the DataFrame as needed


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.
Reading file: bricare_20200101_20200101_27_part1.csv
Reading file: bricare_20230101_20230101_79_part1.csv


In [1]:
import os
import pandas as pd

# Define the directories
input_directory  = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
dummy_directory = r"D:\output"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"

# Define the delimiter
delimiter = ','  # Change this to the appropriate delimiter if necessary (e.g., ';' for semicolon)

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to read and process dummy files in chunks
def read_dummy_files(dummy_directory, delimiter):
    dummy_row = None
    for file in os.listdir(dummy_directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(dummy_directory, file), delimiter=delimiter)
            if dummy_row is None:
                dummy_row = df.iloc[0].to_dict()
    return dummy_row

# Collect dummy data from the dummy files
dummy_row = read_dummy_files(dummy_directory, delimiter)

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
all_ticket_ids_27 = set()
all_ticket_ids_79 = set()
csv_files_27 = []
csv_files_79 = []

for file in os.listdir(input_directory):
    if '_27' in file and file.endswith('.csv'):
        csv_files_27.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_27.update(df['Ticket_ID'].unique())
    elif '_79' in file and file.endswith('.csv'):
        csv_files_79.append(file)
        df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
        all_ticket_ids_79.update(df['Ticket_ID'].unique())

def process_files(csv_files, all_ticket_ids, file_suffix):
    # Generate the full range of Ticket_IDs
    all_ticket_ids = sorted(all_ticket_ids)
    min_id = int(all_ticket_ids[0][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    max_id = int(all_ticket_ids[-1][3:])  # Assuming Ticket_IDs are of the format 'TTBxxxxxxx'
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = dummy_row.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = ticket_id  # Ensure Legacy_ticket_id matches Ticket_ID
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file), delimiter=delimiter)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure all Legacy_ticket_id values match Ticket_ID
    combined_df['Legacy_ticket_id'] = combined_df['Ticket_ID']

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    # Replace NaN with empty strings
    combined_df.fillna('', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{os.path.basename(file).split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False, sep=delimiter)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27')
process_files(csv_files_79, all_ticket_ids_79, '79')

# To read the files in order, you can sort the filenames programmatically
output_files = sorted(os.listdir(output_directory), key=lambda x: (x.split('_')[1], int(x.split('part')[1].split('.')[0])))

for file in output_files:
    df = pd.read_csv(os.path.join(output_directory, file), delimiter=delimiter)
    print(f"Reading file: {file}")
    # Process the DataFrame as needed


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.
Reading file: bricare_20200101_20200101_27_kosong_part1.csv
Reading file: bricare_20230101_20230101_79_this_part1.csv


In [2]:
import os
import pandas as pd

def clean_csv_files(input_folder, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            # Construct the full file path
            file_path = os.path.join(input_folder, filename)
            
            # Read the CSV file
            data = pd.read_csv(file_path)
            
            # Remove the 'Legacy_Ticket_ID' column if it exists
            if 'Legacy_Ticket_ID' in data.columns:
                data = data.drop(columns=['Legacy_Ticket_ID'])
            
            # Rename the 'Legacy_ticket_id' column to 'Legacy_Ticket_ID' if it exists
            if 'Legacy_ticket_id' in data.columns:
                data = data.rename(columns={'Legacy_ticket_id': 'Legacy_Ticket_ID'})
            
            # Replace all NaN values with empty strings
            data = data.fillna('')
            
            # Remove '.0' at the end of all values
            data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))
            
            
            # Save the cleaned data to the output folder
            cleaned_file_path = os.path.join(output_folder, filename)
            data.to_csv(cleaned_file_path, index=False)
            print(f"Processed and saved: {cleaned_file_path}")

# Define the input and output folders
input_folder = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"
output_folder = r"D:\cleaned"

# Call the function to clean all CSV files
clean_csv_files(input_folder, output_folder)

Processed and saved: D:\cleaned\bricare_20200101_20200101_27_kosong_part1.csv
Processed and saved: D:\cleaned\bricare_20230101_20230101_79_this_part1.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_36988\3404566872.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))
C:\Users\maste\AppData\Local\Temp\ipykernel_36988\3404566872.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace('.0', '') if isinstance(x, (int, float)) else str(x))


In [5]:
import os
import pandas as pd

# Define the directories
input_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"

# Define the delimiter
delimiter = ','  # Change this to the appropriate delimiter if necessary (e.g., ';' for semicolon)

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
def collect_ticket_ids(input_directory, file_suffix, delimiter):
    ticket_ids = set()
    csv_files = []

    for file in os.listdir(input_directory):
        if file_suffix in file and file.endswith('.csv'):
            csv_files.append(file)
            df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
            ticket_ids.update(df['Ticket_ID'].unique())

    return sorted(ticket_ids), csv_files

all_ticket_ids_27, csv_files_27 = collect_ticket_ids(input_directory, '_27', delimiter)
all_ticket_ids_79, csv_files_79 = collect_ticket_ids(input_directory, '_79', delimiter)

def process_files(csv_files, all_ticket_ids, file_suffix, delimiter, output_directory):
    # Generate the full range of Ticket_IDs
    min_id = int(all_ticket_ids[0][3:])
    max_id = int(all_ticket_ids[-1][3:])
    full_range_ids = [f'TTB{str(i).zfill(len(all_ticket_ids[0]) - 3)}' for i in range(min_id, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Define the fixed values for the gap rows
    fixed_values = {
        'Call_Type_ID': 1000,
        'Create_Date': '1/1/1999',
        'status': 'Cancelled',
        'TanggalClosed': '1/1/1999',
        'record_type': 'Case Migration'
    }

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = fixed_values.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = ticket_id  # Ensure Legacy_ticket_id matches Ticket_ID
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file), delimiter=delimiter)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure all Legacy_ticket_id values match Ticket_ID
    combined_df['Legacy_ticket_id'] = combined_df['Ticket_ID']

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    # Replace NaN with empty strings
    combined_df.fillna('', inplace=True)
    combined_df.replace('nan', '', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{os.path.basename(csv_files[0]).split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False, sep=delimiter)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27', delimiter, output_directory)
process_files(csv_files_79, all_ticket_ids_79, '79', delimiter, output_directory)

# To read the files in order, you can sort the filenames programmatically
output_files = sorted(os.listdir(output_directory), key=lambda x: (x.split('_')[1], int(x.split('part')[1].split('.')[0])))

for file in output_files:
    df = pd.read_csv(os.path.join(output_directory, file), delimiter=delimiter)
    print(f"Reading file: {file}")
    # Process the DataFrame as needed


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.
Reading file: bricare_20200101_20200101_27_kosong_part1.csv
Reading file: bricare_20230101_20230101_79_this_part1.csv


In [8]:
import os
import pandas as pd

# Define the directories
input_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy"
output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\test_casenumber - Copy\output1"

# Define the delimiter
delimiter = ','  # Change this to the appropriate delimiter if necessary (e.g., ';' for semicolon)

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to generate the legacy ticket ID from the ticket ID
def generate_legacy_ticket_id(ticket_id):
    numeric_part = int(ticket_id[3:])
    return f'TTB{str(numeric_part).zfill(len(ticket_id) - 3)}'

# Collect all Ticket_IDs from existing files
def collect_ticket_ids(input_directory, file_suffix, delimiter):
    ticket_ids = set()
    csv_files = []

    for file in os.listdir(input_directory):
        if file_suffix in file and file.endswith('.csv'):
            csv_files.append(file)
            df = pd.read_csv(os.path.join(input_directory, file), usecols=['Ticket_ID'], delimiter=delimiter)
            ticket_ids.update(df['Ticket_ID'].unique())

    return sorted(ticket_ids), csv_files

all_ticket_ids_27, csv_files_27 = collect_ticket_ids(input_directory, '_27', delimiter)
all_ticket_ids_79, csv_files_79 = collect_ticket_ids(input_directory, '_79', delimiter)

def process_files(csv_files, all_ticket_ids, file_suffix, delimiter, output_directory):
    # Generate the full range of Ticket_IDs from TTB000000000001 to the maximum in the current data
    max_id = int(all_ticket_ids[-1][3:])
    full_range_ids = [f'TTB{str(i).zfill(12)}' for i in range(1, max_id + 1)]

    # Identify missing Ticket_IDs
    missing_ids = set(full_range_ids) - set(all_ticket_ids)

    # Define the fixed values for the gap rows
    fixed_values = {
        'Call_Type_ID': 1000,
        'Create_Date': '1/1/1999',
        'status': 'Cancelled',
        'TanggalClosed': '1/1/1999',
        'record_type': 'Case Migration'
    }

    # Create placeholder rows for missing IDs
    missing_rows = []
    for ticket_id in missing_ids:
        placeholder_row = fixed_values.copy()
        placeholder_row['Ticket_ID'] = ticket_id
        placeholder_row['Legacy_ticket_id'] = ticket_id  # Ensure Legacy_ticket_id matches Ticket_ID
        missing_rows.append(placeholder_row)
    missing_df = pd.DataFrame(missing_rows)

    # Read, combine, and split files if necessary
    combined_df = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(input_directory, file), delimiter=delimiter)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    # Add missing rows to the combined dataframe
    combined_df = pd.concat([combined_df, missing_df], ignore_index=True)
    combined_df.sort_values(by='Ticket_ID', inplace=True)

    # Ensure all Legacy_ticket_id values match Ticket_ID
    combined_df['Legacy_ticket_id'] = combined_df['Ticket_ID']

    # Remove .0 from all values except nominal column
    for col in combined_df.columns:
        if col != 'nominal':  # Replace 'nominal' with the actual name of your nominal column
            combined_df[col] = combined_df[col].apply(lambda x: str(x).replace('.0', '') if isinstance(x, float) else x)

    # Replace NaN with empty strings
    combined_df.fillna('', inplace=True)
    combined_df.replace('nan', '', inplace=True)

    # Ensure the combined dataframe doesn't exceed 300,000 rows per file
    chunk_size = 300000
    num_chunks = (len(combined_df) // chunk_size) + 1

    for i in range(num_chunks):
        chunk_df = combined_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        output_filename = os.path.join(output_directory, f'{os.path.basename(csv_files[0]).split(".csv")[0]}_part{i + 1}.csv')
        chunk_df.to_csv(output_filename, index=False, sep=delimiter)

    print(f"Processing for files with suffix '{file_suffix}' completed.")

# Process the files for both suffixes
process_files(csv_files_27, all_ticket_ids_27, '27', delimiter, output_directory)
process_files(csv_files_79, all_ticket_ids_79, '79', delimiter, output_directory)

# To read the files in order, you can sort the filenames programmatically
output_files = sorted(os.listdir(output_directory), key=lambda x: (x.split('_')[1], int(x.split('part')[1].split('.')[0])))

for file in output_files:
    df = pd.read_csv(os.path.join(output_directory, file), delimiter=delimiter)
    print(f"Reading file: {file}")
    # Process the DataFrame as needed


Processing for files with suffix '27' completed.
Processing for files with suffix '79' completed.
Reading file: bricare_20200101_20200101_27_kosong_part1.csv
Reading file: bricare_20200101_20200101_27_kosong_part2.csv
Reading file: bricare_20200101_20200101_27_kosong_part3.csv
Reading file: bricare_20200101_20200101_27_kosong_part4.csv
Reading file: bricare_20230101_20230101_79_this_part1.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_6796\2663722302.py:102: DtypeWarning: Columns (2,4,5,6,11,12,13,15,19,22,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(output_directory, file), delimiter=delimiter)


In [3]:
import pandas as pd

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_PROD24.csv"
# path=r"D:\Case\account and case 21, 22 July\bricare_case_20_21_22_07_2024\bricare_20240720_20240722_0_79_closed.csv"

df=pd.read_csv(path, sep=';')
unique_df = df.drop_duplicates(subset='SCC_LEGACY_TICKET_ID__C', keep='first')
len(unique_df)
# duplicates = df[df.duplicated(subset='Ticket_ID', keep=False)]
# duplicates_df = df[~df['SCC_LEGACY_TICKET_ID__C'].isin(unique_df['SCC_LEGACY_TICKET_ID__C'])]
# duplicates = df[df.duplicated(subset='SCC_LEGACY_TICKET_ID__C', keep=False)]
# len(duplicates_df)

# unique_ids = set()  # Initialize the set to store unique IDs
# unique_rows = []  # List to store unique rows
# duplicate_rows = []  # List to store duplicate rows

# # Read the CSV file in chunks
# chunksize = 100
# for chunk in pd.read_csv(path, delimiter=';', chunksize=chunksize):
#     for _, row in chunk.iterrows():
#         if row['SCC_LEGACY_TICKET_ID__C'] in unique_ids:
#             duplicate_rows.append(row)
#         else:
#             unique_ids.add(row['SCC_LEGACY_TICKET_ID__C'])
#             unique_rows.append(row)

# # Convert the lists to DataFrames
# unique_df = pd.DataFrame(unique_rows)
# duplicate_df = pd.DataFrame(duplicate_rows)

# # Save the DataFrames to CSV files
# unique_df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\unique_records.csv", index=False, sep=';')
# duplicate_df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\duplicate_records.csv", index=False, sep=';')

# # Print the number of unique and duplicate records
# print(f"Number of unique records: {len(unique_df)}")
# print(f"Number of duplicate records: {len(duplicate_df)}")


3450825

In [5]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Case\extractPROD - Copy.csv"
df = pd.read_csv(file_path, delimiter=';')

# Identify duplicates based on 'SCC_LEGACY_TICKET_ID__C'
duplicate_rows = df[df.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep=False)]

# Count unique and duplicate records
total_records = len(df)
unique_records = total_records - len(duplicate_rows) + len(duplicate_rows['SCC_LEGACY_TICKET_ID__C'].drop_duplicates())
duplicate_count = len(duplicate_rows) - len(duplicate_rows['SCC_LEGACY_TICKET_ID__C'].drop_duplicates())


duplicate_info = {
    "Total Records": total_records,
    "Unique Records": unique_records,
    "Duplicate Records": duplicate_count
}

duplicate_rows.to_csv('duplicate_records.csv', index=False)

duplicate_info


{'Total Records': 8300416,
 'Unique Records': 3477315,
 'Duplicate Records': 4823101}

In [7]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Case\extractPROD - Copy.csv"
df = pd.read_csv(file_path, delimiter=';')

# Identify duplicates based on 'SCC_LEGACY_TICKET_ID__C'
duplicate_mask = df.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep=False)
duplicate_rows = df[duplicate_mask]

# Drop the first occurrence of each duplicated 'SCC_LEGACY_TICKET_ID__C' to keep one unique record
# This will leave only the duplicates that need to be deleted
duplicates_to_delete = duplicate_rows[duplicate_rows.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep='first')]

# Save the records to delete (including IDs) to CSV
duplicates_to_delete.to_csv('duplicate_records_to_delete.csv', index=False)

# Count unique and duplicate records for reference
total_records = len(df)
unique_records = len(df.drop_duplicates(subset=['SCC_LEGACY_TICKET_ID__C']))
duplicate_count = total_records - unique_records

# Prepare duplicate information
duplicate_info = {
    "Total Records": total_records,
    "Unique Records": unique_records,
    "Duplicate Records": duplicate_count
}

duplicate_info


{'Total Records': 8300416,
 'Unique Records': 3477315,
 'Duplicate Records': 4823101}

In [9]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Case\extractPROD.csv"
df = pd.read_csv(file_path, delimiter=';')

# Identify duplicates based on 'SCC_LEGACY_TICKET_ID__C'
duplicate_mask = df.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep=False)
duplicate_rows = df[duplicate_mask]

# Drop the first occurrence of each duplicated 'SCC_LEGACY_TICKET_ID__C' to keep one unique record
# This will leave only the duplicates that need to be deleted
duplicates_to_delete = duplicate_rows[duplicate_rows.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep='first')]

# Save the records to delete (including IDs) to CSV
duplicates_to_delete.to_csv('duplicate_records_to_delete2.csv', index=False)

# Count unique and duplicate records for reference
total_records = len(df)
unique_records = len(df.drop_duplicates(subset=['SCC_LEGACY_TICKET_ID__C']))
duplicate_count = total_records - unique_records

# Prepare duplicate information
duplicate_info = {
    "Total Records": total_records,
    "Unique Records": unique_records,
    "Duplicate Records": duplicate_count
}

duplicate_info


{'Total Records': 8300416,
 'Unique Records': 3477315,
 'Duplicate Records': 4823101}

In [3]:
import pandas as pd



file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_prod_Un.csv"

df = pd.read_csv(file_path, delimiter=';')
df.duplicated(['SCC_LEGACY_TICKET_ID__C'], keep=False)

0          False
1          False
2          False
3          False
4          False
           ...  
3477312    False
3477313    False
3477314    False
3477315     True
3477316     True
Length: 3477317, dtype: bool

In [1]:
import pandas as pd
import glob
import os

# Specify the path to your folder containing the CSV files
folder_path = r"D:\Case\case_2024"

# Ensure the folder path is correctly formatted
if not folder_path.endswith('/'):
    folder_path += '/'

# Use glob to find all CSV files in the specified folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# List to hold dataframes
df_list = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    if 'Ticket_ID' in df.columns:
        df_list.append(df[['Ticket_ID']])  # Keep only the Ticket_ID column

# Concatenate all dataframes in the list into a single dataframe
combined_df = pd.concat(df_list, ignore_index=True)

# Display the combined dataframe
print(combined_df)

# Optionally, save the combined dataframe to a new CSV file
output_path = os.path.join(folder_path, 'combined_ticket_ids.csv')
combined_df.to_csv(output_path, index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_46792\4080367576.py:20: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\maste\AppData\Local\Temp\ipykernel_46792\4080367576.py:20: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\maste\AppData\Local\Temp\ipykernel_46792\4080367576.py:20: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\maste\AppData\Local\Temp\ipykernel_46792\4080367576.py:20: DtypeWarning: Columns (8,11,17,23,30,33,36,47,49,53,56,61,63,66,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\maste\AppData\Local\Temp\ipykernel_46792\4080367576.py:20: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=Fals

               Ticket_ID
0        TTB000050004867
1        TTB000049908332
2        TTB000049929970
3        TTB000049777973
4        TTB000049965755
...                  ...
6651718  TTB000050741769
6651719  TTB000050971441
6651720  TTB000050776910
6651721  TTB000050974895
6651722  TTB000050874373

[6651723 rows x 1 columns]


In [7]:
import pandas as pd
path=r"D:\Case\case_2024\combined_ticket_ids.csv"

df=pd.read_csv(path)
# df['Ticket_ID'].unique()

# duplicates = df[df['Ticket_ID'].duplicated(keep=False)]
df['Ticket_ID'].unique()
duplicates

,Ticket_ID
0,TTB000050004867
1,TTB000049908332
2,TTB000049929970
3,TTB000049777973
4,TTB000049965755
...,...
6451718,TTB000050684721
6451719,TTB000050539111
6451720,TTB000050585890
6451721,TTB000050615438


In [6]:
print(6134520-6651723)

-517203


In [16]:
import pandas as pd
import numpy as np
path=r"D:\Case\account and case 21, 22 July\bricare_case_20_21_22_07_2024\Casedata2122.csv"

df=pd.read_csv(path)
df['Call_Type_ID'] = pd.to_numeric(df['Call_Type_ID'], errors='coerce').fillna(0).replace(np.inf, 0).astype(int).astype(str)
df.to_csv(r"D:\Case\account and case 21, 22 July\bricare_case_20_21_22_07_2024\Casedata2122_cleaned.csv",index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_47088\949659275.py:5: DtypeWarning: Columns (22,28,29,30,32,34,35,36,37,38,40,41,42,46,47,49,51,52,53,54,55,56,57,58,59,61,63,64,66,67,68,71,72,73,74,75,76,77,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


# To update column details by ID

In [9]:
import pandas as pd

# Load the uploaded files
file_1_path = r"D:\Case\extractPROD - Copy.csv"
file_2_path = r"D:\Case\account and case 21, 22 July\bricare_case_20_21_22_07_2024\Casedata2122_cleaned.csv"

# Read the first CSV file with semicolon separator
file_1_df = pd.read_csv(file_1_path, delimiter=';')

# Read the second CSV file with default comma separator
file_2_df = pd.read_csv(file_2_path)

# Merge the two DataFrames on the 'Ticket_ID' column
merged_df = pd.merge(file_2_df, file_1_df[['ID', 'Ticket_ID']], on='Ticket_ID', how='left')

# Display the first few rows of the merged DataFrame
print(merged_df.head())

# If you want to save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_output.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_49068\790842565.py:11: DtypeWarning: Columns (22,28,29,30,32,34,35,36,37,38,40,41,42,46,47,49,51,52,53,54,55,56,57,58,59,61,63,64,66,67,68,71,72,73,74,75,76,77,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  file_2_df = pd.read_csv(file_2_path)


  cifno        Ticket_ID  Call_Type_ID  \
0   NaN  TTB000054076437          8442   
1   NaN  TTB000054076437          8442   
2   NaN  TTB000054076437          8442   
3   NaN  TTB000054076437          8442   
4   NaN  TTB000054076437          8442   

                                           Call_Type          Create_Date  \
0  Nasabah ATM Prima gagal transfer & terdebet ke...  2024-07-20 00:00:17   
1  Nasabah ATM Prima gagal transfer & terdebet ke...  2024-07-20 00:00:17   
2  Nasabah ATM Prima gagal transfer & terdebet ke...  2024-07-20 00:00:17   
3  Nasabah ATM Prima gagal transfer & terdebet ke...  2024-07-20 00:00:17   
4  Nasabah ATM Prima gagal transfer & terdebet ke...  2024-07-20 00:00:17   

  gateway            Jenis_Laporan Nama_Nasabah No_Rekening  Nominal  ...  \
0   Phone  Complaint - Transaction          EKI         NaN      0.0  ...   
1   Phone  Complaint - Transaction          EKI         NaN      0.0  ...   
2   Phone  Complaint - Transaction          EKI      

## Check whether all tickets exist

In [17]:
import pandas as pd

# Reload the uploaded files
file_1_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_ales7.csv"
file_2_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\dummy_data_casenumber_fix_part1.csv"

# Read the first CSV file with semicolon separator
file_1_df = pd.read_csv(file_1_path, delimiter=';')

# Read the second CSV file with default comma separator
file_2_df = pd.read_csv(file_2_path)

# Check if all values in Ticket_ID of file 2 exist in file 1
ticket_id_missing = file_2_df[~file_2_df['Ticket_ID'].isin(file_1_df['Ticket_ID'])]

# Save the missing Ticket_IDs to a new CSV file
missing_file_path = 'missing_ticket_ids.csv'
ticket_id_missing.to_csv(missing_file_path, index=False)


# Display the path to the saved file
print(ticket_id_missing)


    Status Priority  Call Type     record_type Ticket_ID
1  Working      Low       1000  Case Migration   TTB1234


# Combine all files into one

In [1]:
import os
import pandas as pd

# Specify the folder containing the CSV files
folder_path = r"D:\Case\case_2021"

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold dataframes
dataframes = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        # Try reading the CSV file with utf-8 encoding
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # If utf-8 fails, try reading the CSV file with iso-8859-1 encoding
            df = pd.read_csv(file_path, encoding='iso-8859-1')
        except UnicodeDecodeError:
            # If iso-8859-1 fails, try reading the CSV file with latin1 encoding
            df = pd.read_csv(file_path, encoding='latin1')
    dataframes.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Specify the output file path
output_file = r'D:\Case\case_2021\combined_csv_file.csv'

# Save the combined dataframe to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f'All CSV files have been combined into {output_file}')


C:\Users\maste\AppData\Local\Temp\ipykernel_41916\4098341139.py:18: DtypeWarning: Columns (15,36,37,47,53,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\4098341139.py:18: DtypeWarning: Columns (15,31,36,37,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\4098341139.py:18: DtypeWarning: Columns (36,47,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\4098341139.py:18: DtypeWarning: Columns (8,15,17,31,36,46,47,51,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\4098341139.py:18: DtypeWar

All CSV files have been combined into D:\Case\case_2021\combined_csv_file.csv


In [4]:
import pandas as pd

file_path = r"D:\Case\case_2021\combined_csv_file.csv"
chunk_size = 100000  # Adjust the chunk size as needed

smallest_value = float('inf')
biggest_value = float('-inf')
smallest_ticket_id = None
biggest_ticket_id = None

for chunk in pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunk_size):
    chunk['Ticket_ID'] = chunk['Ticket_ID'].astype(str)
    chunk['Numeric_ID'] = chunk['Ticket_ID'].str.extract(r'TTB(\d+)').astype(int)
    
    chunk_min_value = chunk['Numeric_ID'].min()
    chunk_max_value = chunk['Numeric_ID'].max()
    
    if chunk_min_value < smallest_value:
        smallest_value = chunk_min_value
        smallest_ticket_id = chunk[chunk['Numeric_ID'] == chunk_min_value]['Ticket_ID'].iloc[0]
    
    if chunk_max_value > biggest_value:
        biggest_value = chunk_max_value
        biggest_ticket_id = chunk[chunk['Numeric_ID'] == chunk_max_value]['Ticket_ID'].iloc[0]

print(f"The smallest Ticket_ID is {smallest_ticket_id} with value {smallest_value}")
print(f"The biggest Ticket_ID is {biggest_ticket_id} with value {biggest_value}")


C:\Users\maste\AppData\Local\Temp\ipykernel_41916\3751085341.py:11: DtypeWarning: Columns (15,36,37,47,53,63) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunk_size):
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\3751085341.py:11: DtypeWarning: Columns (36,37,47,53,63) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunk_size):
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\3751085341.py:11: DtypeWarning: Columns (36,37,53,63) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunk_size):
C:\Users\maste\AppData\Local\Temp\ipykernel_41916\3751085341.py:11: DtypeWarning: Columns (15,31,36,37,47,63) ha

The smallest Ticket_ID is TTB000007861773 with value 7861773
The biggest Ticket_ID is TTB000038245877 with value 38245877


C:\Users\maste\AppData\Local\Temp\ipykernel_41916\3751085341.py:11: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunk_size):


In [1]:
import pandas as pd

# Define the file path
file_path = r"D:\Case\case_2021\combined_csv_file.csv"

# Try to load the data with different parameters to handle possible issues
try:
    df = pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', parse_dates=['Create_Date'])
except pd.errors.ParserError:
    df = pd.read_csv(file_path, delimiter=',', encoding='latin1', on_bad_lines='skip', parse_dates=['Create_Date'])

# Ensure the 'Create_Date' column is parsed as datetime
df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce')

# Drop rows with NaT (Not a Time) values in 'Create_Date' if any
df = df.dropna(subset=['Create_Date'])

# Find the smallest and biggest dates
smallest_date = df['Create_Date'].min()
biggest_date = df['Create_Date'].max()

print(f"The smallest Create_Date is {smallest_date}")
print(f"The biggest Create_Date is {biggest_date}")


C:\Users\maste\AppData\Local\Temp\ipykernel_42968\2598159650.py:8: DtypeWarning: Columns (8,9,15,17,22,25,31,33,36,37,40,45,46,47,50,51,53,62,63,75,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=',', encoding='utf-8', on_bad_lines='skip', parse_dates=['Create_Date'])


MemoryError: Unable to allocate 2.52 GiB for an array with shape (70, 4832391) and data type object

# Check if all values in small_ids are in large_ids

In [5]:
import pandas as pd

# File paths
file1_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\combined_csv_file - Copy.csv"
file2_path = r"C:\\Users\\maste\\Downloads\\dataloader_v60.0.2\\extract_133k_case_2024.csv"

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path, delimiter=';')

# Check if all values in SCC_LEGACY_TICKET_ID__C from file2 are in file1
all_present = df2['SCC_LEGACY_TICKET_ID__C'].isin(df1['SCC_LEGACY_TICKET_ID__C'])

# Print results
if all(all_present):
    print("All values in SCC_LEGACY_TICKET_ID__C from file2 are present in file1.")
else:
    missing_values = df2[~all_present]
    print("The following values in SCC_LEGACY_TICKET_ID__C from file2 are not present in file1:")
    print(missing_values['SCC_LEGACY_TICKET_ID__C'])


C:\Users\maste\AppData\Local\Temp\ipykernel_15496\2423444586.py:8: DtypeWarning: Columns (9,10,12,16,18,19,23,24,31,32,34,37,46,47,48,49,50,51,52,54,60,62,63,64,67,76,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1_path)


MemoryError: Unable to allocate 937. MiB for an array with shape (73, 1682857) and data type object

In [14]:
import pandas as pd

# File paths
file1_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\combined_csv_file - Copy.csv"
file2_path = r"D:\dataquality2\Prod_133k_legacyid.csv"
output_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\missing_values.csv"

# Read file2 in one go as it's the smaller file
df2 = pd.read_csv(file2_path, delimiter=';')

# Create a set of SCC_LEGACY_TICKET_ID__C values from file2
file2_ids = set(df2['SCC_LEGACY_TICKET_ID__C'])

# Initialize an empty set to collect matched IDs
matched_ids = set()

# Read file1 in chunks with correct delimiter
chunksize = 10**5
for chunk in pd.read_csv(file1_path, chunksize=chunksize):
    matched_ids.update(file2_ids.intersection(chunk['SCC_LEGACY_TICKET_ID__C']))

# Determine missing values
missing_values = file2_ids - matched_ids

# Save missing values to CSV
if not missing_values:
    print("All values in SCC_LEGACY_TICKET_ID__C from file2 are present in file1.")
else:
    missing_df = pd.DataFrame(list(missing_values), columns=['SCC_LEGACY_TICKET_ID__C'])
    missing_df.to_csv(output_path, index=False)
    print(f"The missing values have been saved to {output_path}")


C:\Users\maste\AppData\Local\Temp\ipykernel_28596\1491224758.py:19: DtypeWarning: Columns (34,48,54,60) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file1_path, chunksize=chunksize):
C:\Users\maste\AppData\Local\Temp\ipykernel_28596\1491224758.py:19: DtypeWarning: Columns (16,47,52,54,60) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file1_path, chunksize=chunksize):
C:\Users\maste\AppData\Local\Temp\ipykernel_28596\1491224758.py:19: DtypeWarning: Columns (9,12,18,19,23,24,31,34,37,47,48,50,52,54,62,64,67,76,81) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file1_path, chunksize=chunksize):
C:\Users\maste\AppData\Local\Temp\ipykernel_28596\1491224758.py:19: DtypeWarning: Columns (32,47,48,52,54,60,64) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file1_path, chunksize=

The missing values have been saved to D:\Python2\case_2024\M--\Case_2024_Clean_1\missing_values.csv


In [ ]:
import pandas as pd

# File paths
file1_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\combined_csv_file - Copy.csv"
file2_path = r"D:\dataquality2\Prod_133k_legacyid.csv"
output_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\missing_values.csv"

# Read file2 in one go as it's the smaller file
df2 = pd.read_csv(file2_path, delimiter=';')

# Create a set of SCC_LEGACY_TICKET_ID__C values from file2
file2_ids = set(df2['SCC_LEGACY_TICKET_ID__C'])

# Initialize an empty set to collect matched IDs
matched_ids = set()

# Read file1 in chunks with correct delimiter
chunksize = 10**5
for chunk in pd.read_csv(file1_path, chunksize=chunksize):
    matched_ids.update(file2_ids.intersection(chunk['SCC_LEGACY_TICKET_ID__C']))

# Determine missing values
missing_values = file2_ids - matched_ids

# Save missing values to CSV
if not missing_values:
    print("All values in SCC_LEGACY_TICKET_ID__C from file2 are present in file1.")
else:
    missing_df = pd.DataFrame(list(missing_values), columns=['SCC_LEGACY_TICKET_ID__C'])
    missing_df.to_csv(output_path, index=False)
    print(f"The missing values have been saved to {output_path}")


In [40]:
import pandas as pd

path = r"D:\\Python2\\case_2023\\extract_2023_19_line_toclose.csv"
df2 = pd.read_csv(path, delimiter=';')
# try:
#     df2 = pd.read_csv(path, delimiter=';')
#     print(df2)
# except pd.errors.ParserError as e:
#     print("ParserError:", e)

# # Alternatively, if you want to inspect the file manually
# with open(path, 'r') as file:
#     lines = file.readlines()
#     for i, line in enumerate(lines):
#         print(f"Line {i+1}: {line}")
df2['STATUS']='Closed'
df2.to_csv('2023_19Lines.csv', index=False)


In [9]:
import pandas as pd

file1_path = r"D:\Python2\case_2024\M--\Case_2024_Clean_1\combined_csv_file - Copy.csv"
df1 = pd.read_csv(file1_path)

df1.columns

C:\Users\maste\AppData\Local\Temp\ipykernel_28596\3873219863.py:4: DtypeWarning: Columns (9,10,12,16,18,19,23,24,31,32,34,37,46,47,48,49,50,51,52,54,60,62,63,64,67,76,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1_path)


Index(['Unnamed: 0', 'cifno', 'SCC_LEGACY_TICKET_ID__C', 'Call_Type_ID',
       'Call_Type', 'Create_Date', 'gateway', 'Jenis_Laporan', 'Nama_Nasabah',
       'No_Rekening', 'Nominal', 'status', 'TanggalClosed', 'tanggalTransaksi',
       'Chanel', 'Fitur', 'Nomor_Kartu', 'user_group', 'assgined_to',
       'attachment_done', 'email', 'full_name', 'no_telepon', 'approver_login',
       'approver_name', 'SLAResolution', 'submitter_login_id',
       'submitter_user_group', 'user_login_name', 'Details', 'Jenis_Produk',
       'Last_Modified_By', 'Merchant_ID', 'Modified_Date', 'NOTAS', 'Produk',
       'SLA_Status', 'TID', 'tanggalAttachmentDone', 'TglAssigned',
       'Tgl_Eskalasi', 'AnalisaSkils', 'Attachment_', 'Bank_BRI',
       'Biaya_Admin', 'Suku_Bunga', 'Bunga', 'Butuh_Attachment', 'Cicilan',
       'Hasil_Kunjungan', 'Log_Name', 'MMS_Ticket_Id',
       'Mass_Ticket_Upload_Flag', 'Nama_Supervisor', 'Nama_TL', 'Nama_Wakabag',
       'Nasabah_Prioritas', 'Notify_By', 'Organization'

# To filter the combined file based on Ticket ID

In [36]:
import pandas as pd
import os

# Read the primary file
primary_file = pd.read_csv(r"D:\Python2\case_2023\2023_19Lines.csv")

# Directory containing the other files
directory = r"D:\Python2\case_2023\Case_2023_Clean_2"

# Initialize an empty list to hold the filtered DataFrames
filtered_dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Read each file
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        
        # Filter based on Ticket_ID values
        filtered_df = df[df['Ticket_ID'].isin(primary_file['Ticket_ID'])]
        
        # Append the filtered DataFrame to the list
        filtered_dfs.append(filtered_df)

# Concatenate all filtered DataFrames

all_filtered_data = pd.concat(filtered_dfs, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
all_filtered_data.to_csv(r'D:\Python2\case_2023\filtered_data.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_39000\1121489279.py:18: DtypeWarning: Columns (15,31,47,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_39000\1121489279.py:18: DtypeWarning: Columns (15,22,31,63,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_39000\1121489279.py:18: DtypeWarning: Columns (15,31,47,63,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_39000\1121489279.py:18: DtypeWarning: Columns (15,31,46,47,51,63,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_39000\1121489279.py:18: DtypeWarning: Columns (15,46,47,51,63,71) have mixed types. Specify dtype option on i

In [11]:
import pandas as pd

# Load the CSV file
path = r"D:\Python2\case_2024\133k_2024\error073124121623093.csv"
df = pd.read_csv(path)

# Ensure the column exists
if 'PHONE_SURVEY' in df.columns:
    # Truncate the values in the 'PHONE_SURVEY' column to a maximum of 40 characters
    df['PHONE_SURVEY'] = df['PHONE_SURVEY'].str.slice(0, 40)


# Save the modified DataFrame to a new CSV file
output_path = r"D:\Python2\case_2024\133k_2024\error073124121623093_truncated.csv"
df.to_csv(output_path, index=False)

output_path


'D:\\Python2\\case_2024\\133k_2024\\error073124121623093_truncated.csv'

# Account

In [8]:
# Fill the empty values in column names with No name

import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(r"D:\Python2\case_2024\5-6 aug\bricare_20240805_20240806_0_case_account.csv", delimiter=';')
df['Nama'] = df['Nama'].replace(['', '-'], 'No Name')
df['Nama'] = df['Nama'].apply(lambda x: x[:250] if len(x) > 250 else x)
# Check for duplicate 'cifno'
# duplicate_cifno = df.duplicated('cifno')
# if duplicate_cifno.any():
#     print("There are duplicate 'cifno' values.")
# else:
#     print("All 'cifno' values are unique.")

# Fill empty 'Nama' fields with "No Name"
# df['Nama'].fillna('No Name', inplace=True)
# df['Nama'].replace('', 'No Name', inplace=True)

# Save the updated DataFrame to a new CSV file
# df.to_csv('updated_yourfile.csv', index=False, sep=';')
record_type_values = df['record_type'].unique()

# Ensure there are only two unique values in the record_type column
if len(record_type_values) != 2:
    raise ValueError("The 'record_type' column should have exactly two unique values.")

# Create two separate DataFrames based on the unique values
df1 = df[df['record_type'] == record_type_values[0]]
df2 = df[df['record_type'] == record_type_values[1]]

# Save the DataFrames to separate CSV files
df1.to_csv(f'file_{record_type_values[0]}.csv', index=False)
df2.to_csv(f'file_{record_type_values[1]}.csv', index=False)

print(f"File has been split into 'file_{record_type_values[0]}.csv' and 'file_{record_type_values[1]}.csv'.")



File has been split into 'file_Person Account.csv' and 'file_Business.csv'.


# Change the USER GROUP

In [5]:
import os
import pandas as pd

# Path to the master user group data file
master_user_group_path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx - branch_unit.csv"

# Load the master user group data
master_user_group_df = pd.read_csv(master_user_group_path)

# Create a mapping from Branch Code without leading zeros to Name in the master data
master_user_group_df['Branch Code'] = master_user_group_df['Branch Code'].astype(str).str.lstrip('0')
user_group_mapping = dict(zip(master_user_group_df['Branch Code'], master_user_group_df['Name']))

# Folder containing the CSV files to be updated
input_folder = r"D:\Python2\case_2024\5-6 aug\caseowner\cabang"
output_folder = r"D:\Python2\case_2024\5-6 aug\caseowner\cabang"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to update the user_group column in a given dataframe
def update_user_group(df):
    def map_user_group(value):
        numeric_part = ''.join(filter(str.isdigit, str(value))).lstrip('0')
        return user_group_mapping.get(numeric_part, value) if numeric_part else value
    
    df['user_group'] = df['user_group'].apply(map_user_group)
    return df

# Process each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, f'Updated_{filename}')
        
        # Load the CSV file
        df = pd.read_csv(input_file_path, delimiter=';', on_bad_lines='skip')
        
        # Update the user_group column
        updated_df = update_user_group(df)
        
        # Save the updated dataframe to a new CSV file
        updated_df.to_csv(output_file_path, index=False)

print('All files have been processed and updated.')


All files have been processed and updated.


# Change user group 2

In [16]:
import os
import pandas as pd

# Path to the master user group data file
master_user_group_path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx - branch_unit.csv"

# Load the master user group data
master_user_group_df = pd.read_csv(master_user_group_path)

# Create a mapping from Branch Code without leading zeros to Name in the master data
master_user_group_df['Branch Code'] = master_user_group_df['Branch Code'].astype(str).str.lstrip('0')
user_group_mapping = dict(zip(master_user_group_df['Branch Code'], master_user_group_df['Name']))

# Folder containing the CSV files to be updated
input_folder = r"D:\Python2\case_2024\change_owner\Prod\batch6_working\New folder"
output_folder = r"D:\Python2\case_2024\change_owner\Prod\batch6_working\New folder"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to update the user_group column in a given dataframe
def update_user_group(df):
    # Filter rows where user_group values start with numbers
    df = df[df['user_group'].astype(str).str.match(r'^\d')]
    
    # Mapping logic
    def map_user_group(value):
        numeric_part = ''.join(filter(str.isdigit, str(value))).lstrip('0')
        return user_group_mapping.get(numeric_part, value) if numeric_part else value
    
    df['user_group'] = df['user_group'].apply(map_user_group)
    return df

# Function to process a single CSV file
def process_csv(input_file_path, output_file_path):
    try:
        # Read the entire CSV file with error handling
        df = pd.read_csv(input_file_path, delimiter=';', on_bad_lines='skip', encoding='utf-8')
        
        # Update the user_group column
        updated_df = update_user_group(df)
        
        # Save the updated dataframe to a new CSV file
        updated_df.to_csv(output_file_path, index=False)
        print(f'{input_file_path} has been processed and updated.')
    except pd.errors.ParserError as e:
        print(f'Error processing {input_file_path}: {e}')

# Process each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, f'Updated_{filename}')
        process_csv(input_file_path, output_file_path)

print('All files have been processed and updated.')


D:\Python2\case_2024\change_owner\Prod\batch6_working\New folder\extract - Cabang.csv has been processed and updated.
All files have been processed and updated.


C:\Users\maste\AppData\Local\Temp\ipykernel_27896\1192719588.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_group'] = df['user_group'].apply(map_user_group)


In [5]:
import pandas as pd

path=r"D:\Salesforce\test\Updated_extract_usergroup2021_done - Copy.csv"

df=pd.read_csv(path)
df

,ID,user_group,UNIT_KERJA_2__C,SCC_LEGACY_TICKET_ID__C
0,500Mg00000G889vIAB,5780 - UNIT TALANG PADANG PRINGSEWU,NaN,TTB000032693377


# Filter the updated user group

In [14]:
import pandas as pd

# Assuming you have already read your data into a DataFrame named 'df'
path=r"D:\Salesforce\usergroup2021\Updated_extract_user_group_2021.csv"
df = pd.read_csv(path)

# List of ticket IDs to filter
ticket_ids = [
    'TTB000032745470',
    'TTB000032845281',
    'TTB000032822488',
    'TTB000032866989',
    'TTB000032896091'
]

# Filtering the DataFrame
filtered_df = df[df['SCC_LEGACY_TICKET_ID__C'].isin(ticket_ids)]

# Display the filtered DataFrame
print(filtered_df)

# If you want to save the filtered DataFrame to a new CSV file
# filtered_df.to_csv('filtered_data.csv', index=False)
filtered_df


Empty DataFrame
Columns: [ID, user_group, UNIT_KERJA_2__C, SCC_LEGACY_TICKET_ID__C]
Index: []


,ID,user_group,UNIT_KERJA_2__C,SCC_LEGACY_TICKET_ID__C


# Continue the Tickets

In [7]:
import pandas as pd

# Function to generate the next ticket IDs
def generate_tickets(start_ticket, count):
    tickets = []
    prefix = "TICKET"
    current_number = int(start_ticket.replace(prefix, ""))
    
    number_of_digits = len(start_ticket) - len(prefix)
    
    for _ in range(count):
        current_ticket = f"{prefix}{current_number:0{number_of_digits}}"
        tickets.append(current_ticket)
        current_number += 1
    
    return tickets

# Starting ticket ID
start_ticket_id = "TICKET0012345678910"

# Generate 1000 ticket IDs
next_1000_tickets = generate_tickets(start_ticket_id, 1000)

# Create a DataFrame
df_tickets = pd.DataFrame(next_1000_tickets, columns=['Ticket ID'])

# Save to CSV
csv_file_path = "next_1000_tickets.csv"
df_tickets.to_csv(csv_file_path, index=False)

csv_file_path


'next_1000_tickets.csv'

# update escalated

In [1]:
import pandas as pd

# Load the text file
with open(r"D:\Python2\case_2024\escalated\extract_to_escalated2.txt", 'r') as file:
    lines = file.readlines()

# Process lines and convert to DataFrame
# Assuming each line in the text file is comma-separated
data = []
for line in lines:
    values = line.strip().split(';')
    data.append(values)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.to_csv(r"D:\Python2\case_2024\escalated\extract_to_escalated2.csv", index=False, header=False)


In [2]:
import pandas as pd
import csv

# Load the text file into a DataFrame
df = pd.read_csv(r"D:\Python2\case_2024\escalated\extract_to_escalated2.txt", delimiter=';')

# Save as CSV with minimal quoting
df.to_csv(r"D:\Python2\case_2024\escalated\extract_to_escalated2.csv", index=False, quoting=csv.QUOTE_MINIMAL)


# Update status from Esacalated to Closed

In [2]:
import pandas as pd

# Load the first file into a DataFrame
file1 = pd.read_csv(r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean\bricare_20240510_20240806_0_all.csv")

# Load the second file into a DataFrame
file2 = pd.read_csv(r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean\extract_escalated_to_closed.csv", delimiter=';')

# Merge the DataFrames on the 'Ticket_ID' column
merged_df = pd.merge(file1, file2[['Ticket_ID', 'ID']], on='Ticket_ID', how='left')

# Save the resulting DataFrame to a new file
merged_df.to_csv(r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean\merged.csv", index=False)

# print("Merge completed and saved to merged_file.csv")

merged_df['ID']


# Display the first few rows and the columns of both DataFrames to ensure they are loaded correctly
# file1_info = file1.head(), file1.columns
# file2_info = file2.head(), file2.columns

merged_df

C:\Users\maste\AppData\Local\Temp\ipykernel_28744\4068740087.py:4: DtypeWarning: Columns (15,47,63) have mixed types. Specify dtype option on import or set low_memory=False.
  file1 = pd.read_csv(r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean\bricare_20240510_20240806_0_all.csv")


,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,Nominal,...,Tgl_Returned,Ticket_Referensi,Tiket_Urgency,Tipe_Remark,UniqueID,users,Usergroup_ID,record_type,Legacy_Ticket_ID,ID
0,5188280205773204,TTB000052595425,1101,Sanggahan Pengajuan Kartu Kredit,2024-05-10 01:06:06,Phone,Fraud / Suspicious Transactions,EDI HERMAWAN,NaN,0.0,...,NaN,NaN,NaN,Notes,NaN,Call,5.0,Case Migration,TTB0052595425,500Mg00000EoJEOIA3
1,SCBF095,TTB000052696714,8602,BRIMO,2024-05-15 10:40:22,Phone,Platform & Device Support,SUMARIYANTO,601301000250501,0.0,...,2024-07-16 00:00:00,NaN,NaN,Notes,NaN,Call,4.0,Case Migration,TTB0052696714,500Mg00000EoJErIAN
2,AXG2301,TTB000052710500,8708,Penipuan Kepada Nasabah BRI dan Transaksi dila...,2024-05-15 19:16:48,Phone,Fraud / Suspicious Transactions,ANTON KALEME,520601032951537,11645678.0,...,NaN,NaN,NaN,Notes,NaN,Call,5.0,Case Migration,TTB0052710500,500Mg00000EoJEwIAN
3,LA70603,TTB000052712166,8708,Penipuan Kepada Nasabah BRI dan Transaksi dila...,2024-05-15 20:50:00,Phone,NaN,LINDA ISLAMI,574201025946539,26000000.0,...,NaN,NaN,NaN,Notes,NaN,Call,5.0,Case Migration,TTB0052712166,500Mg00000F3sq7IAB
4,AMND036,TTB000052728166,8708,Penipuan Kepada Nasabah BRI dan Transaksi dila...,2024-05-16 14:02:35,SMS,Fraud / Suspicious Transactions,ABDURRAHMAN,470401030333534,50750000.0,...,NaN,NaN,NaN,Notes,NaN,Non Call,5.0,Case Migration,TTB0052728166,500Mg00000EoJF3IAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20527,SFLGQ91,TTB000054433087,8416,Nasabah BRI Gagal Transaksi Belanja di EDC Ban...,2024-08-06 23:55:02,BRIMO,Complaint - Transaction,SYAIFUL RACHMAN,106001016629500,70000.0,...,NaN,NaN,NaN,Notes,NBMB718820065417,Non Call,1.0,Case Migration,TTB0054433087,500Mg00000GgEYpIAN
20528,AZXI564,TTB000054433088,8434,"Pengisian Pulsa HP gagal, Saldo Berkurang",2024-08-06 23:56:35,BRIMO,NaN,ASTINA Y BATALIPU,523101015490533,20000.0,...,NaN,NaN,NaN,Notes,NBMB718820306932,Non Call,1.0,Case Migration,TTB0054433088,500Mg00000GgEYqIAN
20529,MRRM771,TTB000054433081,8940,Pengajuan Aplikasi Briguna Digital,2024-08-06 23:59:01,Phone,Platform & Device Support,MULYANA,091901006677501,0.0,...,NaN,NaN,NaN,Notes,NaN,Call,12801.0,Case Migration,TTB0054433081,500Mg00000GgEYrIAN
20530,AZXI564,TTB000054433091,8434,"Pengisian Pulsa HP gagal, Saldo Berkurang",2024-08-06 23:59:17,BRIMO,NaN,ASTINA Y BATALIPU,523101015490533,1500.0,...,NaN,NaN,NaN,Notes,NBMB718820714364,Non Call,1.0,Case Migration,TTB0054433091,500Mg00000GgEYsIAN


In [10]:
import pandas as pd

import pandas as pd

# Load the CSV file
file_path = r"D:\Python2\case_2024\5-6 aug\data bricare close new clean\error080724045250191.csv"
df = pd.read_csv(file_path)

# Display the column names to find the correct column for email
print(df.columns)

# Assuming the email column is named 'email', we will filter out the unwanted email
if 'email' in df.columns:
    df_cleaned = df[df['email'] != 'user@bri.co.id']
else:
    df_cleaned = df  # No changes needed if 'email' column does not exist
    print("No 'email' column found in the dataset.")

# Save the cleaned dataframe to a new CSV file
output_file_path = r"D:\Python2\case_2024\5-6 aug\data bricare close new clean\error080724045250191_account.csv"
df_cleaned['RECORD_TYPE']='Person Account'
df_cleaned.to_csv(output_file_path, index=False)

# Confirm the process
print(f"Cleaned data saved to: {output_file_path}")


Index(['CIFNO', 'TICKET_ID', 'CALL_TYPE_ID', 'CALL_TYPE', 'CREATE_DATE',
       'GATEWAY', 'JENIS_LAPORAN', 'NAMA_NASABAH', 'NO_REKENING', 'NOMINAL',
       'STATUS', 'TANGGALCLOSED', 'TANGGALTRANSAKSI', 'CHANEL', 'FITUR',
       'NOMOR_KARTU', 'USER_GROUP', 'ASSGINED_TO', 'ATTACHMENT_DONE', 'EMAIL',
       'FULL_NAME', 'NO_TELEPON', 'APPROVER_LOGIN', 'APPROVER_NAME',
       'SLARESOLUTION', 'SUBMITTER_LOGIN_ID', 'SUBMITTER_USER_GROUP',
       'USER_LOGIN_NAME', 'DETAILS', 'JENIS_PRODUK', 'LAST_MODIFIED_BY',
       'MERCHANT_ID', 'MODIFIED_DATE', 'NOTAS', 'PRODUK', 'SLA_STATUS', 'TID',
       'TANGGALATTACHMENTDONE', 'TGLASSIGNED', 'TGL_ESKALASI', 'ANALISASKILS',
       'ATTACHMENT_', 'BANK_BRI', 'BIAYA_ADMIN', 'SUKU_BUNGA', 'BUNGA',
       'BUTUH_ATTACHMENT', 'CICILAN', 'HASIL_KUNJUNGAN', 'LOG_NAME',
       'MMS_TICKET_ID', 'MASS_TICKET_UPLOAD_FLAG', 'NAMA_SUPERVISOR',
       'NAMA_TL', 'NAMA_WAKABAG', 'NASABAH_PRIORITAS', 'NOTIFY_BY',
       'ORGANIZATION', 'OUTPUT_SETTLEMENT', 'PHON

In [ ]:
import pandas as pd

# Load the first file into a DataFrame
file1 = pd.read_csv(r"D:\Python2\case_2024\5-6 aug\data bricare close new clean\bricare_20240110_20240804_0_all.csv")

# Load the second file into a DataFrame
file2 = pd.read_csv(r"D:\Python2\case_2024\5-6 aug\data bricare close new clean\extract_to_escalated.csv")


# Load the second file into a DataFrame with the correct delimiter
file2 = pd.read_csv(file2_path, delimiter=';')

# Ensure the 'Ticket_ID' and 'ID' columns are present
file2 = file2[['Ticket_ID', 'ID']]

# Merge the DataFrames on the 'Ticket_ID' column
merged_df = pd.merge(file1, file2, on='Ticket_ID', how='left')

# Save the resulting DataFrame to a new file
output_path = '/mnt/data/merged_file_escalated.csv'
merged_df.to_csv(output_path, index=False)

output_path


# Load the second file into a DataFrame with the correct delimiter
file2 = pd.read_csv(file2_path, delimiter=';')

# Ensure the 'Ticket_ID' and 'ID' columns are present
file2 = file2[['Ticket_ID', 'ID']]

# Merge the DataFrames on the 'Ticket_ID' column
merged_df = pd.merge(file1, file2, on='Ticket_ID', how='left')

# Save the resulting DataFrame to a new file
output_path = '/mnt/data/merged_file_escalated.csv'


merged_df.to_csv(output_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Merged DataFrame", dataframe=merged_df)

output_path



# Account error

In [27]:
import pandas as pd

# Load the uploaded file
file_path = r"D:\Python2\case_2024\missing_ttb_from attachment\error080724030058935.csv"
data = pd.read_csv(file_path, dtype={'cifno': str})  # Ensure 'cifno' is read as a string

# Add the columns 'Jenis_Nasabah', 'tipe_nasabah', and 'record_type' with default values
data['Jenis_Nasabah'] = None  # Assuming empty means None
data['tipe_nasabah'] = 'Individu'
data['record_type'] = 'Person Account'

# Filter the columns and rows based on the specified conditions
filtered_data = data.loc[
    (data['Jenis_Nasabah'].isna()) &  # Jenis_Nasabah is empty
    (data['tipe_nasabah'] == 'Individu') &  # tipe_nasabah is Individu
    (data['record_type'] == 'Person Account'),  # record_type is Person Account
    ['cifno', 'Nama_Nasabah', 'no_telepon', 'email', 'Jenis_Nasabah', 'tipe_nasabah', 'record_type']  # Selected columns
]

# Ensure full numbers are retained in the CSV output
pd.options.display.float_format = '{:.0f}'.format

# Save the filtered data to a new CSV file
output_path = r"D:\Python2\case_2024\missing_ttb_from attachment\error080724030058935_done.csv"
filtered_data.to_csv(output_path, index=False)

# Display the filtered data to the user
print(filtered_data)


KeyError: "['cifno', 'Nama_Nasabah', 'no_telepon', 'email'] not in index"

In [28]:
import pandas as pd
import csv

# Define the path to the CSV file
file_path = r"D:\Python2\case_2024\missing_ttb_from attachment\error080724030058935.csv"

# Initialize a list to store rows
rows = []

# Open and read the CSV file, handling BOM
with open(file_path, 'r', encoding='utf-8-sig') as file:
    reader = csv.reader(file, delimiter=';')
    current_row = []
    for line in reader:
        if len(line) > 1:
            if current_row:
                rows.append(current_row)
            current_row = line
        else:
            current_row.extend(line)

    # Don't forget to add the last collected row
    if current_row:
        rows.append(current_row)

# Convert list of rows to DataFrame
data = pd.DataFrame(rows[1:], columns=rows[0])

# Strip any leading/trailing whitespace from column names
data.columns = data.columns.str.strip()

# Print out the column names to debug
print("Column names in the DataFrame:", data.columns.tolist())

# Ensure 'cifno' is read as a string if it exists
if 'cifno' in data.columns:
    data['cifno'] = data['cifno'].astype(str)
else:
    print("Column 'cifno' not found in the data")

# Add the columns 'Jenis_Nasabah', 'tipe_nasabah', and 'record_type' with default values
data['Jenis_Nasabah'] = None  # Assuming empty means None
data['tipe_nasabah'] = 'Individu'
data['record_type'] = 'Person Account'

# Filter the columns and rows based on the specified conditions
filtered_data = data.loc[
    (data['Jenis_Nasabah'].isna()) &  # Jenis_Nasabah is empty
    (data['tipe_nasabah'] == 'Individu') &  # tipe_nasabah is Individu
    (data['record_type'] == 'Person Account'),  # record_type is Person Account
    ['cifno', 'Nama_Nasabah', 'no_telepon', 'email', 'Jenis_Nasabah', 'tipe_nasabah', 'record_type']  # Selected columns
]

# Ensure full numbers are retained in the CSV output
pd.options.display.float_format = '{:.0f}'.format

# Save the filtered data to a new CSV file
output_path = r"D:\Python2\case_2024\missing_ttb_from attachment\error080724030058935_done.csv"
filtered_data.to_csv(output_path, index=False)

# Display the filtered data to the user
print(filtered_data)


Column names in the DataFrame: ['CIFNO,"TICKET_ID","CALL_TYPE_ID","CALL_TYPE","CREATE_DATE","GATEWAY","JENIS_LAPORAN","NAMA_NASABAH","NO_REKENING","NOMINAL","STATUS","TANGGALCLOSED","TANGGALTRANSAKSI","CHANEL","FITUR","NOMOR_KARTU","USER_GROUP","ASSGINED_TO","ATTACHMENT_DONE","EMAIL","FULL_NAME","NO_TELEPON","APPROVER_LOGIN","APPROVER_NAME","SLARESOLUTION","SUBMITTER_LOGIN_ID","SUBMITTER_USER_GROUP","USER_LOGIN_NAME","DETAILS","JENIS_PRODUK","LAST_MODIFIED_BY","MERCHANT_ID","MODIFIED_DATE","NOTAS","PRODUK","SLA_STATUS","TID","TANGGALATTACHMENTDONE","TGLASSIGNED","TGL_ESKALASI","ANALISASKILS","ATTACHMENT_","BANK_BRI","BIAYA_ADMIN","SUKU_BUNGA","BUNGA","BUTUH_ATTACHMENT","CICILAN","HASIL_KUNJUNGAN","LOG_NAME","MMS_TICKET_ID","MASS_TICKET_UPLOAD_FLAG","NAMA_SUPERVISOR","NAMA_TL","NAMA_WAKABAG","NASABAH_PRIORITAS","NOTIFY_BY","ORGANIZATION","OUTPUT_SETTLEMENT","PHONE_SURVEY","RETURN_TICKET","SETTLEMENT_BY","SETTLEMENT_ID","SETTLEMENT","SITE_USER","STATUS_RETURN","STATUS_TRANSAKSI","SUBMITT

KeyError: "['cifno', 'Nama_Nasabah', 'no_telepon', 'email'] not in index"

# To close Taspen

In [9]:
import pandas as pd

path=r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\error081024020347257.csv"
path2=r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\error081024020347257_done.csv"
df=pd.read_csv(path)
df['Jenis_Nasabah'] = None  
df['tipe_nasabah'] = 'Individu'
df['record_type1'] = 'Person Account'
df
df.to_csv(path2, index=False)

# Ticket Change

In [2]:
import pandas as pd

# Paths to the files
file_1_path = r"D:\Python2\case_2024\30 july- 4 aug\extract_perubahan_ticket - Copy.csv"
file_2_path = r"D:\Python2\case_2024\30 july- 4 aug\TICKET_ID_bricare_20240111_20240729_mod_less_4agustus_0_79_closed.csv"
filtered_file_1_path = r"D:\Python2\case_2024\30 july- 4 aug\filtered_perubahan_tiket.csv"

# Load file 1
df_file_1 = pd.read_csv(file_1_path, delimiter=';')

# Load the modified file 2
df_file_2_modified = pd.read_csv(file_2_path)

# Extract the Ticket_ID column from the modified file 2
ticket_ids_file_2_modified = df_file_2_modified['Ticket_ID'].dropna().unique()

# Filter the rows in file 1 based on Ticket_ID values in the modified file 2
filtered_df_file_1 = df_file_1[df_file_1['Ticket_ID'].isin(ticket_ids_file_2_modified)]

# Display the filtered dataframe
print(filtered_df_file_1)

# Save the filtered dataframe to a new file
filtered_df_file_1.to_csv(filtered_file_1_path, index=False, sep=';')


                         ID        Ticket_ID     STATUS
153839   500Mg00000F6DvoIAF  TTB000049945668     Closed
153857   500Mg00000F6Dw6IAF  TTB000049945717     Closed
153985   500Mg00000F6DxSIAV  TTB000049945832     Closed
154127   500Mg00000EoHklIAF  TTB000049946039     Closed
154131   500Mg00000EoHkpIAF  TTB000049946053     Closed
...                     ...              ...        ...
3729004  500Mg00000GZSdRIAX  TTB000054264303  Escalated
3729006  500Mg00000GZSdSIAX  TTB000054264305  Escalated
3729007  500Mg00000GZSdTIAX  TTB000054264306  Escalated
3729008  500Mg00000GZSdUIAX  TTB000054264307  Escalated
3729009  500Mg00000GZSdVIAX  TTB000054264308  Escalated

[55387 rows x 3 columns]


# Try to generate the dummy data to change case owner in UAT

In [7]:
import pandas as pd

# Load the original data
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\UAT\dummy_data_case_uat - 79_new - Copy (2) - Copy.csv"
data = pd.read_csv(file_path, delimiter=';')

# Load the new data containing Call_Type_ID values
file_path_new = r"D:\dataquality2\Cleaned4_Call_Type - Copy.csv"
call_type_data = pd.read_csv(file_path_new, delimiter=';', lineterminator='\n', encoding='ISO-8859-1')

# Load the branch unit data containing user_group values
file_path_branch_unit = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx - branch_unit.csv"
branch_unit_data = pd.read_csv(file_path_branch_unit)

# Ensure Call_Type_ID is an integer by filling NaN values with -1 first
call_type_data['Call_Type_ID'] = call_type_data['Call_Type_ID'].fillna(-1).astype(int)

# Extract user_group values from the branch unit data
user_groups = branch_unit_data['Name'].unique()

# Function to generate multiple rows
def generate_rows(data, call_type_data, user_groups, num_rows, start_ticket_id):
    # Convert start_ticket_id to an integer for manipulation
    base_id = int(start_ticket_id[7:])
    prefix = start_ticket_id[:7]
    
    # Extract Call_Type_ID values from the new dataset
    call_type_ids = call_type_data['Call_Type_ID'].unique()
    
    # Create a list to hold the new rows
    new_rows = []
    
    # Generate the specified number of rows
    for i in range(num_rows):
        new_row = data.iloc[0].copy()
        new_row['Ticket_ID'] = f"{prefix}{base_id + i:010d}"
        new_row['Call_Type_ID'] = call_type_ids[i % len(call_type_ids)]
        new_row['user_group'] = user_groups[i % len(user_groups)]
        
        # Ensure No_Rekening and Nomor_Kartu are strings
        if 'No_Rekening' in new_row and pd.notna(new_row['No_Rekening']):
            new_row['No_Rekening'] = '{:.0f}'.format(float(new_row['No_Rekening'].replace(',', '.')))
        if 'Nomor_Kartu' in new_row and pd.notna(new_row['Nomor_Kartu']):
            new_row['Nomor_Kartu'] = '{:.0f}'.format(float(new_row['Nomor_Kartu'].replace(',', '.')))
        
        new_rows.append(new_row)
    
    # Convert the list to a DataFrame
    new_data = pd.DataFrame(new_rows)
    
    return new_data

# Specify the number of rows to generate and the starting Ticket_ID
num_rows = 5
start_ticket_id = 'TICKET001234567891'

# Generate the rows
new_data = generate_rows(data, call_type_data, user_groups, num_rows, start_ticket_id)

# Save the new data to a CSV file
output_file_path = 'generated_data.csv'
new_data.to_csv(output_file_path, index=False, sep=';')

# Display the new data
new_data.head()


,case_number,record_type,cif,Ticket_ID,Call_Type_ID,Call_Type,Details,Create_Date,gateway,Jenis_Laporan,...,Tanggal_Settlement,Tgl_Foward,Tgl_In_Progress,Tgl_Returned,Ticket_Referensi,Tiket_Urgency,Tipe_Remark,UniqueID,users,Usergroup_ID
0,TICKET001339022,Case Migration,NaN,TICKET01234567891,0,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,1/1/2023 7:07,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4
0,TICKET001339022,Case Migration,NaN,TICKET01234567892,999,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,1/1/2023 7:07,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4
0,TICKET001339022,Case Migration,NaN,TICKET01234567893,1000,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,1/1/2023 7:07,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4
0,TICKET001339022,Case Migration,NaN,TICKET01234567894,1001,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,1/1/2023 7:07,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4
0,TICKET001339022,Case Migration,NaN,TICKET01234567895,1002,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,1/1/2023 7:07,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4


# To prepare the file to change owner

In [2]:
import pandas as pd

path=r"D:\Python2\case_2024\change_owner\extract_legacy_calltype_usergroup_caseowner.csv"

df=pd.read_csv(path, delimiter=';')
df.drop(columns=['Column2'])

,SCC_LEGACY_TICKET_ID__C,SCC_CALL_TYPE__C,STATUS,SCC_USER_GROUP__C,OWNERID,CASE_OWNER_X_USER__C,SCC_CASE_OWNER__C
0,TTB000050358103,a1IMg000000DQUbMAO,Escalated,NaN,00GMg0000000sFJMAY,False,SCC - Contact Center
1,TTB000050749322,a1IMg000000DQXYMA4,Escalated,NaN,00GMg0000000sFNMAY,False,SCC - Non Fraud Transaction - ACQUIRER
2,TTB000050956050,a1IMg000000DQXNMA4,Escalated,NaN,00GMg0000000sFrMAI,False,ORD - Fraud Management & Recovery Desk
3,TTB000050357999,a1IMg000000DQUbMAO,Escalated,NaN,00GMg0000000sFJMAY,False,SCC - Contact Center
4,TTB000050357993,a1IMg000000DQUbMAO,Escalated,NaN,00GMg0000000sFJMAY,False,SCC - Contact Center
...,...,...,...,...,...,...,...
47099,TTB000078030896,a1IMg000000DQWJMA4,Escalated,NaN,00GMg0000000sFMMAY,False,SCC - Non Fraud Transaction - ISSUER
47100,TTB000054238436,a1IMg000000DQXPMA4,Escalated,ADMIN QSC,005Mg000001PtAQIA0,True,Migration
47101,TTB000054252514,a1IMg000000DQXPMA4,Escalated,ADMIN QSC,005Mg000001PtAQIA0,True,Migration
47102,TTB000054118155,a1IMg000000DQXPMA4,Escalated,ADMIN QSC,005Mg000001PtAQIA0,True,Migration


In [3]:
import pandas as pd

path=r"D:\Python2\case_2024\change_owner\extract_case_type.csv"

df=pd.read_csv(path, delimiter=';')
df

,NAME,ID
0,1035,a1IMg000000DQSvMAO
1,7999,a1IMg000000DQSwMAO
2,9000,a1IMg000000DQSxMAO
3,8638,a1IMg000000DQSyMAO
4,1037,a1IMg000000DQSzMAO
...,...,...
521,8407,a1IMg000000Ec2EMAS
522,1111,a1IMg000000Ekz3MAC
523,8971,a1IMg000000F2vxMAC
524,8972,a1IMg000000F341MAC


In [17]:
import pandas as pd

# Load the datasets
legacy_df = pd.read_csv(r"D:\Python2\case_2024\change_owner\extract_legacy_calltype_usergroup_caseowner.csv", delimiter=';')
case_type_df = pd.read_csv(r"D:\Python2\case_2024\change_owner\extract_case_type.csv", delimiter=';')

# Merge the dataframes based on the matching columns
merged_df = legacy_df.merge(case_type_df, left_on='SCC_CALL_TYPE__C', right_on='ID', how='left')

# Select relevant columns and rename 'NAME' to 'CALL_TYPE_NAME' for clarity
final_df = merged_df[['SCC_LEGACY_TICKET_ID__C', 'SCC_CALL_TYPE__C', 'STATUS', 'SCC_USER_GROUP__C', 'OWNERID', 'CASE_OWNER_X_USER__C', 'SCC_CASE_OWNER__C', 'NAME']]
final_df.rename(columns={'NAME': 'CALL_TYPE_NAME'}, inplace=True)

final_df=final_df=final_df.drop(['SCC_CALL_TYPE__C', 'CASE_OWNER_X_USER__C','OWNERID'], axis=1)
final_df['Priority']='Low'
final_df['record_type']='Case Migration'
final_df
# len(final_df.SCC_USER_GROUP__C.unique())
# final_df.to_csv(r"D:\Python2\case_2024\change_owner\test_case_ownerUAT.csv",index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_43472\1811329917.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'NAME': 'CALL_TYPE_NAME'}, inplace=True)


,SCC_LEGACY_TICKET_ID__C,STATUS,SCC_USER_GROUP__C,SCC_CASE_OWNER__C,CALL_TYPE_NAME,Priority,record_type
0,TTB000050358103,Escalated,NaN,SCC - Contact Center,2202,Low,Case Migration
1,TTB000050749322,Escalated,NaN,SCC - Non Fraud Transaction - ACQUIRER,8899,Low,Case Migration
2,TTB000050956050,Escalated,NaN,ORD - Fraud Management & Recovery Desk,8711,Low,Case Migration
3,TTB000050357999,Escalated,NaN,SCC - Contact Center,2202,Low,Case Migration
4,TTB000050357993,Escalated,NaN,SCC - Contact Center,2202,Low,Case Migration
...,...,...,...,...,...,...,...
47099,TTB000078030896,Escalated,NaN,SCC - Non Fraud Transaction - ISSUER,8227,Low,Case Migration
47100,TTB000054238436,Escalated,ADMIN QSC,Migration,8713,Low,Case Migration
47101,TTB000054252514,Escalated,ADMIN QSC,Migration,8713,Low,Case Migration
47102,TTB000054118155,Escalated,ADMIN QSC,Migration,8713,Low,Case Migration


# Match group

In [1]:
import pandas as pd

# Define the file paths
extract_group_uat_path = r"D:\Python2\case_2024\change_owner\extract_group_UAT.csv"
test_case_owner_uat_path = r"D:\Python2\case_2024\change_owner\test_case_ownerUAT - 1 line.csv"

# Load the CSV files with the correct delimiter
extract_group_uat = pd.read_csv(extract_group_uat_path, delimiter=';')
test_case_owner_uat = pd.read_csv(test_case_owner_uat_path, delimiter=';')

# Merging the dataframes on the 'case_owner' column
merged_df = pd.merge(test_case_owner_uat, extract_group_uat, how='left', on='case_owner')

merged_df.head()


,SCC_LEGACY_TICKET_ID__C,STATUS,SCC_USER_GROUP__C,SCC_CASE_OWNER__C,CALL_TYPE_NAME,Priority,record_type,case_owner,uker,ID
0,TTB000051463308,Escalated,CSM-1,CDD - Business Operation Team - ITEM,7200,Low,Case Migration,SCC - Contact Center,#VALUE!,00GMR0000000fqq2AA
1,TTB000051463308,Escalated,CSM-1,CDD - Business Operation Team - ITEM,7200,Low,Case Migration,SCC - Contact Center,#VALUE!,00GMR0000001ZOj2AM


In [1]:
"C:\Users\maste\Downloads\Flexible_CSV_Data.csv"

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [2]:
import os
import pandas as pd

# Directory containing the CSV files
file_path = r"D:\Python2\remark\clean"

import pandas as pd



try:
    # Read the CSV file into a DataFrame with error handling
    df = pd.read_csv(file_path, error_bad_lines=False, warn_bad_lines=True)

    # Count the number of rows in the DataFrame
    row_count = len(df)

    # Print the row count for the file
    print(f'{file_path}: {row_count} rows')

    # Print the total row count (since there's only one file here)
    print(f'Total rows in the file: {row_count}')
except Exception as e:
    print(f'Error reading {file_path}: {e}')



Error reading D:\Python2\remark\clean: read_csv() got an unexpected keyword argument 'error_bad_lines'


# Count all TTB in all files in a folder for REMARK

In [9]:
import os

def count_ttb_in_file(file_path):
    ttb_count = 0
    with open(file_path, 'r', errors='ignore') as file:
        for line in file:
            ttb_count += line.count('TTB')
    return ttb_count

def count_ttb_in_folder(folder_path):
    total_ttb_count = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            total_ttb_count += count_ttb_in_file(file_path)
    return total_ttb_count

# Specify the folder path
folder_path = r"D:\Python2\remark\bricare_all_batch2_troublelog\bricare_all_batch2_troublelog"

# Count the number of 'TTB' entries in all files within the folder
total_ttb_count = count_ttb_in_folder(folder_path)
print(f"Total number of 'TTB' entries in all files within the folder: {total_ttb_count}")


Total number of 'TTB' entries in all files within the folder: 3936352


In [3]:
import os
import re

def find_unique_ttb_in_file(file_path):
    unique_ttb_set = set()
    with open(file_path, 'r', errors='ignore') as file:
        for line in file:
            ttb_matches = re.findall(r'TTB\d{12}', line)
            unique_ttb_set.update(ttb_matches)
    return unique_ttb_set

def find_unique_ttb_in_folder(folder_path):
    total_unique_ttb_set = set()
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            unique_ttb_set = find_unique_ttb_in_file(file_path)
            total_unique_ttb_set.update(unique_ttb_set)
    return total_unique_ttb_set

# Specify the folder path
folder_path = r"D:\Python2\remark\clean"

# Find unique 'TTB' entries in all files within the folder
total_unique_ttb_set = find_unique_ttb_in_folder(folder_path)
print(f"Total number of unique 'TTB' entries in all files within the folder: {len(total_unique_ttb_set)}")


Total number of unique 'TTB' entries in all files within the folder: 16840


# ACCOUNT TO CHECK THE DUPLICATES

In [3]:
import pandas as pd


# Load the CSV file
# file_path = r"D:\Python2\case_2024\5-6 aug\data_akun_5_6\error080724121347897.csv"
file_path - r"D:\Python2\case_2024\5-6 aug\data_akun_5_6\error080724122100281.csv"
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
df.head()

# Filter rows where the 'ERROR' column does not contain the word 'duplicate'
filtered_df = df[~df['ERROR'].str.contains('duplicate', case=False)]


# Display the first few rows of the filtered dataframe
filtered_df.head()


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [26]:
import pandas as pd

path=r"C:\Users\maste\Downloads\bricare_20240111_20240729_0_case_ttb_missing_79_clean.csv"

df=pd.read_csv(path)
df['status'].fillna('Escalated', inplace=True)
empty_values_exist = df['status'].isnull().sum()
df.to_csv(r"C:\Users\maste\Downloads\bricare_20240111_20240729_0_case_ttb_missing_79_clean_new.csv", index=False)
# empty_values_exist

df

C:\Users\maste\AppData\Local\Temp\ipykernel_21000\2974680954.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['status'].fillna('Escalated', inplace=True)


,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,Nominal,...,Tgl_In_Progress,Tgl_Returned,Ticket_Referensi,Tiket_Urgency,Tipe_Remark,UniqueID,users,Usergroup_ID,record_type,Legacy_Ticket_ID
0,RPY5441,TTB000049954663,8907,Taspen,2024-01-11 20:32:35,MMS,Request,ROSNITA,7.387010e+14,0.0,...,NaN,2024-08-05 00:00:00,NaN,NaN,Notes,NaN,Non Call,8856.0,Case Migration,TTB0049954663
1,YGZ0987,TTB000052590234,8602,BRIMO,2024-05-09 16:47:07,Phone,Platform & Device Support,YUNDHASTRI WULANDARI,1.146010e+14,0.0,...,NaN,2024-08-05 00:00:00,NaN,NaN,Notes,NaN,Call,4.0,Case Migration,TTB0052590234
2,3565101280220705,TTB000052655911,1101,Sanggahan Pengajuan Kartu Kredit,2024-05-13 14:36:17,Phone,Fraud / Suspicious Transactions,RISTRI ARIKA,NaN,0.0,...,NaN,2024-08-05 00:00:00,NaN,NaN,Notes,NaN,Call,5.0,Case Migration,TTB0052655911
3,NBC5930,TTB000052760439,8907,Taspen,2024-05-17 13:37:08,SMS,Request,BESTY SUYANTO,4.200102e+13,0.0,...,NaN,2024-08-05 00:00:00,NaN,NaN,Notes,NaN,Non Call,2908.0,Case Migration,TTB0052760439
4,P045610,TTB000052765512,8907,Taspen,2024-05-17 14:30:58,SMS,Request,PAINTEN,4.010275e+11,0.0,...,NaN,2024-08-05 00:00:00,NaN,NaN,Notes,NaN,Non Call,5490.0,Case Migration,TTB0052765512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,FA91164,TTB000054244476,1038,CERIA - Komplain Transaksi Tidak Diakui,2024-07-29 09:56:15,Phone,Fraud / Suspicious Transactions,FATMA ALI HAMID LARIMA,5.660101e+13,0.0,...,NaN,2024-07-31 11:43:36,NaN,NaN,Notes,NaN,Call,10102.0,Case Migration,TTB0054244476
248,ANEY666,TTB000054249329,8411,Salah Transfer antar BRI,2024-07-29 12:04:27,Phone,NaN,AHMAD HABIBI,3.554010e+14,2700000.0,...,2024-07-31 14:57:54,2024-07-31 10:12:28,NaN,NaN,Notes,NaN,Call,5035.0,Case Migration,TTB0054249329
249,4687400201234600,TTB000054255107,2700,Informasi dan Pendaftaran Autodebet Kartu Kredit,2024-07-29 14:57:11,Surat,NaN,JACUB SUTISNA,NaN,0.0,...,NaN,2024-07-31 00:00:00,NaN,NaN,Notes,NaN,Non Call,15.0,Case Migration,TTB0054255107
250,NIP3888,TTB000054256033,8713,Gagal Top Up dan Gagal Transaksi BRIZZI tapi S...,2024-07-29 15:30:32,Walk-In,Complaint - Transaction,nuke julianti,5.080101e+13,100000.0,...,2024-08-01 15:32:51,2024-07-31 19:30:13,NaN,NaN,Notes,NaN,Non Call,2301.0,Case Migration,TTB0054256033


In [3]:
import pandas as pd

# path=r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean\error080824043430366.csv"

path=r"D:\Python2\error081624121056959.csv"


df=pd.read_csv(path)
df['Jenis_Nasabah'] = None  
df['tipe_nasabah'] = 'Individu'
df['record_type_1'] = 'Person Account'
df.to_csv(r"D:\Python2\error081624121056959_done.csv", index=False)

In [6]:
import pandas as pd
import re

# Function to read and process the CSV file with a multi-character delimiter
def read_and_clean_csv(file_path, delimiter):
    with open(file_path, 'r') as file:
        content = file.read()
    
    lines = content.split('\n')
    parsed_data = []
    for line in lines:
        if delimiter in line:
            split_line = line.split(delimiter)
            # Ensuring the split line has exactly 3 elements (TroubleTicketId, Notes, Status)
            if len(split_line) == 3:
                parsed_data.append(split_line)
    
    return parsed_data

# Define the file path and multi-character delimiter
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Drone_20240808.csv"
delimiter = '|-|-|'

# Read and clean the new file using the custom delimiter
new_cleaned_data = read_and_clean_csv(file_path, delimiter)

# Convert the list of rows into a DataFrame
cleaned_df = pd.DataFrame(new_cleaned_data, columns=['TroubleTicketId', 'Notes', 'Status'])

# Filter rows where TroubleTicketId follows the TICKET format
cleaned_df = cleaned_df[cleaned_df['TroubleTicketId'].apply(lambda x: re.match(r'^TICKET\d+$', x) is not None)]

# Further clean and format the DataFrame to match the expected output
cleaned_df['Notes'] = cleaned_df['Notes'].str.replace('\n', ' ').str.strip()
cleaned_df['TroubleTicketId'] = cleaned_df['TroubleTicketId'].str.strip()
cleaned_df['Status'] = cleaned_df['Status'].str.strip()

# Save the cleaned DataFrame to a new CSV file
cleaned_df.to_csv('Cleaned_Drone_20240808.csv', index=False)

# Display the head and summary of the cleaned DataFrame
print(cleaned_df.head())
print(cleaned_df.describe(include='all'))


Empty DataFrame
Columns: [TroubleTicketId, Notes, Status]
Index: []
       TroubleTicketId Notes Status
count                0     0      0
unique               0     0      0
top                NaN   NaN    NaN
freq               NaN   NaN    NaN


In [8]:
import pandas as pd

path=r"C:\Users\maste\Downloads\Transformed_Ticket_IDs.csv"

df=pd.read_csv(path)
df.to_csv(r"C:\Users\maste\Downloads\Transformed_Ticket_IDs_1line.csv", index=False)

# Correct Nomer kartu dan Nomer rekening

In [ ]:
import pandas as pd

# Load the CSV file with the correct delimiter
file_path = '/mnt/data/ttbcc.csv'
df = pd.read_csv(file_path, delimiter=';')

# Define the transformation function
def transform_ticket_id(ticket_id):
    if ticket_id.startswith('TTB'):
        return 'TTB' + ticket_id[3:].zfill(12)
    return ticket_id

# Apply the transformation to the scc_legacy_ticket_id__c column
df['scc_legacy_ticket_id__c'] = df['scc_legacy_ticket_id__c'].apply(transform_ticket_id)

# Save the updated dataframe to a new CSV file
output_file_path = '/mnt/data/transformed_ttbcc.csv'
df.to_csv(output_file_path, index=False, sep=';')


# Drone

In [15]:
import pandas as pd

# Function to parse the file based on the clarified structure with correct delimiter
def parse_file_preserve_format(file_path):
    with open(file_path, 'r') as file:
        data = []
        ticket_data = {}
        current_ticket_id = None
        current_notes = []
        
        for line in file:
            if '|-|-|' in line:
                parts = line.strip().split('|-|-|')
                
                ticket_id = parts[0].strip('"')
                if ticket_id.startswith("TICKET") or ticket_id.startswith("TTB"):
                    if current_ticket_id is not None:
                        ticket_data['Notes'] = '\n'.join(current_notes).strip().strip('"')
                        data.append(ticket_data)
                    
                    ticket_data = {'TroubleTicketId': ticket_id, 'Notes': '', 'Status': ''}
                    current_ticket_id = ticket_id
                    current_notes = []
                    
                    if len(parts) > 1:
                        current_notes.append(parts[1].strip())
                    
                    if len(parts) > 2:
                        ticket_data['Status'] = parts[2].strip('"')
                else:
                    if current_ticket_id is not None:
                        current_notes.append(line.rstrip())
            else:
                if current_ticket_id is not None:
                    current_notes.append(line.rstrip())
        
        if current_ticket_id is not None:
            ticket_data['Notes'] = '\n'.join(current_notes).strip().strip('"')
            data.append(ticket_data)
    
    return pd.DataFrame(data)

# Parse the file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Drone\drone20240808.csv"
df_parsed_correctly = parse_file_preserve_format(file_path)

# Function to correctly extract Status from Notes
def extract_status(notes):
    if '|-|-|' in notes:
        return notes.split('|-|-|')[-1].strip('"')
    return ''

df_parsed_correctly['Status'] = df_parsed_correctly['Notes'].apply(extract_status)

# Clean the Notes column to remove Status part
df_parsed_correctly['Notes'] = df_parsed_correctly['Notes'].apply(lambda x: x.split('|-|-|')[0].strip())

# Remove surrounding double quotes from Notes
df_parsed_correctly['Notes'] = df_parsed_correctly['Notes'].apply(lambda x: x.strip('"'))

df_parsed_correctly.to_csv('cleaned_drone.csv', index=False)


In [16]:
import pandas as pd

# Function to parse the file based on the clarified structure with correct delimiter
def parse_file_preserve_format(file_path):
    with open(file_path, 'r') as file:
        data = []
        ticket_data = {}
        current_ticket_id = None
        current_notes = []
        
        for line in file:
            if '|-|-|' in line:
                parts = line.strip().split('|-|-|')
                
                ticket_id = parts[0].strip('"')
                if ticket_id.startswith("TICKET") or ticket_id.startswith("TTB"):
                    if current_ticket_id is not None:
                        ticket_data['Notes'] = '\n'.join(current_notes).strip().strip('"')
                        data.append(ticket_data)
                    
                    ticket_data = {'TroubleTicketId': ticket_id, 'Notes': '', 'Status': ''}
                    current_ticket_id = ticket_id
                    current_notes = []
                    
                    if len(parts) > 1:
                        current_notes.append(parts[1].strip())
                    
                    if len(parts) > 2:
                        ticket_data['Status'] = parts[2].strip('"')
                else:
                    if current_ticket_id is not None:
                        current_notes.append(line.rstrip())
            else:
                if current_ticket_id is not None:
                    current_notes.append(line.rstrip())
        
        if current_ticket_id is not None:
            ticket_data['Notes'] = '\n'.join(current_notes).strip().strip('"')
            data.append(ticket_data)
    
    return pd.DataFrame(data)

# Function to clean up Notes column
def clean_notes(notes):
    notes = notes.replace('\\n\\n', '\n').replace('\\n', '\n').replace('\n\\', '\n').strip()
    return notes

# Parse the file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Drone\drone20240808.csv"
df_parsed_correctly = parse_file_preserve_format(file_path)

# Function to correctly extract Status from Notes
def extract_status(notes):
    if '|-|-|' in notes:
        return notes.split('|-|-|')[-1].strip('"')
    return ''

df_parsed_correctly['Status'] = df_parsed_correctly['Notes'].apply(extract_status)

# Clean the Notes column to remove Status part and clean newlines
df_parsed_correctly['Notes'] = df_parsed_correctly['Notes'].apply(lambda x: clean_notes(x.split('|-|-|')[0].strip()))

# Remove surrounding double quotes from Notes
df_parsed_correctly['Notes'] = df_parsed_correctly['Notes'].apply(lambda x: x.strip('"'))

df_parsed_correctly.to_csv('cleaned_drone2.csv', index=False)
df_parsed_correctly.head()


,TroubleTicketId,Notes,Status
0,TTB000053698249,{\n \&#34;NAMA MERCHANT\&#34;: \&#34;NASI KEB...,Closed
1,TTB000053698530,{\n \&#34;NAMA MERCHANT\&#34;: \&#34;CAMBRIGE...,Closed
2,TTB000053698541,{\n \&#34;NAMA MERCHANT\&#34;: \&#34;KAMADIKA...,Closed
3,TTB000053699338,{\n \&#34;NAMA MERCHANT\&#34;: \&#34;MOORI BE...,Closed
4,TTB000053699344,{\n \&#34;NAMA MERCHANT\&#34;: \&#34;BIANCA R...,Closed


# To close DRONE Tickets

## Find max and min TTB

In [1]:
import pandas as pd


# path=r"D:\Python2\datadroneclosed.csv"
path=r"D:\Python2\Close5\testing SABRINA.csv"

df=pd.read_csv(path, delimiter=';')
# df=df.iloc[:1]
# df.to_csv(r"D:\Python2\extract_droneStatus_open_1lines.csv", index=False)
min_value = df['SCC_LEGACY_TICKET_ID__C'].min()
max_value = df['SCC_LEGACY_TICKET_ID__C'].max()
df.to_csv(path, index=False)
min_value, max_value

('TTB000078498061', 'TTB000078498646')

## Check the missing values

In [16]:
import pandas as pd

# Load the CSV files
csv_file_path_1 = r"D:\Python2\Close4\cobaclosed1.csv"
csv_file_path_2 = r"D:\Python2\Close4\save_merged_file.csv"

# Reading the CSV files
csv_df_1 = pd.read_csv(csv_file_path_1)
csv_df_2 = pd.read_csv(csv_file_path_2)

# Extracting the SCC_LEGACY_TICKET_ID__C column from both dataframes
ids_1 = csv_df_1['SCC_LEGACY_TICKET_ID__C']
ids_2 = csv_df_2['SCC_LEGACY_TICKET_ID__C']

# Identifying missing values
missing_in_1 = ids_2[~ids_2.isin(ids_1)]
missing_in_2 = ids_1[~ids_1.isin(ids_2)]

# Creating DataFrames for missing values
missing_in_1_df = pd.DataFrame(missing_in_1, columns=['SCC_LEGACY_TICKET_ID__C'])
missing_in_2_df = pd.DataFrame(missing_in_2, columns=['SCC_LEGACY_TICKET_ID__C'])

# Saving the results to CSV files
# missing_in_1_df.to_csv('missing_in_cobaclosed1.csv', index=False)
missing_in_2_df.to_csv('missing_in_save_merged_file.csv', index=False)
missing_in_2_df
# print("Missing values in cobaclosed1.csv saved to missing_in_cobaclosed1.csv")
# print("Missing values in save_merged_file.csv saved to missing_in_save_merged_file.csv")


,SCC_LEGACY_TICKET_ID__C
0,TTB000054447632
1,TTB000054442717
2,TTB000054446048
3,TTB000054446181
4,TTB000054419622
5,TTB000054412180
6,TTB000054412649
197,TTB000054389346
198,TTB000054441582
214,TTB000054367079


## Filter

In [7]:
import pandas as pd

# Load the two CSV files
extract_file_path = r"D:\Python2\Close7\extract_23_done.csv"
datadroneclosed_file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy.csv"

extract_df = pd.read_csv(extract_file_path)
datadroneclosed_df = pd.read_csv(datadroneclosed_file_path, delimiter=';')

# Identify the columns in extract that are not in datadroneclosed
remaining_columns = [col for col in extract_df.columns if col not in datadroneclosed_df.columns]

# Merge the dataframes: take all columns from datadroneclosed and the remaining columns from extract
merged_df = pd.merge(datadroneclosed_df, extract_df[remaining_columns + ['SCC_LEGACY_TICKET_ID__C']], on='SCC_LEGACY_TICKET_ID__C', how='inner')

# Save the merged dataframe to a new CSV file
output_file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv" 
merged_df.to_csv(output_file_path, index=False)

print(f"Merged data saved to {output_file_path}")


Merged data saved to D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv


In [15]:
import pandas as pd

path=r"D:\Python2\Close7\extract_117_dataloader.csv"

df=pd.read_csv(path)
df=df[df['STATUS']=="Escalated"]
df.to_csv("D:\Python2\Close7\extract_117_dataloader_open.csv",index=False)

# Filter Drone ADI

In [1]:
import pandas as pd

# Load the two CSV files
extract_file_path = r"D:\extract_drone_.csv"
datadroneclosed_file_path = r"D:\Python2\drone_open.csv"  

# Load the dataframes
extract_df = pd.read_csv(extract_file_path)
datadroneclosed_df = pd.read_csv(datadroneclosed_file_path, delimiter=';')

# Get the list of SCC_LEGACY_TICKET_ID__C from datadroneclosed_df
filter_ids = datadroneclosed_df['CASENUMBER'].unique()

# Filter the extract_df based on the SCC_LEGACY_TICKET_ID__C values in datadroneclosed_df
filtered_df = extract_df[extract_df['CASENUMBER'].isin(filter_ids)]

# Save the filtered dataframe to a new CSV file
filtered_output_file_path = r'D:\Python2\filtered_extract_drone_not_migration2.csv'  # Update this with your desired output file path
filtered_df.to_csv(filtered_output_file_path, index=False)

print(f"Filtered data saved to {filtered_output_file_path}")


C:\Users\maste\AppData\Local\Temp\ipykernel_17640\2193017327.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  extract_df = pd.read_csv(extract_file_path)


Filtered data saved to D:\Python2\filtered_extract_drone_not_migration2.csv


## Fix no telp

In [8]:
# import pandas as pd
# import re

# # Load the merged CSV file
# merged_file_path = r"D:\Python2\Close5\save_merged_file_sabrina_done2.csv"

# # Function to clean phone numbers
# def clean_phone_number(phone):
#     if pd.isna(phone):
#         return phone
#     # Remove non-numeric characters
#     phone = re.sub(r'\D', '', str(phone))
#     # Ensure the phone number is between 9 and 14 digits
#     if 9 <= len(phone) <= 14:
#         return phone
#     return ''  # Return empty if it doesn't meet the criteria

# # Apply the function to the Cust_Current_Phone__c column
# merged_df['CUST_CURRENT_PHONE__C'] = merged_df['CUST_CURRENT_PHONE__C'].apply(clean_phone_number)

# # Save the cleaned file
# cleaned_file_path = r"D:\Python2\Close5\save_merged_file_sabrina_done3.csv"
# merged_df.to_csv(cleaned_file_path, index=False)

# print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to D:\Python2\Close5\save_merged_file_sabrina_done3.csv


In [14]:
import pandas as pd

path=r"D:\Python2\save_merged_file_7k.csv"

df=pd.read_csv(path)
df=df['CUST_CURRENT_PHONE__C']

C:\Users\maste\AppData\Local\Temp\ipykernel_4360\1078473945.py:5: DtypeWarning: Columns (11,33,45,60,64,65,96,112,120,124,127,130,134,136,142,150,167,173,174,190,209,210,220,221,273,275,276,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


0    8.123045e+13
1             NaN
2    8.536053e+13
3    8.564362e+13
4    8.515682e+13
Name: CUST_CURRENT_PHONE__C, dtype: float64

In [14]:
# import pandas as pd

# # Load the uploaded CSV file
# input_file_path = r"D:\Python2\Close4\save_merged_file2.csv"
# df_latest = pd.read_csv(input_file_path)

# # Reprocess the dates by forcing them to be timezone-naive
# def correct_dates_force_timezone_naive(row):
#     try:
#         # Convert to datetime and force remove any timezones
#         row['CREATEDDATE'] = pd.to_datetime(row['CREATEDDATE'], errors='coerce').tz_localize(None)
#         row['CLOSEDDATE'] = pd.to_datetime(row['CLOSEDDATE'], errors='coerce').tz_localize(None)

#         # Apply the rule: if CLOSEDDATE < CREATEDDATE, set CREATEDDATE = CLOSEDDATE
#         if pd.notnull(row['CLOSEDDATE']) and pd.notnull(row['CREATEDDATE']):
#             if row['CLOSEDDATE'] < row['CREATEDDATE']:
#                 row['CREATEDDATE'] = row['CLOSEDDATE']

#     except Exception as e:
#         print(f"Error processing row: {e}")  # Print the error for debugging, remove if not needed

#     return row

# # Apply the date correction function to the dataframe
# df_final_corrected = df_latest.apply(correct_dates_force_timezone_naive, axis=1)

# # Save the corrected dataframe to a new CSV file
# output_file_path_final_corrected = r"D:\Python2\Close4\save_merged_file2.csv"
# df_final_corrected.to_csv(output_file_path_final_corrected, index=False)

# output_file_path_final_corrected



'D:\\Python2\\Close4\\save_merged_file2.csv'

In [2]:
import pandas as pd
from datetime import datetime

# Custom date parser function to handle Indonesian month names
def parse_date(date_str):
    months = {
        'Januari': 'January', 'Februari': 'February', 'Maret': 'March',
        'April': 'April', 'Mei': 'May', 'Juni': 'June',
        'Juli': 'July', 'Agustus': 'August', 'September': 'September',
        'Oktober': 'October', 'November': 'November', 'Desember': 'December'
    }
    
    for ind, eng in months.items():
        date_str = date_str.replace(ind, eng)
    
    return datetime.strptime(date_str, '%d %B %Y')

# Load the CSV file
file_path = r"D:\Python2\Close7\New Bricare Issuer 2 Over SLA(3) - Copy.csv"
df = pd.read_csv(file_path, delimiter=';')

# Convert the 'CLOSEDDATE' column using the custom parser
df['CLOSEDDATE'] = df['CLOSEDDATE'].apply(parse_date).dt.strftime('%d/%m/%Y %H:%M:%S')

# Save the updated dataframe back to a CSV file
output_file_path = r"D:\Python2\Close7\New Bricare Issuer 2 Over SLA(3) - Copy.csv"
df.to_csv(output_file_path, index=False)

print(f"File has been updated and saved as {output_file_path}")


AttributeError: 'float' object has no attribute 'replace'

In [3]:
import pandas as pd
from datetime import datetime

# Custom date parser function to handle Indonesian month names
def parse_date_safe(date_str):
    # Check if the input is a string to avoid issues with NaN or non-string values
    if isinstance(date_str, str):
        months = {
            'Januari': 'January', 'Februari': 'February', 'Maret': 'March',
            'April': 'April', 'Mei': 'May', 'Juni': 'June',
            'Juli': 'July', 'Agustus': 'August', 'September': 'September',
            'Oktober': 'October', 'November': 'November', 'Desember': 'December'
        }
        
        # Replace Indonesian month names with English equivalents
        for ind, eng in months.items():
            date_str = date_str.replace(ind, eng)
        
        # Parse the date string into a datetime object
        return datetime.strptime(date_str, '%d %B %Y')
    return date_str  # Return as-is if not a string

# Load the CSV file with the correct delimiter
file_path = r"D:\Python2\Close7\New Bricare Issuer 2 Over SLA(3) - Copy.csv"
df = pd.read_csv(file_path, delimiter=';')

# Apply the date parsing function and handle NaT values properly
df['CLOSEDDATE'] = df['CLOSEDDATE'].apply(parse_date_safe).apply(
    lambda x: x.strftime('%d/%m/%Y %H:%M:%S') if isinstance(x, pd.Timestamp) else ''
)

# Save the updated dataframe back to a CSV file
output_file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy.csv"
df.to_csv(output_file_path, index=False, sep=';')

print(f"File has been updated and saved as {output_file_path}")


File has been updated and saved as D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy.csv


In [9]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv"
df = pd.read_csv(file_path)

# Convert the CLOSEDDATE column to the desired format 'yyyy-mm-dd hh:mm'
df['CLOSEDDATE'] = pd.to_datetime(df['CLOSEDDATE'], format='%d/%m/%Y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M')

# Save the updated DataFrame to a new CSV file
updated_file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv"
df.to_csv(updated_file_path, index=False)

print(f"Updated file saved to: {updated_file_path}")


Updated file saved to: D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv


## Status and to fix no telp

In [23]:
from datetime import datetime, timedelta
import pandas as pd

path=r"D:\Python2\Close7\extract_117_dataloader_open_cleaned.csv"

df=pd.read_csv(path)
# df['CLOSEDDATE']=df['CREATEDDATE']

# df['CREATEDDATE'] = pd.to_datetime(df['CREATEDDATE'])
# df['CLOSEDDATE'] = df['CREATEDDATE'] + timedelta(hours=1)


# df['SCC_CUST_PHONE1__C'] = '0' + df['SCC_CUST_PHONE1__C'].astype(str)
# df['CUST_CURRENT_PHONE__C'] = '0' + df['CUST_CURRENT_PHONE__C'].astype(str)
df['SUPPLIEDPHONE'] = '0' + df['SUPPLIEDPHONE'].astype(str)

# df['STATUS']='Closed'

df.to_csv(r"D:\Python2\Close7\extract_117_dataloader_open_cleaned.csv", index=False)

# Filter Drone for ERROR

In [3]:
import pandas as pd

# Load the merged CSV file
# merged_df = pd.read_csv(r"D:\Python2\save_merged_file.csv")

merged_df = pd.read_csv(r"D:\Python2\Close2\extract_Prod.csv")



# Load the Excel file
# error_df = pd.read_excel(r"D:\Python2\error_221.xlsx")

error_df = pd.read_excel(r"D:\Python2\Close2\1281.xlsx")

# Filter the merged dataframe using the ticket IDs from the error dataframe
filtered_df = merged_df[merged_df['SCC_LEGACY_TICKET_ID__C'].isin(error_df['SCC_LEGACY_TICKET_ID__C'])]

# filtered_df['CLOSEDDATE'] = pd.to_datetime(df['CLOSEDDATE'], format='%d/%m/%Y %H:%M:%S')


# Save the filtered dataframe to a new CSV file
filtered_file_path = r"D:\Python2\save_merged_file_1281.csv"

filtered_df.to_csv(filtered_file_path, index=False)

filtered_df.shape, filtered_file_path


C:\Users\maste\AppData\Local\Temp\ipykernel_26632\1762696848.py:6: DtypeWarning: Columns (17,44,61,62,93,97,104,109,110,111,120,127,133,135,136,139,141,145,148,151,165,172,175,189,208,209,219,273,283) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv(r"D:\Python2\Close2\extract_Prod.csv")


((1281, 300), 'D:\\Python2\\save_merged_file_1281.csv')

# Add additional sentences in column SCC_DRONE_REMARK_UPDATE__C

In [8]:
import pandas as pd

# Load the CSV file
file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv" 
df = pd.read_csv(file_path)

# Add the sentences from 'REMARK' to the end of 'SCC_DRONE_REMARK_UPDATE__C' on a new line
df['SCC_DRONE_REMARK_UPDATE__C'] = df['SCC_DRONE_REMARK_UPDATE__C'].fillna('') + '\n' + df['REMARK'].fillna('')

# Save the updated dataframe back to a CSV file
output_file_path = r"D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv" 
df.to_csv(output_file_path, index=False)

print(f"File has been updated and saved as {output_file_path}")


File has been updated and saved as D:\Python2\Close7\Updated_New_Bricare_Issuer_2_Over_SLA_Copy_DONE.csv


In [7]:
# import pandas as pd

# # Load the CSV file
# file_path = r"D:\Python2\Close2\extract_1line_UAT.csv"
# df = pd.read_csv(file_path)

# # Define the additional sentence to be added
# additional_sentence = "This is the new sentence."

# # Function to add the sentence in a new line
# def add_sentence_in_new_line(remark):
#     if pd.isna(remark):  # Check if the cell is empty or NaN
#         return additional_sentence
#     else:
#         return f"{remark}\n{additional_sentence}"

# # Apply the function to the SCC_DRONE_REMARK_UPDATE__C column
# df['SCC_DRONE_REMARK_UPDATE__C'] = df['SCC_DRONE_REMARK_UPDATE__C'].apply(add_sentence_in_new_line)

# # Save the modified DataFrame back to the CSV file
# df.to_csv(r"D:\Python2\Close2\extract_1line_UAT_remarkupdated.csv", index=False)

# print("The additional sentence has been added and the file is saved as 'extract_1line_UAT_updated.csv'.")


In [12]:
import pandas as pd

# Load the CSV file
# file_path = r"D:\Python2\Close2\save_merged_file_1281.csv"
file_path=r"D:\Python2\Close6\extract_17_done.csv"
df = pd.read_csv(file_path)

# Define the additional sentence to be added
additional_sentence = "Mohon lengkapi dengan emergency receipt/ struk transaksi."

# Function to add the sentence in a new line
def add_sentence_in_new_line(remark):
    if pd.isna(remark): 
        return additional_sentence
    else:
        return f"{remark}\n{additional_sentence}"

# Apply the function to the SCC_DRONE_REMARK_UPDATE__C column
df['SCC_DRONE_REMARK_UPDATE__C'] = df['SCC_DRONE_REMARK_UPDATE__C'].apply(add_sentence_in_new_line)

# # Fill the STATUS column with "Closed"
# df['STATUS'] = 'Closed'

# # Fill the CLOSEDDATE column with the same value as CREATEDDATE
# df['CLOSEDDATE'] = df['CREATEDDATE']


# Save the modified DataFrame back to the CSV file
df.to_csv(r"D:\Python2\Close6\extract_17_done.csv", index=False)




## to remove .0

In [20]:
import pandas as pd
from pandas.tseries.offsets import BDay

# Load the dataset
file_path = r"D:\Python2\Close7\extract_117_dataloader_open.csv"
df = pd.read_csv(file_path)

# Remove ".0" at the end of values in 'SCC_CARD_NUMBER__C' and 'SCC_ACCOUNT_NUMBER__C'
df['SCC_CARD_NUMBER__C'] = df['SCC_CARD_NUMBER__C'].astype(str).str.replace('.0', '', regex=False)
df['SCC_ACCOUNT_NUMBER__C'] = df['SCC_ACCOUNT_NUMBER__C'].astype(str).str.replace('.0', '', regex=False)

# Convert 'CREATEDDATE' to datetime and add 3 business days
df['CREATEDDATE'] = pd.to_datetime(df['CREATEDDATE'], errors='coerce')
df['CLOSEDDATE'] = df['CREATEDDATE'] + BDay(3)

# # Save the modified DataFrame to a new CSV file
# output_file_path = r"D:\Python2\Close7\extract_117_dataloader_open_cleaned.csv"
# df.to_csv(output_file_path, index=False)

# print("Modifications completed and saved to:", output_file_path)
# Check if '.0' is still present in SCC_CARD_NUMBER__C and SCC_ACCOUNT_NUMBER__C

# Check SCC_CARD_NUMBER__C
if df['SCC_CARD_NUMBER__C'].str.endswith('.0').any():
    print("Some values in 'SCC_CARD_NUMBER__C' still end with '.0'.")
else:
    print("All values in 'SCC_CARD_NUMBER__C' have '.0' removed.")

# Check SCC_ACCOUNT_NUMBER__C
if df['SCC_ACCOUNT_NUMBER__C'].str.endswith('.0').any():
    print("Some values in 'SCC_ACCOUNT_NUMBER__C' still end with '.0'.")
else:
    print("All values in 'SCC_ACCOUNT_NUMBER__C' have '.0' removed.")


All values in 'SCC_CARD_NUMBER__C' have '.0' removed.
All values in 'SCC_ACCOUNT_NUMBER__C' have '.0' removed.


In [32]:
import pandas as pd

path=r"D:\Python2\Close7\extract_117_dataloader_open_cleaned.csv"

df=pd.read_csv(path)
df=df[df['ACCOUNTID']=='001Mg0000089X1VIAU']
df=df[df['CUST_CURRENT_PHONE__C']==0]
df['CUST_CURRENT_PHONE__C']=''
df.to_csv(r"D:\Python2\Close7\extract_117_dataloader_open_cleaned_1lineerror.csv", index=False)

In [10]:
import pandas as pd

# Read the CSV file into a DataFrame
filtered_df = pd.read_csv(r"D:\Python2\save_merged_file_7k.csv")

# Convert the 'CREATEDDATE' and 'CLOSEDDATE' columns to datetime format
filtered_df['CREATEDDATE'] = pd.to_datetime(filtered_df['CREATEDDATE'])
filtered_df['CLOSEDDATE'] = pd.to_datetime(filtered_df['CLOSEDDATE'])

# Specify the file path to save the DataFrame
filtered_file_path = r"D:\Python2\save_merged_file_7k.csv"

# Save the DataFrame back to a CSV file
filtered_df.to_csv(filtered_file_path, index=False)

# Output the shape of the DataFrame and the file path
filtered_df.shape, filtered_file_path


C:\Users\maste\AppData\Local\Temp\ipykernel_4360\1763359392.py:4: DtypeWarning: Columns (11,33,45,60,64,65,96,112,120,124,127,130,134,136,142,150,167,173,174,190,209,210,220,221,273,275,276,284) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv(r"D:\Python2\save_merged_file_7k.csv")
C:\Users\maste\AppData\Local\Temp\ipykernel_4360\1763359392.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  filtered_df['CLOSEDDATE'] = pd.to_datetime(filtered_df['CLOSEDDATE'])


((7395, 300), 'D:\\Python2\\save_merged_file_7k.csv')

# Waktu Transaksi

In [18]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Load the dataset
file_path = r"D:\error080824083548216.csv"
df = pd.read_csv(file_path)

# Generate a list of random times within the same day
base_time = datetime.strptime('2024-08-07T07:00:00.000+0000', '%Y-%m-%dT%H:%M:%S.%f%z')
time_deltas = [timedelta(hours=i) for i in range(24)]  # List of hourly intervals within a day

# Randomly assign these times to the column
random_times = [base_time + random.choice(time_deltas) for _ in range(len(df))]
df['SCC_WAKTU_TRANSAKSI__C'] = [time.strftime('%Y-%m-%dT%H:%M:%S.%f%z') for time in random_times]

# Save the modified dataframe
output_file_path = r"D:\error080824083548216_clean.csv"
df.to_csv(output_file_path, index=False)


output_file_path


'D:\\error080824083548216_clean.csv'

file_path = r"D:\error080824083548216.csv"

In [23]:
import pandas as pd
from datetime import datetime, timedelta
import random
import re

# Load the dataset
file_path = r"D:\error080824083548216.csv"
df = pd.read_csv(file_path)

# Generate a list of random times within the same day
base_time = datetime.strptime('2024-08-07T07:00:00.000+0000', '%Y-%m-%dT%H:%M:%S.%f%z')
time_deltas = [timedelta(hours=i) for i in range(24)]  # List of hourly intervals within a day

# Randomly assign these times to the column
random_times = [base_time + random.choice(time_deltas) for _ in range(len(df))]
df['SCC_WAKTU_TRANSAKSI__C'] = [time.strftime('%Y-%m-%dT%H:%M:%S.%f%z') for time in random_times]

# Function to check if phone number is valid
def is_valid_phone(phone):
    # Define a simple regex for a valid phone number (adjust as needed)
    phone_pattern = re.compile(r'^\+?\d[\d\s-]{7,}\d$')
    return bool(phone_pattern.match(phone))

# Convert phone numbers to string and then validate
df['CUST_CURRENT_PHONE__C'] = df['CUST_CURRENT_PHONE__C'].astype(str)

# Check the phone numbers
df['is_valid_phone'] = df['CUST_CURRENT_PHONE__C'].apply(is_valid_phone)

# Split into valid and invalid phone numbers
valid_phones_df = df[df['is_valid_phone']].copy()
invalid_phones_df = df[~df['is_valid_phone']].copy()

# Add zero in front of each valid phone number
valid_phones_df['CUST_CURRENT_PHONE__C'] = '0' + valid_phones_df['CUST_CURRENT_PHONE__C']

# Set the CUST_CURRENT_PHONE__C column to empty for invalid phone numbers
invalid_phones_df['CUST_CURRENT_PHONE__C'] = ''

# Save the dataframes to CSV files
valid_output_file_path = 'valid_phones.csv'
invalid_output_file_path = 'invalid_phones.csv'
combined_output_file_path = 'combined_phones.csv'

valid_phones_df.to_csv(valid_output_file_path, index=False)
invalid_phones_df.to_csv(invalid_output_file_path, index=False)

# Combine valid and invalid phone dataframes for the combined file
combined_df = pd.concat([valid_phones_df, invalid_phones_df])
combined_df.to_csv(combined_output_file_path, index=False)

print(f"Valid phone numbers saved to: {valid_output_file_path}")
print(f"Invalid phone numbers saved to: {invalid_output_file_path}")
print(f"Combined file saved to: {combined_output_file_path}")


Valid phone numbers saved to: valid_phones.csv
Invalid phone numbers saved to: invalid_phones.csv
Combined file saved to: combined_phones.csv


# Combine all Tickets

In [1]:
import os
import pandas as pd

# Set the directory containing your CSV files
folder_path = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\Bricare Case Closed PT 1"

# Create an empty list to store DataFrames
dfs = []

# Loop through all the CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read each CSV file with a fallback encoding and specify the delimiter
        try:
            df = pd.read_csv(file_path, encoding='utf-8', delimiter=';')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=';')  # or 'latin1'
        
        # Check if 'Ticket_ID' column exists
        if 'Ticket_ID' in df.columns:
            # Extract the Ticket_ID column and store it in the list
            dfs.append(df[['Ticket_ID']])
        else:
            print(f"'Ticket_ID' column not found in file: {file_name}")

# Concatenate all DataFrames in the list
if dfs:  # Proceed if there are any DataFrames to concatenate
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Save the combined Ticket_IDs to a new CSV file
    combined_df.to_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\combines_ticketID2.csv", index=False)
else:
    print("No valid 'Ticket_ID' columns found in any of the files.")


'Ticket_ID' column not found in file: ._bricare_20240101_20240807_0_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_1000001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_1200001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_1400001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_1600001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_1800001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_200001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_400001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_600001_79_closed.csv
'Ticket_ID' column not found in file: ._bricare_20240101_20240807_800001_79_closed.csv


ParserError: Error tokenizing data. C error: Expected 1 fields in line 1618, saw 3


# To find all common values

In [18]:
import pandas as pd

# Load the files into pandas DataFrames
# file1_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\New folder\combines_ticketID.csv")  # Replace 'file1.csv' with your actual file path
# file2_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_check_closed24.csv", delimiter=';')  # Replace 'file2.csv' with your actual file path

file1_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\combines_ticketID - 26 no status.csv")  # Replace 'file1.csv' with your actual file path
file2_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_check_closed24.csv", delimiter=';')  # Replace 'file2.csv' with your actual file path

# Filter the DataFrame from File 1 to include only those rows where Ticket_ID is also in File 2
filtered_df = file1_df[file1_df['Ticket_ID'].isin(file2_df['Ticket_ID'])]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\sf_open_bricare_closed2.csv", index=False)  # Replace 'filtered_file1.csv' with your desired file name
filtered_df


,Ticket_ID
0,TTB000049844098
1,TTB000051015925
2,TTB000051591293
3,TTB000051845825
4,TTB000051965050
5,TTB000050134135
6,TTB000050160303
7,TTB000052956480
8,TTB000053067022
9,TTB000053327530


# To find common values 2

In [57]:
import pandas as pd
import os

# Define the paths for the files
file1_path = r"D:\extract_new_status_done_onlyTTB.csv"
folder_path = r"D:\Python2\case_2024\7 aug\update to closed\bricare_all_update_status_clean" # Replace with the actual folder containing multiple CSV files

# Load the first file into a pandas DataFrame
file1_df = pd.read_csv(file1_path)
file1_df

# Iterate over each file in the folder (for this example, just the uploaded file)
for filename in os.listdir(folder_path):
    if filename.startswith("bricare") and filename.endswith(".csv"):  # Process only bricare CSV files
        file2_path = os.path.join(folder_path, filename)
        file2_df = pd.read_csv(file2_path)

        # Filter the DataFrame from File 1 to include only those rows where Ticket_ID is also in File 2
        filtered_df = file1_df[file1_df['Ticket_ID'].isin(file2_df['Ticket_ID'])]

        # Save the filtered DataFrame to a new CSV file, named after the original File 2
        output_filename = f"sf_open_bricare_closed2_{filename}"
        output_path = os.path.join(folder_path, output_filename)
        # filtered_df.to_csv(output_path, index=False)

# Display the last processed DataFrame (if you want to check the result)
filtered_df



C:\Users\maste\AppData\Local\Temp\ipykernel_27896\1898664671.py:16: DtypeWarning: Columns (15,47,63) have mixed types. Specify dtype option on import or set low_memory=False.
  file2_df = pd.read_csv(file2_path)


,Ticket_ID


# To serach all relevant information based on Ticket_ID

In [23]:
import pandas as pd
import os
import glob

# Load the sf_open_bricare_closed file
# sf_open_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_nomer_kartu_ttb.csv", encoding='ISO-8859-1')  # Adjust encoding if needed
sf_open_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\combines_ticketID - 26 no status - 2.csv")  # Adjust encoding if needed

# Specify the folder containing the bricare files
folder_path = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\bricare_all_closed_batch2_2024"  # Replace with the folder containing your bricare files
# output_file = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\no_kartu.csv"  # Replace with your desired output file path
output_file = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\to_close2.csv"  # Replace with your desired output file path

# Initialize an empty list to store the filtered DataFrames
filtered_dfs = []

# Iterate over all files matching the pattern 'bricare_*_closed.csv'
for file_name in glob.glob(os.path.join(folder_path, 'bricare_*_closed.csv')):
    try:
        # Load the current bricare file
        bricare_df = pd.read_csv(file_name, encoding='ISO-8859-1')  # Adjust encoding if needed
        
        # Filter the DataFrame to include only rows where Ticket_ID exists in sf_open_df
        filtered_df = bricare_df[bricare_df['Ticket_ID'].isin(sf_open_df['Ticket_ID'])]
        
        # Append the filtered DataFrame to the list
        filtered_dfs.append(filtered_df)
    except UnicodeDecodeError:
        print(f"Could not read file {file_name} due to encoding error. Skipping this file.")

# Concatenate all filtered DataFrames into a single DataFrame
combined_filtered_df = pd.concat(filtered_dfs, ignore_index=True)

# Save the combined filtered DataFrame to a new CSV file
combined_filtered_df.to_csv(output_file, index=False)

# Optional: Display the combined data to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Combined Filtered Bricare Data", dataframe=combined_filtered_df)


ParserError: Error tokenizing data. C error: Expected 3 fields in line 43, saw 7


# To add column ID on the left

In [6]:
import pandas as pd

# Load the to_close and extract_check_closed24 files into pandas DataFrames
to_close_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\to_close.csv")
extract_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_check_closed24.csv", delimiter=';')

# Merge the DataFrames on the Ticket_ID column to add the ID column from extract_df to to_close_df
merged_df = pd.merge(to_close_df, extract_df[['Ticket_ID', 'ID']], on='Ticket_ID', how='left')

# Filter the merged DataFrame to keep only rows where the status column is 'Closed'
filtered_df = merged_df[merged_df['status'] == 'Closed']


# Save the merged DataFrame to a new CSV file
output_file = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\merged_to_close_with_ID2.csv"
filtered_df.to_csv(output_file, index=False)




C:\Users\maste\AppData\Local\Temp\ipykernel_34960\4143017330.py:4: DtypeWarning: Columns (17,23,36,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  to_close_df = pd.read_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\to_close.csv")


# To fix no kartu and no rekening

In [17]:
import pandas as pd

# Load the CSV file with the correct delimiter
file_path = r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_nomer_kartu.csv"
data = pd.read_csv(file_path, delimiter=';')

# Convert the card numbers to text format, ensuring no scientific notation
data['SCC_CARD_NUMBER__C'] = data['SCC_CARD_NUMBER__C'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else '')

# Convert the account numbers to text format, ensuring no scientific notation
data['SCC_ACCOUNT_NUMBER__C'] = data['SCC_ACCOUNT_NUMBER__C'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else '')

# Display the first few rows of the corrected dataset
data.to_csv(r"D:\Python2\case_2024\bricare_all_closed_batch2_2024\extract_nomer_kartu_fixed.csv", index=False)


# To fix no telp in account

In [1]:
import pandas as pd

# Load the CSV file with the correct delimiter ';'
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\bricare_case_account_missing.csv"
data = pd.read_csv(file_path, delimiter=';')

# Function to clean the phone numbers
def clean_phone_number(phone_number):
    if pd.isna(phone_number):
        return phone_number  # Return NaN as is
    
    # If the number is in scientific notation, convert it to an integer
    if isinstance(phone_number, float):
        phone_number = str(int(phone_number))
    else:
        phone_number = str(phone_number)
    
    # Remove any non-numeric characters
    phone_number = ''.join(filter(str.isdigit, phone_number))
    
    # Add a leading zero if the phone number doesn't start with '0'
    if not phone_number.startswith('0'):
        phone_number = '0' + phone_number
    
    return phone_number

# Apply the function to the 'No_telp' column
data['No_telp'] = data['No_telp'].apply(clean_phone_number)

# Display the first few rows of the cleaned data
print(data.head())

# Optionally, save the cleaned data to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\bricare_case_account_missing_clean.csv"
data.to_csv(output_path, index=False)


     cifno       No_telp
0  TFI1251  085230254781
1  SJNC508  081390357064
2  MADX789  082363303463
3  IMF8791  082114456934
4  AKDX781  085322820709


# Clean Data

In [ ]:
# Full code to clean and adjust the dataset

import pandas as pd
import re

# Load the dataset with the correct delimiter
new_data_cleaned = pd.read_csv(new_file_path, delimiter=';', error_bad_lines=False)

# Drop columns that are entirely empty
new_data_cleaned = new_data_cleaned.dropna(axis=1, how='all')

# Function to adjust Ticket_ID to 12-digit format
def adjust_ticket_id(ticket_id):
    if len(ticket_id) == 13:  # TTB followed by 10 digits
        return f"TTB0000{ticket_id[3:]}"  # Pad with zeros to make 12 digits after TTB
    return ticket_id

# Apply the adjustment function to the Ticket_ID column
new_data_cleaned['Ticket_ID'] = new_data_cleaned['Ticket_ID'].apply(adjust_ticket_id)

# Change the status to "Escalated"
new_data_cleaned['status'] = 'Escalated'

# Convert date columns to datetime format
date_columns = ['Create_Date', 'tanggalTransaksi', 'Modified_Date', 'TanggalClosed']  # Adjust based on available columns
for col in date_columns:
    if col in new_data_cleaned.columns:
        new_data_cleaned[col] = pd.to_datetime(new_data_cleaned[col], errors='coerce')

# Fill missing values for specific columns (if required)
new_data_cleaned = new_data_cleaned.fillna({'Jenis_Laporan': 'Unknown', 'email': 'no_email@example.com'})

# Save the cleaned and updated data to a new CSV file
cleaned_file_path = '/mnt/data/cleaned_bricks_data_final.csv'
new_data_cleaned.to_csv(cleaned_file_path, index=False)

cleaned_file_path


In [26]:
import pandas as pd


path=r"D:\extract_new_status.csv"


df=pd.read_csv(path)
df=df[df['STATUS']=='New']
df = df[(df['SCC_CASE_OWNER__C'] == 'Migration') | (df['SCC_CASE_OWNER__C'] == 'migration')]
df.to_csv(r"D:\extract_new_status_done.csv")

In [41]:
import pandas as pd

path=r"D:\extract_new_status_done.csv"
path2=r"D:\extract_new_status_done_onlyTTB.csv"
df=pd.read_csv(path)
# df=df.drop(columns=['Unnamed: 0.1'])
df.to_csv(r"D:\extract_new_status_done.csv", index=False)
df=df['SCC_LEGACY_TICKET_ID__C']

# df.to_csv(path2, index=False)

# Search for a Ticket

In [3]:
import pandas as pd


import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the Ticket_ID to search for
search_id = 'TTB000054253533'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the Ticket_ID is present in the 'Ticket_ID' column
        if search_id in df['Ticket_ID'].values:
            print(f"Ticket_ID {search_id} found in file: {filename}")
            break
else:
    print(f"Ticket_ID {search_id} not found in any files.")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\618758926.py:19: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\618758926.py:19: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\618758926.py:19: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\618758926.py:19: DtypeWarning: Columns (15,46,47,51,53,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\618758926.py:19: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Specify dtype opt

Ticket_ID TTB000054253533 found in file: bricare_20240101_20240807_3600001_79_closed.csv


In [12]:
import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the path to the file containing the list of Ticket_IDs
# ticket_id_file = r"D:\extract_new_status_done_onlyTTB.csv"


ticket_id_file = r"D:\escalated_havingcloseddate.txt"

# Define the output CSV file
output_file = 'found_ticket_ids_2.csv'

# Determine the file type and read the Ticket_IDs
if ticket_id_file.endswith('.txt'):
    with open(ticket_id_file, 'r') as file:
        ticket_ids = [line.strip() for line in file.readlines()]
elif ticket_id_file.endswith('.csv'):
    df_ticket_ids = pd.read_csv(ticket_id_file)
    ticket_ids = df_ticket_ids['Ticket_ID'].tolist()
else:
    raise ValueError("Unsupported file type. The file must be either a .txt or .csv.")

# List to collect the results
results = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if any of the Ticket_IDs are present in the 'Ticket_ID' column
        found_ids = df[df['Ticket_ID'].isin(ticket_ids)]
        
        if not found_ids.empty:
            for ticket_id in found_ids['Ticket_ID']:
                results.append({'Ticket_ID': ticket_id, 'File': filename})

# Convert the results list to a DataFrame
df_results = pd.DataFrame(results)

# Save the results to a CSV file
df_results.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\239589948.py:35: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\239589948.py:35: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\239589948.py:35: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\239589948.py:35: DtypeWarning: Columns (15,46,47,51,53,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\239589948.py:35: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Specify dtype opt

Results saved to found_ticket_ids_2.csv


# Collect all information

In [7]:
import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the path to the file containing the list of Ticket_IDs
ticket_id_file = r"D:\extract_new_status_done_onlyTTB.csv"

# Define the output CSV file
output_file = 'found_ticket_ids_with_details.csv'

# Determine the file type and read the Ticket_IDs
if ticket_id_file.endswith('.txt'):
    with open(ticket_id_file, 'r') as file:
        ticket_ids = [line.strip() for line in file.readlines()]
elif ticket_id_file.endswith('.csv'):
    df_ticket_ids = pd.read_csv(ticket_id_file)
    ticket_ids = df_ticket_ids['Ticket_ID'].tolist()
else:
    raise ValueError("Unsupported file type. The file must be either a .txt or .csv.")

# List to collect all rows with found Ticket_IDs
all_found_data = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if any of the Ticket_IDs are present in the 'Ticket_ID' column
        found_ids = df[df['Ticket_ID'].isin(ticket_ids)]
        
        if not found_ids.empty:
            found_ids['Source_File'] = filename  # Add a column indicating the source file
            all_found_data.append(found_ids)

# Concatenate all found data into a single DataFrame
df_all_found = pd.concat(all_found_data, ignore_index=True)

# Save the collected data to a CSV file
df_all_found.to_csv(output_file, index=False)

print(f"All found data saved to {output_file}")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2796145597.py:32: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2796145597.py:32: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2796145597.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  found_ids['Source_File'] = filename  # Add a column indicating the source file
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2796145597.py:32: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. Speci

All found data saved to found_ticket_ids_with_details.csv


In [8]:
import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the path to the exclusion file
exclusion_file = r"D:\extract_ttb24.csv"

# Define the output CSV file
output_file = 'remaining_tickets.csv'

# Load the exclusion list from the exclusion file
df_exclusion = pd.read_csv(exclusion_file)
excluded_tickets = df_exclusion['Ticket_ID'].tolist()

# List to collect all rows with Ticket_IDs not in the exclusion list
all_remaining_data = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename != os.path.basename(exclusion_file):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Filter rows where Ticket_ID starts with 'TTB' and is not in the exclusion list
        remaining_tickets = df[df['Ticket_ID'].str.startswith('TTB') & ~df['Ticket_ID'].isin(excluded_tickets)]
        
        if not remaining_tickets.empty:
            remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
            all_remaining_data.append(remaining_tickets)

# Concatenate all remaining data into a single DataFrame
df_all_remaining = pd.concat(all_remaining_data, ignore_index=True)

# Save the collected data to a CSV file
df_all_remaining.to_csv(output_file, index=False)

print(f"Remaining tickets saved to {output_file}")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2617288852.py:26: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2617288852.py:26: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2617288852.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2617288852.py:26: DtypeWarning: Columns (15,46,47,51,53,59) have mixed type

Remaining tickets saved to remaining_tickets.csv


# TTB createddate up to midnight 6 aug

In [10]:
import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the path to the exclusion file
exclusion_file = r"D:\extract_ttb24.csv"

# Define the output CSV file
output_file = 'remaining_tickets_upto_6aug.csv'

# Load the exclusion list from the exclusion file
df_exclusion = pd.read_csv(exclusion_file)
excluded_tickets = df_exclusion['Ticket_ID'].tolist()

# Define the cutoff date
cutoff_date = pd.to_datetime('2024-08-06 23:59:59')

# List to collect all rows with Ticket_IDs not in the exclusion list
all_remaining_data = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename != os.path.basename(exclusion_file):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Convert the Create_Date column to datetime
        df['Create_Date'] = pd.to_datetime(df['Create_Date'])
        
        # Filter rows where Ticket_ID starts with 'TTB', is not in the exclusion list, and is within the cutoff date
        remaining_tickets = df[
            df['Ticket_ID'].str.startswith('TTB') &
            ~df['Ticket_ID'].isin(excluded_tickets) &
            (df['Create_Date'] <= cutoff_date)
        ]
        
        if not remaining_tickets.empty:
            remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
            all_remaining_data.append(remaining_tickets)

# Concatenate all remaining data into a single DataFrame
df_all_remaining = pd.concat(all_remaining_data, ignore_index=True)

# Save the collected data to a CSV file
df_all_remaining.to_csv(output_file, index=False)

print(f"Remaining tickets saved to {output_file}")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\849014698.py:29: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\849014698.py:29: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\849014698.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\849014698.py:29: DtypeWarning: Columns (15,46,47,51,53,59) have mixed types. S

Remaining tickets saved to remaining_tickets_upto_6aug.csv


# TTB createddate up to 6 aug and closeddate 6 aug 24 midnight

In [11]:
import os
import pandas as pd

# Define the directory containing the files
directory = r"C:\Users\maste\Downloads\bricare_2024_data_clean"

# Define the path to the exclusion file
exclusion_file = r"D:\extract_ttb24.csv"

# Define the output CSV file
output_file = 'remaining_tickets_CC_6aug.csv'

# Load the exclusion list from the exclusion file
df_exclusion = pd.read_csv(exclusion_file)
excluded_tickets = df_exclusion['Ticket_ID'].tolist()

# Define the cutoff date
cutoff_date = pd.to_datetime('2024-08-06 23:59:59')

# List to collect all rows with Ticket_IDs not in the exclusion list
all_remaining_data = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename != os.path.basename(exclusion_file):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Convert the Create_Date and TanggalClosed columns to datetime
        df['Create_Date'] = pd.to_datetime(df['Create_Date'])
        df['TanggalClosed'] = pd.to_datetime(df['TanggalClosed'])
        
        # Filter rows where Ticket_ID starts with 'TTB', is not in the exclusion list,
        # and both Create_Date and TanggalClosed are within the cutoff date
        remaining_tickets = df[
            df['Ticket_ID'].str.startswith('TTB') &
            ~df['Ticket_ID'].isin(excluded_tickets) &
            (df['Create_Date'] <= cutoff_date) &
            (df['TanggalClosed'] <= cutoff_date)
        ]
        
        if not remaining_tickets.empty:
            remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
            all_remaining_data.append(remaining_tickets)

# Concatenate all remaining data into a single DataFrame
df_all_remaining = pd.concat(all_remaining_data, ignore_index=True)

# Save the collected data to a CSV file
df_all_remaining.to_csv(output_file, index=False)

print(f"Remaining tickets saved to {output_file}")


C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2776318400.py:29: DtypeWarning: Columns (15,36,46,47,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2776318400.py:29: DtypeWarning: Columns (15,17,30,33,36,47,49,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2776318400.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_tickets['Source_File'] = filename  # Add a column indicating the source file
C:\Users\maste\AppData\Local\Temp\ipykernel_8912\2776318400.py:29: DtypeWarning: Columns (15,46,47,51,53,59) have mixed type

Remaining tickets saved to remaining_tickets_CC_6aug.csv


In [12]:
import pandas as pd

path=r"D:\dataquality2\remaining_tickets_CC_6aug.csv"

df=pd.read_csv(path)
df=df.iloc[1178:1179]
df.to_csv(r"D:\dataquality2\remaining_tickets_CC_6aug_1line.csv",index=False)

## Generate copies of rows

In [8]:
import pandas as pd


input_file_path = r"C:\Users\maste\Downloads\extract-case.csv"
output_file_path = r"C:\Users\maste\Downloads\extract-case_done.csv"



df = pd.read_csv(input_file_path)


df['APPROVAL_STATUS__C'] = 'Waiting for Checker'
df['OWNERID'] = '00GMR0000000fqe2AA'
copies_per_row = 1  
duplicated_df = df.loc[df.index.repeat(copies_per_row)].reset_index(drop=True)


start_ticket_id = 1642667  
prefix = "TICKET00"

# Create a sequence of ticket IDs
total_rows = len(duplicated_df)
duplicated_df['SCC_LEGACY_TICKET_ID__C'] = [f"{prefix}{start_ticket_id + i}" for i in range(total_rows)]


duplicated_df.to_csv(output_file_path, index=False)

print(f"Duplicated rows with updated 'APPROVAL_STATUS__C' and sequential 'SCC_LEGACY_TICKET_ID__C' saved to {output_file_path}")



Duplicated rows with updated 'APPROVAL_STATUS__C' and sequential 'SCC_LEGACY_TICKET_ID__C' saved to C:\Users\maste\Downloads\extract-case_done.csv


In [5]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_SIT.csv"

df=pd.read_csv(path)
df['EXTERNAL_ID__C']=df['NAME']
df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_SIT_cleaned.csv", index=False)

In [4]:
import pandas as pd

# Load the data from the uploaded CSV files
data1 = pd.read_csv(r"C:\Users\maste\Downloads\data_bricarelamavsbricarebaru\data1_newbricare.csv")
data2 = pd.read_csv(r"C:\Users\maste\Downloads\data_bricarelamavsbricarebaru\data2_bricarelama.csv")

# Compare the 'Ticket_ID' column in both files to find values in data2 that are not in data1
# Assuming 'Ticket_ID' is the column name in both files
missing_in_data1 = data2[~data2['Ticket_ID'].isin(data1['Ticket_ID'])]

missing_in_data1.to_csv("missing_TTB.csv", index=False)

In [12]:
import pandas as pd


send_to_drone = pd.read_csv(r"C:\Users\maste\Downloads\sendtodrone.csv")


exclude_ids = set(send_to_drone['Call_Type_ID'])

big_file = pd.read_csv(r"C:\Users\maste\Downloads\bricare_20240101_20240807_0_79_clean (1)\bricare_20240101_20240807_0_79.csv")
filtered_big_file = big_file[~big_file['Call_Type_ID'].isin(exclude_ids)]
# filtered_big_file.to_csv('filtered_bigfile.csv', index=False)

big_file = big_file[big_file['status'] != 'Closed']
# big_file


# print("Filtering complete. The filtered data is saved in 'filtered_bigfile.csv'.")


C:\Users\maste\AppData\Local\Temp\ipykernel_3884\1456068170.py:9: DtypeWarning: Columns (15,17,36,46,47,51,53,56,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  big_file = pd.read_csv(r"C:\Users\maste\Downloads\bricare_20240101_20240807_0_79_clean (1)\bricare_20240101_20240807_0_79.csv")


# Case type take out only two columns

In [2]:
import pandas as pd


df= pd.read_excel(r"C:\Users\maste\Downloads\Master Case Type Aktif-2024-10-02-15-10-22.xlsx")


df_filtered

c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Case Type: Case Type Number,Deskripsi Case,Total SLA,Level 1 Escalation Team,Level 1 OLA,Level 2 Escalation Team,Level 2 OLA,Level 3 Escalation Team,Level 3 OLA,Level 4 Escalation Team,Level 4 OLA,List Dokumen yang dibutuhkan,Additional Details,Sub Case Reference
0,8634,Merchant Mengajukan Pengembalian Dana (Refund)...,4,FST - Merchant Care & Operation,4.0,NaN,NaN,NaN,NaN,NaN,NaN,"1. Berita Acara, \n2. Notif Transaksi Berhasil...",Store merchant dengan data sbb : \nID...,NaN
1,1000,Nasabah Menanyakan Informasi Pengajuan Terkait...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Information Cara pengajuan Ceria : \nNasabah m...,"1. Cara, Syarat & Ketentuan\n2. Status"
2,1002,Nasabah Menyakan Terkait Promo Dan Program Ceria,4,CDD - Strgy&Prod Credit Card&Dgtl Lend,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xxx,NaN
3,1003,Nasabah Mengajukan Pelunasan Awal Cicilan Ceria,5,DDB - Helpdesk,3.0,DDB - Operational - CERIA,2.0,NaN,NaN,NaN,NaN,NaN,xxx,NaN
4,1005,Nasabah Mengajukan Pemblokiran Ceria,1,DDB - Helpdesk,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nasabah mengajukan pemblokiran Sementara Akun ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,8102-2,Nasabah Komplain Terkait Ketidaksesuaian Angsu...,10,SCC - Customer Resolution,1.0,Branch,9.0,NaN,NaN,NaN,NaN,NaN,Nasabah Komplain terkait Saldo Tertahan (Hold ...,1. Saldo Tertahan (Hold Dana)\r\n2. Ketidakses...
378,8102-3,Nasabah Komplain Terkait Data Nasabah Pinjaman,10,SCC - Customer Resolution,1.0,Branch,9.0,NaN,NaN,NaN,NaN,NaN,Nasabah Komplain terkait Data SLIK\r\nNo KTP :...,1. SLIK - Status Kolektabilitas\r\n2. SIKP
379,8102-4,Nasabah Komplain Terkait Sanggahan Kepemilikan...,10,SCC - Customer Resolution,1.0,Branch,9.0,NaN,NaN,NaN,NaN,NaN,Nasabah Komplain terkait Sanggahan Kepemilikan...,NaN
380,8102-5,Nasabah Komplain Terkait Permasalahan Asuransi...,10,SCC - Customer Resolution,1.0,Branch,9.0,NaN,NaN,NaN,NaN,NaN,Nasabah Komplain terkait Permasalahan Asuransi...,NaN


In [22]:
import pandas as pd



# UAT
# file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_UAT.csv"
# df = pd.read_csv(file_path)
# df_duplicated = pd.concat([df] * 1000, ignore_index=True)
# # df_duplicated['OWNERID']='005MR0000005T4bYAE'
# df_duplicated['OWNERID']='005MR0000005FCvYAM'
# df_duplicated=df_duplicated.iloc[:750]
# output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_UAT_test_1line.csv"
# df_duplicated.to_csv(output_file_path, index=False)
# output_file_path



# 8812
# file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_traininguat8812.csv"
# df = pd.read_csv(file_path, dtype={'CUST_CURRENT_PHONE__C': str})
# df_duplicated = pd.concat([df] * 50, ignore_index=True)
# # df_duplicated['OWNERID']='005MR0000005T4bYAE'
# # df_duplicated=df_duplicated.iloc[:750]
# df['CUST_CURRENT_PHONE__C'] = df['CUST_CURRENT_PHONE__C'].apply(lambda x: f"{int(x):0>10}" if pd.notnull(x) else x)
# output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\UAT_training_8812.csv"
# df_duplicated.to_csv(output_file_path, index=False)
# output_file_path
# df_duplicated['CUST_CURRENT_PHONE__C']

#8915
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_traininguat8915_100k.csv"
df = pd.read_csv(file_path, dtype={'CUST_CURRENT_PHONE__C': str})

# df['SCC_AMOUNT__C']
df_duplicated = pd.concat([df] * 50, ignore_index=True)
# df_duplicated['OWNERID']='005MR0000005T4bYAE'
df['CUST_CURRENT_PHONE__C'] = df['CUST_CURRENT_PHONE__C'].apply(lambda x: f"{int(x):0>10}" if pd.notnull(x) else x)
# df_duplicated=df_duplicated.iloc[:750]
output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\UAT_training_8915_amount100k.csv"
df_duplicated.to_csv(output_file_path, index=False)
# output_file_path
df_duplicated['CUST_CURRENT_PHONE__C']



0     085876692752
1     085876692752
2     085876692752
3     085876692752
4     085876692752
5     085876692752
6     085876692752
7     085876692752
8     085876692752
9     085876692752
10    085876692752
11    085876692752
12    085876692752
13    085876692752
14    085876692752
15    085876692752
16    085876692752
17    085876692752
18    085876692752
19    085876692752
20    085876692752
21    085876692752
22    085876692752
23    085876692752
24    085876692752
25    085876692752
26    085876692752
27    085876692752
28    085876692752
29    085876692752
30    085876692752
31    085876692752
32    085876692752
33    085876692752
34    085876692752
35    085876692752
36    085876692752
37    085876692752
38    085876692752
39    085876692752
40    085876692752
41    085876692752
42    085876692752
43    085876692752
44    085876692752
45    085876692752
46    085876692752
47    085876692752
48    085876692752
49    085876692752
Name: CUST_CURRENT_PHONE__C, dtype: object

# Create PDF with size of 1 KB

In [ ]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font('Arial', 'B', 12)

# Add a cell with minimal content to reduce the file size
pdf.cell(0, 10, 'Test PDF for 1KB size')

# Save the PDF to a file
file_path = "small_pdf.pdf"
pdf.output(file_path)

# Check the file size
import os
file_size = os.path.getsize(file_path)

# Display the file size
file_size, file_path


# To Calculate the business day

In [1]:
from datetime import datetime
import numpy as np


def calculate_business_days_detailed(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%m/%d/%Y, %I:%M %p")
    end_date = datetime.strptime(end_date_str, "%m/%d/%Y, %I:%M %p")
    business_days = np.busday_count(start_date.date(), end_date.date())
    
    if business_days == 0 and start_date.date() == end_date.date():
        time_difference = (end_date - start_date).total_seconds() / 3600  
        return f"Same business day, time difference: {time_difference:.2f} hours"
    
    return f"Business days difference: {business_days} days"
start_date_detailed = "7/13/2024, 4:21 PM"
end_date_detailed = "10/10/2024, 10:17 AM"
business_days_detailed_diff = calculate_business_days_detailed(start_date_detailed, end_date_detailed)
print(business_days_detailed_diff)


Business days difference: 63 days


In [3]:
from datetime import datetime

start_date_detailed = "7/13/2024, 4:21 PM"
end_date_detailed = "10/10/2024, 10:17 AM"


date_format = "%m/%d/%Y, %I:%M %p"


start_date = datetime.strptime(start_date_detailed, date_format)
end_date = datetime.strptime(end_date_detailed, date_format)


date_difference = end_date - start_date

print(date_difference)


88 days, 17:56:00


In [9]:
from datetime import datetime, timedelta
import numpy as np

# Function to calculate the difference in business days with hours and minutes accounted for
def calculate_business_days_detailed_with_time(start_date_str, end_date_str):
    # Convert the input strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%m/%d/%Y, %I:%M %p")
    end_date = datetime.strptime(end_date_str, "%m/%d/%Y, %I:%M %p")
    
    # Use numpy's busday_count to calculate business days between the dates (without time)
    business_days = np.busday_count(start_date.date(), end_date.date())
    
    # If the dates are on the same business day, we calculate time difference directly
    if start_date.date() == end_date.date():
        time_difference = (end_date - start_date).total_seconds() / 3600  # in hours
        return f"Same business day, time difference: {time_difference:.2f} hours"
    
    # Calculate remaining hours on the start and end dates using Python's timedelta
    start_of_next_day = datetime.combine(start_date.date(), datetime.min.time()) + timedelta(days=1)
    end_of_previous_day = datetime.combine(end_date.date(), datetime.min.time())
    
    remaining_start_day_hours = (start_of_next_day - start_date).total_seconds() / 3600  # in hours
    remaining_end_day_hours = (end_date - end_of_previous_day).total_seconds() / 3600  # in hours
    
    # Total business days with the remaining hours of the first and last day
    return f"Business days: {business_days}, additional hours: {remaining_start_day_hours + remaining_end_day_hours:.2f}"

# Example usage
start_date_detailed = "09/24/2024, 3:02 PM"
end_date_detailed = "10/07/2024, 10:17 AM"
business_days_detailed_diff = calculate_business_days_detailed_with_time(start_date_detailed, end_date_detailed)
print(business_days_detailed_diff)


Business days: 9, additional hours: 19.25


In [4]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Load your stock data (e.g., CSV file with historical prices)
# For example purposes, let's assume you have a DataFrame `df`
# where 'Date' is the index and columns include 'Open', 'High', 'Low', 'Close', 'Volume'

# Example: Load a dataset (assuming you have 'stock_data.csv')
# df = pd.read_csv('stock_data.csv')

# For simplicity, let's simulate some financial data
data = {
    'Open': [100, 102, 101, 103, 102],
    'High': [102, 103, 102, 104, 103],
    'Low': [99, 101, 100, 102, 101],
    'Close': [101, 102, 102, 103, 103],
    'Volume': [1000, 1500, 1200, 1300, 1400]
}
df = pd.DataFrame(data)

# The target variable is the 'Close' price of the next day
df['Next_Close'] = df['Close'].shift(-1)

# Drop the last row as it has NaN in 'Next_Close'
df = df.dropna()

# Define features (Open, High, Low, Volume) and target (Next_Close)
X = df[['Open', 'High', 'Low', 'Volume']]
y = df['Next_Close']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting Regressor model
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
gbr.fit(X_train, y_train)

# Make predictions
y_pred = gbr.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Display the predictions vs actual values
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)


Mean Squared Error: 0.9999822924791344
   Actual   Predicted
1   102.0  102.999991


In [1]:
import pandas as pd

df=pd.read_csv(r'c:\Users\maste\Downloads\dataloader_v60.0.2\extract_overlevel2 di SIT.csv')
df['CREATEDDATE']='2024-09-14T02:30:10.000Z'
df['LASTMODIFIEDDATE']='2024-09-14T17:00:04.000Z'
df.to_csv('test_level2.csv', index=False)

# df=pd.read_csv(r"D:\dataquality2\test_level2.csv")
# df['CREATEDDATE']